In [7]:
import numpy  as np


pos = np.load(r'/media/ncclab306/database7/wangm/simnibs_leadfield/pos_hcp4_2.npy')
lfm = np.load(r'/media/ncclab306/database7/wangm/simnibs_leadfield/lfm_hcp4_2.npy')


In [6]:
import numpy  as np
model = '2'
label ='hippo'
TARGET_POSITION =  [-31, -20, -14] # dlpfc   [-39, 34, 37] # hippo [-31, -20, -14]
position = TARGET_POSITION
distance = np.zeros(len(pos))
for i in range(len(pos)):
    distance[i] = (pos[i, 0] - position[0])**2 + (pos[i, 1] - position[1])**2 + (pos[i, 2] - position[2])**2

AVOID_POSITION = np.where(distance < 10**2)
AVOID_POSITION = AVOID_POSITION[0]
print(len(AVOID_POSITION))
print(len(AVOID_POSITION)/len(pos))


NUM_ELE = 75
def tis_function6_focality(x):
    #print(x)
    eam = np.zeros(len(pos))
    electrode1 = int(round(x[2] * 74))
    electrode2 = int(round(x[3] * 74))
    stimulation1 = np.zeros(NUM_ELE)
    stimulation1[electrode1] = 2 * x[0]
    stimulation1[electrode2] = -(2 * x[0])
    e1 = np.array([np.matmul(lfm[:, :, 0].T, stimulation1), np.matmul(lfm[:, :, 1].T, stimulation1),np.matmul(lfm[:, :, 2].T, stimulation1)]).T /1000
    #print(np.max(e1))
    electrode3 = int(round(x[4] * 74))
    electrode4 = int(round(x[5] * 74))
    print(electrode1,electrode2,electrode3,electrode4,x[0],x[1])
    stimulation2 = np.zeros(NUM_ELE)
    # stimulation2[electrode3] = 1.5 - x[0]
    # stimulation2[electrode4] = -1.5 + x[0]
    stimulation2[electrode3] = 2 * x[1]
    stimulation2[electrode4] = -(2 * x[1])
    e2 = np.array([np.matmul(lfm[:, :, 0].T, stimulation2), np.matmul(lfm[:, :, 1].T, stimulation2),np.matmul(lfm[:, :, 2].T, stimulation2)]).T / 1000
    for i in range(len(e1)):
        l_x = np.sqrt(e1[i].dot(e1[i]))
        l_y = np.sqrt(e2[i].dot(e2[i]))
        point = e1[i].dot(e2[i])
        cos_ = point/(l_x*l_y)
        #print(cos_)
        if cos_ <= 0:   # one of the fields must be flipped when a > 90 degrees
            e1[i] = -e1[i]
            cos_ = -cos_
        if l_y < l_x:
            if l_y < l_x * cos_:
                eam[i] = 2 * l_y
            else:
                eam[i] = 2 * np.linalg.norm(np.cross(e2[i], (e1[i] - e2[i]))) / np.linalg.norm(e1[i] - e2[i])
        else:
            if l_x < l_y * cos_:
                eam[i] = 2 * l_x
            else:
                eam[i] = 2 * np.linalg.norm(np.cross(e1[i], (e2[i] - e1[i]))) / np.linalg.norm(e2[i] - e1[i])
    return eam[AVOID_POSITION],eam

x = np.array([
   5.000000000000000000e-01,5.000000000000000000e-01,9.729729729729730270e-01,4.189189189189189144e-01,1.000000000000000000e+00,2.702702702702702853e-01

])

a,b = tis_function6_focality(x)

np.save("checklzm/all_ti_"+model+label+".npy",b)
np.save("checklzm/target_ti"+model+label+".npy",a)
print("done")

def tdcs_function_vol(s):
    eam = ((np.matmul(lfm[:, :, 0].T, s)) ** 2 + (np.matmul(lfm[:, :, 1].T, s)) ** 2 + (
        np.matmul(lfm[:, :, 2].T, s)) ** 2) ** 0.5
    return eam[AVOID_POSITION]/1000,eam/1000

x = np.array([

1.490204886979165644e-03,-1.460028571366719929e-03,-3.924420604320202899e-04,-5.510632545101522631e-03,-9.637223596885083343e-04,1.769916222570666609e-03,-1.705831705967379708e-03,2.221643953279339723e-04,-6.573259627602086319e-04,1.000000000000000000e+00,-6.520375910046092433e-03,7.158251282862737597e-04,5.803206638852516017e-03,-1.336267728804199639e-03,9.355410727345820493e-03,-3.482269756490595503e-03,-3.067211983075052162e-04,-8.777592575047316048e-04,-3.746265191219002796e-03,1.839544885574084798e-03,1.000000000000000000e+00,-1.662667000924170080e-03,-6.098391705588325495e-03,5.434112019259140780e-04,-3.873520810545455495e-04,-2.636414667479713980e-03,7.081989479438388635e-03,3.189457481672861028e-03,1.074001604464053239e-03,5.851281448626934056e-03,-2.007033517530613931e-03,4.690875820112483732e-03,3.684999749573869554e-04,4.205499922155173098e-04,3.542826660179021481e-03,-1.256986355395627725e-03,3.526953776962295922e-03,1.573114103693776245e-03,6.831320137487932598e-04,-1.792615193978316923e-03,4.048304837818858526e-03,9.688527406418769028e-05,1.340309624176666548e-04,-2.039440031070368503e-03,-2.349078938365094932e-03,-4.191364790953678796e-03,-2.033361087976422177e-03,2.633870700032171080e-03,-3.933441879354043769e-03,-7.192465507437906247e-04,-4.193661837015374982e-04,-3.915269838553274429e-03,-2.720819234151555105e-03,1.629728438782618305e-03,2.517430750802698847e-03,-5.140520535721972845e-03,-6.025794449619620297e-04,-4.870496431410464844e-03,-1.880876951510082606e-03,3.509326693500493255e-04,1.997323156914587727e-03,5.262741855897520860e-05,3.633328252892857602e-03,2.317140141261261888e-03,2.693449166772429737e-03,-2.524819051908541029e-03,-3.883916694825753635e-03,3.597926826491711430e-03,1.548735902575412715e-03,3.408831168566987813e-03,-4.802533378094955840e-04,2.083619163511171712e-03,-9.996563815199441816e-01,-5.340963968339133491e-04,-1.000000000000000000e+00,
1.288764941345604473e-02,-3.845632612631155745e-02,4.358038500303344775e-02,3.003341383579043872e-02,-6.039645024065204426e-02,-5.204008897086231239e-02,-5.785042021358011155e-02,5.166750395743013174e-02,1.426522031366988207e-02,2.964821496139913304e-02,-4.711760796969440002e-02,8.179641431324927159e-02,1.194586710099134440e-02,3.691163425737918613e-02,5.975872771509439102e-02,3.788457267846866497e-03,3.295076577865002632e-02,-5.879192235164729802e-02,6.775103303710487516e-04,6.308670331568436151e-02,7.111677132255465050e-01,-1.471527670930827258e-01,3.286884395383771890e-02,1.588450752421320493e-03,3.024025472015771585e-02,3.189847809167171977e-02,-4.930903023817190256e-02,4.907609384047294243e-02,-2.426062599871196129e-02,-4.206709923818812563e-02,6.825218918237208054e-02,7.651279749414683501e-03,3.745560126835315029e-03,2.936879812556983729e-02,-1.297476403250133192e-01,4.862485143137116467e-03,-4.683898181102622016e-02,4.347150885675098858e-02,-1.835936209915284972e-02,-1.939674616608517282e-02,-2.239781267862667874e-02,-2.179274693808300536e-02,2.660515632867774261e-02,-7.539441693833718802e-03,1.170352939527938632e-02,2.640434594811904429e-02,4.048751069661256363e-02,1.429180169494586802e-02,6.822419795118501995e-02,-2.484366768088697164e-02,-5.165302354499146792e-02,-5.656622907935360961e-02,-3.343958521195510014e-02,-8.049657679455177878e-03,-1.792858233054699102e-02,-5.004541683648019990e-02,-3.138394640765317795e-02,-1.494725252508425847e-02,-6.745184238001564192e-02,-1.933245946541558777e-03,9.293066190717602648e-02,-2.070336890744237796e-02,-7.202232357657701450e-03,-5.112144918360395779e-02,5.683942702641293920e-02,-2.411934280489368490e-02,-1.184984330528116303e-01,-3.218396700333445853e-03,-9.830446610982059019e-02,-4.917816858162863541e-02,-2.317608945861648895e-02,9.297962691909875121e-02,-6.653752171545590510e-02,3.043970192035331743e-02,-1.572373478064376107e-02,
2.783860705079075834e-03,-5.618109135110500515e-03,3.160932336743794285e-03,-1.095163998530400418e-04,-4.688172627193551434e-03,-4.879208723267282333e-03,-5.292600010216357009e-03,5.242845340473960974e-03,1.122630514545627789e-03,9.311393291996850063e-01,-5.773076576827677886e-03,7.253368917213540851e-03,3.399445203409443006e-03,3.603546279431210815e-03,4.573157356037849813e-03,-3.911613030344936182e-03,4.191571006636772567e-03,6.973836587194637282e-04,-1.997446239941295942e-03,3.686660490207299690e-03,9.865801487379194379e-01,-1.398524561685841883e-02,4.804701994302381513e-04,1.272580579831124672e-03,2.473201547116443688e-03,-8.127248107039596048e-04,-2.955667578117670419e-03,3.012740476200752304e-03,2.018640122846714936e-03,-1.167918846355267548e-03,4.228491394136370263e-03,1.540170899655512841e-03,-1.664627818529110931e-03,4.281619126360099864e-03,-5.567139258566403126e-03,3.508638851102618625e-03,-4.294989921904044348e-03,1.677097086775328899e-03,1.217129516845061495e-03,-3.324586676043714967e-03,-5.367682242562194644e-04,3.250254159588440717e-03,3.146738669259865895e-03,1.163625600351091485e-03,2.857931530757296806e-03,1.935622171245737766e-03,-2.123242865356336079e-03,6.266165828386245640e-04,2.839539744398319326e-03,3.552266999052006158e-03,-4.125665171350927085e-03,-3.598408570859176758e-03,-6.204109542131379038e-03,-1.242914709310198595e-03,-3.894322693088977244e-03,-8.327307038753915813e-03,3.214576231464153638e-04,-1.623434475688793899e-03,-6.398299356883575296e-03,1.864985055647271073e-03,6.132050826283860778e-03,-2.590335602659088465e-05,-8.840565673983898087e-04,-1.217705702841385268e-03,4.051238806923887492e-03,-4.517019235580387385e-03,-1.181871176730946160e-02,7.250414313726326332e-03,-1.006862682512014301e-02,8.048551000311189585e-04,-5.775424399289269539e-04,1.660233130751573305e-03,-9.342980044138151641e-01,3.840448237457008696e-03,-9.301114148746884025e-01,
1.869484617158196499e-02,-2.655283813883301675e-02,3.344429975063665378e-02,4.575280354705699853e-03,-3.985505581767207622e-02,-3.962406748789724931e-02,-3.358911464684496972e-02,3.636259274449515216e-02,2.752370879505005793e-03,9.216472052322041697e-02,-5.099334506117651766e-02,7.403807881876768582e-02,6.986304802049415202e-03,3.283123336291923766e-03,4.775964205785245692e-02,1.883312933184623506e-02,2.495386602651747734e-02,-4.748638430474389632e-02,9.648999679156073511e-03,1.224012867153228927e-02,6.819222185251222479e-01,-1.110487184083383816e-01,1.500489563110790625e-02,1.626097814984546822e-03,5.767500593341729972e-03,1.391515963186673241e-02,-2.923960987219978783e-02,3.973825576223025818e-02,-1.946245437869213030e-02,-1.435299405977399911e-02,5.448742189040770706e-02,-3.361780449470131663e-03,1.556152405064692087e-02,7.047167415996036244e-03,-9.081690535702391376e-02,1.515816472808809504e-02,-2.811598089725826990e-02,1.159709171214186145e-02,-2.938988236926342451e-02,-1.486469489386971037e-02,-5.376331599873147848e-02,-3.914338065246145137e-03,7.754260548288552379e-03,5.229009140332698327e-04,-1.163315076324489099e-02,3.598526004882234008e-02,3.079254014962354638e-02,9.852497618434544885e-03,7.770629287909151806e-02,-9.004703183635071745e-03,-8.263669341219201381e-02,-7.854086300299935319e-02,-1.578229823123363657e-02,-1.149599022890096696e-02,-2.066305595035634532e-02,-3.204365659306580155e-02,-1.423738958570346796e-02,2.072020617659257880e-04,-4.394607412993635498e-02,2.576449094474168172e-02,5.443077264919604641e-02,1.676802534604011213e-02,2.543423479345508983e-02,-2.494168390189048512e-02,3.306948989906367126e-02,-3.121363428123056800e-03,-1.193182230481268369e-01,1.269025948117271166e-02,-9.761813734132422649e-02,-3.318907178659483148e-02,-2.143132333442775411e-03,8.822686622492806330e-02,-6.650961034659014226e-02,8.224392142797002161e-03,-8.473645089894218940e-02,
3.148959289160623010e-02,-2.508014511847912742e-02,2.180903108298478180e-02,-3.934919216122600982e-02,-6.217804746468980764e-02,-6.249627661585891586e-02,-6.965822488207908114e-02,4.639126033376561598e-02,2.662472841420238101e-03,3.811901481643714673e-02,-8.394903345368705250e-02,4.763000759308423648e-02,4.155717467714437063e-02,1.736505457266397656e-03,3.060370193863959365e-03,-1.936028214669675146e-02,3.408240077915216520e-02,-2.306751047073909294e-02,1.659160153085334141e-02,1.046012825008294234e-01,9.412975040145327155e-01,-1.442056640072963414e-01,-2.087908506588288110e-02,4.396886599284088176e-02,5.877205766501615580e-02,-1.588192080606759538e-03,-1.678777440013649869e-02,5.727126168378527532e-02,-8.663490191873355131e-03,-3.852788160386579758e-02,4.522297294313439020e-02,5.457957469436162945e-03,1.046392392087951151e-02,-3.050336116184744831e-03,-9.365904738141123120e-02,1.148018048085352322e-03,-1.790637565147545560e-02,-1.628960953828675201e-02,-6.079166777631296592e-03,-4.399384265383160519e-02,-6.508184355387832201e-03,1.626911773066308520e-02,2.550392964867949711e-02,1.895471810095562468e-02,1.735569421547929897e-02,3.887971638405265939e-02,1.416089380612987371e-02,1.612533692655856304e-02,1.756429825235533704e-02,5.890971928589092826e-03,-5.621444136982661072e-02,-4.981877443573945463e-02,-3.092642902778289932e-02,1.070572357765889847e-02,-4.528315237944219307e-02,-8.277556612549860637e-02,3.712419305930414470e-03,-9.624953811440777746e-03,-5.036323847731434544e-02,-2.180308002694517425e-02,6.642543246240341370e-02,-3.065144378809913500e-02,2.207100880705246959e-02,-3.357071555562028686e-02,5.564926962402259175e-02,-3.556795373788657066e-02,-1.218659559954418797e-01,8.690436197329071133e-02,-1.027817321031753439e-01,1.166693000059611761e-02,-2.142306825067644138e-02,4.044153057171821081e-02,-2.273259947341305376e-01,1.454350886560630898e-02,5.850281296918169449e-03,
2.702122232254930076e-02,-3.061065343427003815e-02,3.241022636528839707e-02,-7.524711915393360838e-03,-4.894179323617386945e-02,-5.588888479003709109e-02,-5.895754719107362818e-02,4.566804436965921088e-02,5.663021755030867542e-03,8.670044769251594730e-03,-7.188768214942375945e-02,6.875588031171360470e-02,2.897742293168090871e-02,1.018225763607047245e-02,3.830392970568846644e-02,-1.325564990597199663e-02,3.871751864216192718e-02,-3.965967719876942882e-02,1.039155160545124704e-02,6.191078736491081869e-02,8.017550707501037710e-01,-1.358873777483037837e-01,-1.618118923038399173e-03,2.183353520954760610e-02,2.268343015892849285e-02,1.162259322052341948e-03,-1.925492285504903922e-02,5.081207678192030397e-02,-1.694119905255433989e-03,-3.215001802491738309e-02,4.394773335847952656e-02,4.836106523416580649e-03,3.813132925121460667e-03,5.242841281265492488e-03,-1.011210353803816547e-01,1.691118745376345156e-02,-2.415539666410993977e-02,-1.264023218887920519e-02,-1.952542482205452123e-02,-3.942176650705091906e-02,-2.348589956901717471e-02,-5.152197355244223648e-03,1.454725816986260356e-02,1.591712029608552056e-03,1.900387561132491077e-03,2.321003147275567738e-02,2.217688543311899241e-02,3.323223089430885589e-03,4.353079434410681015e-02,-8.146837775980724644e-03,-5.858728408060497050e-02,-6.960890947060549561e-02,-3.861633157810133443e-02,6.048332541487967326e-03,-2.718703956447842893e-02,-7.446429975085137876e-02,-1.284219655791524542e-02,-1.467401169041718592e-02,-4.624018585892521183e-02,4.568935563562106367e-03,7.294978365346169169e-02,-2.140642279600341730e-02,2.079731504551028329e-02,-2.419151717867657966e-02,4.067153809887319449e-02,-2.729490631155750621e-02,-1.221995882087806018e-01,5.234663036264591840e-02,-9.927612132292426561e-02,5.372413166888814762e-03,-1.516738592452922252e-02,5.392547091220969285e-02,-1.271465510625504614e-01,2.547272340283746744e-02,3.262178322390085761e-02,
-5.919960931498935430e-03,-3.685820868925870908e-02,3.830302507490719599e-02,-3.710523608699961562e-02,-3.703056038736470684e-02,-2.638479719109498162e-02,-7.245353879245844397e-02,1.611556000646665113e-02,9.850812043177146343e-03,5.942448135599037728e-02,-4.787099087859075786e-02,7.947062156360694407e-02,1.929702585760243588e-02,3.688871230054382305e-02,7.869295465745795859e-02,-8.682484104096666044e-03,4.085856613877714150e-02,-2.008860203843967213e-02,2.639552987091326949e-02,4.693844643931058963e-02,8.623664668328869798e-01,-1.351037684424793439e-01,-3.712530054763359816e-02,2.440849084768959010e-02,-8.338536380086108979e-03,-2.232445080339350019e-02,-5.167685509182115555e-02,6.239405492026352945e-02,-3.016207629031913626e-02,-3.015558673382812138e-02,5.214506691102104019e-02,3.265411494776111756e-02,3.107292014622765192e-02,3.742419825522466659e-03,-7.520608925777676201e-02,1.121192098353115951e-02,-1.763111799722474893e-02,4.781995198608876563e-03,-5.779436656763080327e-02,-7.333852368665916277e-02,-5.595308984253000162e-02,2.927328176389980968e-02,-3.817500103163984360e-02,-7.877606330338016161e-03,-3.334538769905532468e-02,2.966437105117274153e-02,7.260288213636299781e-03,3.679798401812622854e-02,7.121333378654134960e-02,3.626017967455923141e-02,-7.775471773552314336e-02,-8.495615214204671473e-02,-1.642684030933928840e-02,-2.111499856924308610e-03,-2.935631984406598949e-02,-7.374174451444351353e-02,-6.736928010143972134e-05,-2.131116424994802111e-02,-6.706860307270290367e-02,1.557895514001107762e-02,1.003001310118889627e-01,-3.295040007998098486e-02,2.874856281623423826e-02,-2.509240357754414369e-02,9.468447285697623172e-03,-2.714579073553550354e-02,-1.479567560088033418e-01,4.383166416337194726e-03,-1.230221258256987693e-01,-3.852950958597491787e-02,1.527275442711718140e-02,1.041552480354296928e-01,-1.072634917768938434e-01,1.323289965139886476e-02,-2.715354342398132448e-02,
2.996441406302749466e-02,-3.474509726559110029e-02,2.461048241961114741e-02,-1.442487988178469890e-02,-5.849809524245500048e-02,-5.687380743958934948e-02,-6.574716357849139103e-02,4.670843427173126949e-02,1.526651241505383449e-02,7.010278923343590818e-02,-7.698322742869498359e-02,6.818028122520881917e-02,2.650020646993740681e-02,1.615794288961833192e-02,2.877333847825971902e-02,-9.874846939819754629e-04,1.948602750962224323e-02,-4.587073680656238162e-02,2.459056441158002232e-02,5.897769717676210749e-02,7.639118731962371012e-01,-1.564449566286275906e-01,-1.259312412300563760e-03,3.325078812157563485e-02,4.008423835871831320e-02,9.131838593978404417e-03,-3.680205860772765847e-02,4.927318323012563200e-02,-2.406948329283287863e-02,-8.937234294512874655e-03,6.816398489889616275e-02,1.153257836205545846e-02,-1.193653910139931204e-02,5.055788296443423924e-03,-1.195548866814891698e-01,1.223248502121996283e-02,-4.803949900732280842e-02,7.842002365981218248e-03,-1.122676537960779625e-03,-3.777049934872165476e-02,-3.434424785156230814e-02,-8.698979741249057707e-03,4.373012448781828099e-02,2.556870199881343050e-02,1.861525235414159679e-02,5.114205971104675286e-02,4.309643754491781359e-02,1.848028754311403093e-02,6.775882893306396648e-02,-1.852361323376975483e-02,-6.103000915680872884e-02,-6.842621794784342626e-02,-4.176234059771304769e-02,1.624406873625840186e-03,-4.258920184704856338e-02,-7.163537491240036525e-02,-2.972256138909966372e-03,-7.217696747028342838e-03,-4.345149634085495455e-02,5.595405325327120621e-03,8.822068883336271761e-02,-1.084789759151806587e-02,2.382270252570113656e-02,-2.768106599454599887e-02,4.312513293431395611e-02,-1.207779050888998368e-02,-1.308952784639802192e-01,4.761928033102225122e-02,-1.115401191910720818e-01,-3.031214065983709252e-02,-1.536087409618361148e-02,7.963097528210943965e-02,-1.072959571324524630e-01,2.090304985211640168e-02,-4.234358170007923583e-02,
-2.007121750499953061e-02,-3.588251916559679483e-02,1.060079147845699454e-02,-3.658629496183794794e-02,-5.273815017880745942e-02,-5.503547141281919103e-02,-5.225119572971052839e-02,5.417939191392941861e-02,-4.383987555299249939e-03,7.466986529440899911e-02,-5.477423661096328389e-02,5.199280465548863917e-02,8.614184278716650878e-03,3.519381563723367190e-02,4.318240248715771168e-03,2.698402222630306990e-02,1.373438746000147509e-02,-6.577479742648764671e-02,4.397815181404082785e-02,9.231994030914550398e-02,9.411161077772773886e-01,-1.532724181519198037e-01,-2.970986835660889558e-02,3.638086018047251291e-02,3.709103097665941012e-02,-1.398966543900857784e-02,-6.160635238045364392e-02,2.813791333300495745e-02,-4.881333505967758168e-02,-3.597738944673302408e-02,3.683949674302944288e-02,1.061597178355710991e-02,-8.203026079323522124e-04,-1.421588248765762522e-02,-8.525904019679458878e-02,1.839871819879138753e-02,-6.012574644912013766e-02,1.857329797707670677e-02,1.531369587496841933e-02,-2.793598311130049069e-02,-5.841839049150694685e-02,-1.326833470626873762e-02,3.834757892678132551e-02,8.079977077096598542e-03,1.163123767419198609e-02,5.537117901495999805e-02,1.840398818579780382e-03,2.796235293183833848e-02,6.887893319622576116e-02,2.274220582335693963e-03,-5.105723686458574218e-02,-6.843909327607744508e-02,-1.157823361626900138e-02,-2.452264021101732694e-02,-1.427459568672044543e-02,-4.588603623466135095e-02,-1.005125128251274300e-02,1.785451393040739243e-02,-5.566683038215493318e-02,-4.529401579409899140e-03,5.682868129897171439e-02,-2.022818415960742772e-02,2.560489546998228130e-02,-5.898477857622082265e-02,2.497780650551127690e-02,9.847061140039997662e-03,-1.395212376097023088e-01,6.841981339704161780e-02,-1.036457637833161360e-01,-2.894333687437115540e-02,-1.294984213277908247e-03,6.637891852798198655e-02,-1.220802540001277858e-01,4.492246951447245729e-03,-5.302194891315492598e-02,
2.528317128782788664e-02,-3.470890486765836058e-02,3.490330084620506257e-02,-7.307688200767676473e-03,-4.829742486899794851e-02,-8.793342897080665133e-02,-7.635915182785617261e-02,5.469371491618585379e-02,-1.994257355319477287e-02,-2.906000853899826097e-02,-7.164163807517426319e-02,5.863884481075871263e-02,4.512945864332086610e-02,-9.923255729028367805e-03,4.948485476400622807e-03,-2.674387891858864136e-02,1.435491709371519362e-02,-2.285996698212699671e-02,2.689236775291364151e-02,8.999373735372367955e-02,8.870457570027006078e-01,-1.597510160020252368e-01,1.573785710165463009e-02,3.376528043138161961e-02,4.292930038347485266e-02,-7.084118318924827011e-03,-3.860371900247375893e-02,6.199622091333820140e-02,-2.168667834542441913e-02,-5.643454461753941165e-02,4.951072112007483955e-02,2.857999662297321108e-03,3.272006574820287422e-02,-2.678048847949489851e-03,-9.371422834052277329e-02,2.409447309529618581e-02,-2.913297005329237160e-02,-2.340986289353498215e-02,-2.271821250593300881e-02,-4.746533836273991724e-02,-5.170290290409006173e-02,-5.013065098585244711e-03,1.699483712187998749e-02,-2.512811850238907269e-03,-2.686572783028633865e-02,2.768624213515383584e-02,8.934511676576030459e-03,3.427477745132209708e-02,4.515942140984686992e-02,2.659434569598993769e-02,-8.288536785817546271e-02,-8.371798145281095227e-02,-1.748706801294836258e-02,1.364941709001378511e-02,-2.611794730466663814e-02,-1.059175870239280615e-01,1.890636442976274811e-02,-1.682086162295774734e-02,-4.724940042264698381e-02,-1.661809579007672219e-02,6.439135016657673849e-02,-4.712108801203457320e-02,2.372039698591491691e-02,-3.860244565596640531e-02,1.885655878952601297e-02,-5.035402782543613176e-02,-1.120319135187045900e-01,6.120424810736153215e-02,-1.210641307320629440e-01,-6.801185504228932435e-03,-1.596876974564629542e-02,3.977780074347982142e-02,-8.711946146545834768e-02,1.261676914756273624e-02,9.266858523406051873e-02,
2.735858535220659499e-02,-2.457129874251355248e-02,4.047894933243553067e-02,-1.969505981764066438e-02,-6.102861549385717871e-02,-5.922688849860550248e-02,-6.286762673424256542e-02,4.882106438253580594e-02,-2.103076858117168199e-03,6.361285109687506156e-02,-7.646851866203065173e-02,9.057691236027404225e-02,3.524737353573061116e-02,2.007122100563132547e-03,5.005577331315490264e-02,-7.325978029099518285e-04,3.234383021230344962e-02,-3.543808413101465216e-02,1.145939153937151379e-02,3.830260180804628822e-02,8.779781838946710826e-01,-1.545152483313478042e-01,-4.382002702258778332e-03,2.140499537810071351e-02,1.664907224418426396e-02,7.126913303691102125e-05,-2.542511484690710560e-02,5.405517462612226265e-02,-2.140318290422790412e-02,-2.715664920735232427e-02,5.853932314740264986e-02,3.626786677617555715e-03,1.297301086049949120e-03,-3.158852143562627252e-03,-1.028658105325350042e-01,1.147628413757474999e-02,-2.877223050485941055e-02,-5.594936502173970816e-03,-2.764742697454173823e-02,-4.337852498488728448e-02,-3.793919828900809560e-02,-3.108398420212569666e-03,7.882895376944731511e-03,-2.435783827056276721e-03,-1.810573081754354718e-02,2.351230951701419047e-02,1.691275282286904685e-02,2.702209839187164064e-02,6.521388815002274186e-02,-1.265972917640058315e-03,-7.541905741593525614e-02,-7.751716049925277963e-02,-2.740614871512332684e-02,-9.178657633633475338e-03,-3.128708790978711879e-02,-7.559667751204263153e-02,-1.264389101579594873e-03,-9.351603521187549714e-03,-5.051567174185542936e-02,1.347289392172880060e-02,8.274061241005906120e-02,-2.268324817054304804e-02,2.927802477840126677e-02,-3.148387944266160809e-02,3.571525444933068560e-02,-2.980135391661132768e-02,-1.443394791590634774e-01,5.106750292407898073e-02,-1.137667581459596844e-01,-1.675801654172535959e-02,-1.698999354078665688e-02,8.099778869557551497e-02,-1.353860636629288239e-01,1.689530149938760217e-02,-3.447516851803264504e-02,
2.265277396798749676e-02,-1.968502407982467700e-02,1.537986202928383339e-02,-4.760740653021693081e-02,-5.553306211667857761e-02,-6.627786410743322087e-02,-6.303426297460615335e-02,4.121115762436138030e-02,-1.500479362137960987e-03,-1.260752422216124269e-02,-6.974772750376093122e-02,4.900829024124854494e-02,3.285764210135593694e-02,1.083002655118478312e-02,1.404816670213380678e-02,-1.962930637214148344e-02,1.305146847208554783e-02,-1.811711422435566959e-02,2.383329479441783499e-02,6.352688008640561268e-02,8.910711074474674254e-01,-1.321413567363070030e-01,-1.380056473488724955e-02,4.573404870186215804e-02,4.288623945043192748e-02,-2.434529144387014010e-03,-3.220077539338137701e-02,4.185407275498406049e-02,-1.702142068773090239e-02,-1.987829614221640895e-02,6.361626922079610669e-02,4.340935697446712122e-03,-1.170643029145801081e-03,-3.226479092676683147e-03,-8.703351923948132729e-02,1.712443425657014448e-02,-3.930640599504535426e-02,1.040691478752523574e-03,-1.039145129770593923e-02,-3.824046162186960085e-02,-1.506754470655364934e-02,2.829744086267849124e-03,2.862790966687649499e-02,2.779012394122111484e-02,1.653447924645820577e-03,4.203854582416473307e-02,1.449171727999110787e-02,1.266600297170513909e-02,3.364556979550611948e-02,-4.042730476383827806e-03,-5.294677151493420997e-02,-5.095349305904618709e-02,-4.529647033325975847e-02,-1.024664712876571793e-05,-3.939044599919244716e-02,-8.576386171372038514e-02,6.127406574402253364e-03,-1.114489283099402706e-02,-5.026391873903364449e-02,-7.020086421667891610e-03,6.267074577263377633e-02,-1.579898876422803841e-02,1.132237398198278691e-02,-2.223597465437376908e-02,2.246497787051165290e-02,-1.928111058825585783e-02,-1.094813264747773879e-01,7.089404033355918566e-02,-9.945731221212428252e-02,-1.685265963434244563e-02,-1.633478190485173481e-03,4.691245876337143267e-02,-1.334654927905474520e-01,2.327749954375935015e-02,5.577987294245595473e-02,
2.139688403130016181e-02,-1.473994570219521599e-02,-1.339016274719602440e-02,-2.886301924756175252e-02,-3.340726692338909926e-02,-6.293138335599329780e-02,-1.580522941429066131e-02,4.323841675026626058e-02,1.634724850508478475e-03,8.782745426130408872e-02,-6.394745327579377803e-02,6.385802740571527858e-02,9.514833454745337191e-03,-8.414339985981976466e-03,1.099658728450014777e-02,-1.042933271399945627e-02,3.483753257289179395e-02,-4.818112122718272439e-02,7.788534329949279439e-03,3.785285923905663819e-02,9.737474418160257938e-01,-1.048283661288128693e-01,-2.605746596654325009e-02,1.648239216256851269e-02,4.118556404646146751e-02,1.714947807902487270e-02,9.171187918191103794e-03,-5.254708512760136729e-03,-5.956425536456141229e-03,-1.789232361079165501e-02,4.796591876501180018e-02,5.761039719824848082e-03,5.341875632324237511e-03,-2.692995177083205502e-02,-7.758120455058356568e-02,-1.764509552640938089e-02,-4.940304953530097465e-02,1.586980786008470451e-02,2.205648680010008356e-02,-1.786924539416449853e-02,-1.057876454930925525e-02,-1.190147302494634629e-02,4.199376022121840274e-02,1.078391651576925819e-02,1.525087473371913860e-02,1.088539676560851960e-02,1.900920370366634971e-02,7.369680876859061036e-03,2.991897777100863132e-02,-3.426710266062954480e-02,-3.494892501730036127e-02,-4.127487262449702765e-02,-5.653345250247937548e-02,-1.511245589021367047e-02,-3.910217807065062556e-02,-4.863102023907983629e-02,4.737984879873802062e-03,-7.515273971468616754e-03,-2.635061200188318353e-02,-3.712901105372222454e-03,4.732667037370666407e-02,2.051278103234202566e-03,1.535062054806744791e-02,-2.323722729795697425e-02,4.339354925307876976e-02,-1.046401008228506346e-02,-7.842373232029228869e-02,9.121195220972257156e-02,-7.308539465572416738e-02,9.516538533091346841e-03,-2.117629553741542936e-02,3.375206415734133270e-02,-2.163995865334635837e-01,4.519136178924415201e-03,-9.670998734406335839e-02,
3.252664805151399274e-03,-3.057447301004230858e-02,4.546674417243224894e-03,-2.002129898004142636e-02,-1.954887321732324754e-02,-2.656772438168680806e-02,-6.975329015107646635e-02,8.771271645504785663e-03,7.089170245942119641e-03,2.173305610247688313e-01,-6.814446299930414230e-02,1.548383896556533926e-02,1.660679852672842377e-02,3.069447190505273856e-02,1.716953191831755571e-02,-3.296105478311989190e-02,4.316877941891458792e-02,-3.778535920939647685e-02,1.294568273848054757e-02,8.321245191460957535e-02,1.000000000000000000e+00,-7.475878043169489895e-02,-4.970109508164198553e-03,2.726637143979141961e-02,5.094807459736874072e-02,-1.311815548556452202e-02,-2.061619347499986030e-03,4.433594171782635068e-02,-2.705908268862358343e-02,-2.636731830701342927e-02,4.487997900441097893e-02,4.310780423461699901e-02,-1.699154529993464335e-02,1.401950471627616199e-02,-7.455971777160522174e-02,9.297914466155401611e-03,-1.548126499550416327e-02,1.345385421350243221e-02,-2.033409787639683941e-02,-4.645018266503386090e-02,1.050541654013088799e-02,-2.892999788949371376e-05,1.497411804775590494e-02,8.739298980900429836e-03,3.572644770868473041e-02,2.026436457720462436e-02,1.916211592428695640e-02,2.768003866076926972e-03,2.245862027258169191e-02,-1.677825641211400792e-03,-1.216517814403074438e-02,-4.413069176720312015e-02,-3.367041400808114260e-02,2.039154468629814435e-02,-1.345346883948844371e-02,-8.479659113569446016e-02,-3.269379835147919605e-02,-4.351620279789777251e-02,-4.772828403283441157e-02,-1.117013940181230298e-02,5.618824985045826892e-02,-4.509014471072106400e-02,9.875174965976937780e-05,-2.376219068068075382e-02,1.923166908423654120e-02,-1.448014370699581867e-02,-3.046541327437851665e-02,4.722154372616882423e-02,-5.407015071472233425e-02,5.854618032150118306e-03,7.616288475887567772e-03,7.206305840820180497e-03,-2.878034786919601018e-01,2.192036435859382598e-02,-1.649006425301038359e-01,
4.076585037119416499e-02,-1.978346630450074362e-02,3.858674413975758666e-02,-2.667918924855781221e-02,-4.695751445312144196e-02,-4.230697048719455483e-02,-6.339483145105651030e-02,2.093524617271414964e-02,7.667186689776226902e-03,4.878908280724547630e-02,-6.764902105398352961e-02,5.052153270665770551e-02,1.720508528388457054e-02,-8.067327532392670753e-04,2.646282954065776877e-02,5.181851740621152419e-03,-2.228937709057091099e-04,-1.021320415213177249e-02,2.284314501648514184e-02,5.198209802152665981e-02,7.424257507198331885e-01,-1.201512553938096362e-01,1.917106950086564995e-03,5.344119307441880934e-02,4.330156645650064251e-02,8.864612759242940232e-03,-2.517403233737298962e-02,4.066942868797440713e-02,-2.171712749039835660e-02,-3.050673707447899841e-02,5.776882246574255309e-02,2.078763038447518571e-03,-9.537187385004582824e-03,2.507180662916383071e-02,-1.053380559088630403e-01,1.074299285695229714e-02,-3.294945485961695009e-02,4.451384330580341817e-03,-2.199075763285318619e-02,-1.885181600805095267e-02,-1.784552321851935741e-02,1.675325279704466264e-04,1.761428394838123607e-02,1.999800262652464550e-02,2.817075473687424142e-02,4.200609525366201524e-02,3.314769773471615971e-02,1.674214252360087599e-02,3.595433890331378723e-02,-3.820469166414556562e-03,-6.135111888358223131e-02,-5.610264713504722889e-02,-5.441520658599208193e-02,1.223813966352759429e-02,-4.969375622267820092e-02,-6.558432732392015241e-02,-9.922153818265208591e-03,-1.618210304875244976e-03,-3.443645825458078319e-02,2.300153898800498878e-02,6.077975192441509300e-02,-2.520771438311011220e-02,2.232352257785136934e-02,-1.603611896996719882e-02,4.076389077825333385e-02,-1.877833192386636069e-02,-1.132908648805305424e-01,2.592327896928787317e-02,-1.084460430251555918e-01,-2.709866493532878812e-02,-8.433174624523052942e-03,4.747215107441832976e-02,-4.082537903686411784e-02,1.837007370442345078e-02,-4.718930348712091494e-02,
-7.519746131352880725e-03,-1.525276818188581185e-02,8.215216860550069583e-03,-2.570511575448686248e-02,-3.083227507031683495e-02,-2.749510069354052511e-02,-2.634438436158918068e-02,1.854182671525958503e-02,1.411912250494512869e-02,3.484123861208466844e-01,-2.699867899337422827e-02,4.607791225192285622e-02,1.134366094939914804e-04,1.265919743398211117e-02,1.755407258321078912e-02,6.248083549962341054e-03,-4.798299860578517095e-03,-2.856090491457944844e-02,3.038483153407280091e-02,2.169559410304393310e-02,9.909265818662467140e-01,-7.297756939496953532e-02,-1.358457580227243749e-02,2.603579063319476014e-02,1.846354595093640041e-02,-3.534522717512006418e-03,-3.346611123224628448e-02,1.843822265479493766e-02,-4.101502404944654817e-02,-8.223755084953557337e-03,3.547082590421314630e-02,9.694366179803709516e-03,-2.862012557686815714e-03,-8.294220329219198495e-03,-4.790497251337448581e-02,6.711831608929463844e-03,-3.651601189205294640e-02,2.218223151943708032e-02,6.731560718766627094e-03,-1.624339951107170443e-02,-3.760956310274725378e-02,5.866097045288537482e-03,1.779961484600905280e-02,1.445135146756788835e-02,1.235034929521983821e-02,5.137540566222871530e-02,3.100481168466308335e-02,1.695755590108538255e-02,4.073061675442674057e-02,-3.597730945569841549e-03,-3.534624330328692277e-02,-3.062841221585173301e-02,-1.779721126165231815e-02,-1.208176602750183661e-02,-2.468762487567319891e-02,-1.751982212868172240e-02,-7.416414179271932014e-03,1.203155104035247280e-02,-4.048270370761283127e-02,8.555457270311414564e-03,4.190852029821200142e-02,-8.199588749413500716e-03,1.649233728760162582e-02,-2.214036246159913762e-02,1.904785699056374751e-02,1.709627913170019831e-02,-7.098240757685711499e-02,2.857601628869027577e-03,-6.069382736563560393e-02,-5.398298685865517837e-02,-5.287796631451083251e-04,6.850654911592488072e-02,-5.382069581362364241e-01,-1.476754813242151833e-03,-3.219901988572749785e-01,
-1.193114288514207644e-03,-1.816406661772790310e-02,2.022847138894683788e-02,-4.345831653937538841e-02,-5.521380288352834009e-02,-5.498613496705142406e-02,-6.173018324457999612e-02,4.711550659338353819e-02,-1.857621985957898930e-03,1.116700420879046529e-02,-5.609045799882760031e-02,5.404044056449813782e-02,3.601232919823950385e-02,2.406610140488932292e-02,7.264441476348672266e-03,-2.265881941651262665e-02,2.746203081582390682e-02,-2.269945070386234290e-02,1.785957921890254235e-02,9.544041074283936277e-02,9.852823117024396282e-01,-1.334518069508177085e-01,-3.202482010450316552e-02,2.856727175075684036e-02,4.107170694881637218e-02,-1.996861113549864772e-03,-2.762612373514870129e-02,4.596602328682576621e-02,-1.991863747366205348e-02,-4.601203139919982088e-02,3.830442602042806455e-02,1.303947090286486366e-02,6.371207165869141598e-03,-2.043468396550699590e-02,-7.199533543487135479e-02,1.913488799470425272e-03,-2.914514469783303224e-02,-4.113745174673853755e-03,9.772258681317730344e-03,-6.027695130403186441e-02,-1.376861327051834283e-02,4.052019051254927839e-03,1.586217194343314887e-02,1.036468701486597567e-02,4.442784550424301593e-04,1.989176904807677185e-02,-7.487883079304910594e-03,3.077660188185209200e-02,2.164004859481527543e-02,6.019485044076611918e-03,-3.209209034233506674e-02,-3.663964204443317940e-02,-2.655512504570179538e-02,1.348539981964706758e-03,-3.360615399305810769e-02,-7.697979185213320308e-02,9.909134572334084637e-03,-1.325908969924918801e-02,-5.244427572767448170e-02,-3.412452860239478802e-02,7.056642261643672476e-02,-5.264525250437748077e-02,6.775969335942842731e-03,-3.298379423557360607e-02,3.161065036350277357e-02,-3.054695240696126898e-02,-9.567464862111001167e-02,8.212562055340492939e-02,-8.541478342179228966e-02,4.486886884810768750e-03,-2.687661915021060527e-02,3.903963636492738254e-02,-2.390239990370890411e-01,2.512234043163361075e-02,5.297172923472148742e-02,
-3.140654785786041805e-03,-3.078529102455981187e-03,-2.160631996664279589e-02,-6.257573367776891882e-02,-9.704780507317290264e-03,-1.612336626908000836e-02,-3.363537824751263589e-02,5.488275620873420309e-03,-4.221546335576566167e-03,4.744665395569005462e-01,-3.526723851954342848e-02,-1.661848617670222059e-02,1.447622609131514471e-02,3.697824759262401811e-03,-3.008753683815588650e-02,-2.629526672373878535e-02,1.314131200570249278e-02,3.602565260178174425e-03,1.730902523593316061e-02,6.408866782563116260e-02,1.000000000000000000e+00,-2.160454621718891147e-02,-5.373439774758431015e-02,2.399309739447920073e-02,4.017792262255891200e-02,-2.008983213493346215e-02,1.061147960798896304e-02,2.457244000137308315e-02,-1.028837247362903752e-02,-1.579557615668271062e-02,4.935606177216023474e-03,2.292761567759311198e-02,6.441418751180639363e-03,-2.488351873936421599e-02,-1.474937428170813858e-03,-2.041944830094911609e-03,-1.942568975544640714e-03,-2.072614704570735489e-02,1.516739724712370951e-02,-4.052158375991847528e-02,1.382574629134812952e-02,2.300780254079668949e-02,7.192778224162282852e-03,2.020146109216251415e-02,1.481804463950924314e-02,1.092140669214955798e-02,-1.658758498843102350e-02,1.069446090876438951e-02,-1.809713096139607333e-02,1.688050513034521777e-02,-5.401533457854407194e-03,-1.171113838771175100e-02,-1.204688967732389648e-02,1.672328899567503829e-02,-2.602967831069372823e-02,-4.951610017987008450e-02,1.098745419107305561e-02,-1.822176306248748934e-02,-7.220722143612953725e-03,-2.305225142126931387e-02,1.589836065836961604e-03,-3.073093652982031299e-02,5.297731911354587250e-03,6.211742573133029882e-03,-1.867277808189417819e-03,-3.916909997444785289e-03,3.282004073807895383e-04,8.405121161696853249e-02,-2.543460358619172024e-02,3.792660840854658599e-02,5.663989550312165111e-04,-3.738345907601964280e-02,-6.616543798879159777e-01,5.992462278726188446e-03,-4.298806350921016151e-01,
-1.224657655403967038e-02,-1.100637361496923164e-02,1.317929071752751979e-03,-2.717671187268354618e-02,4.539790043556965471e-04,-6.173747515335167163e-03,-1.561870334390067644e-02,-1.239433650283108812e-02,4.513390913111325203e-04,7.032163312976943415e-01,-1.969837842665938046e-02,6.188792759835047563e-03,1.295188779967662226e-02,2.191983082377321757e-03,-6.501166076508618080e-03,-1.271132979325262716e-02,8.994220053941341664e-03,-1.165814945575952694e-02,2.051000107683449702e-02,2.703231351016989123e-02,1.000000000000000000e+00,-1.552921832258081884e-02,-1.834510145426729300e-02,2.021742499123618808e-02,2.750842543225134884e-03,-2.299341544901230966e-02,5.157319250169368346e-03,8.214090138856877665e-03,-9.245219738531961981e-03,-9.377974387347557533e-03,-8.586453746574421175e-04,8.975578836912665587e-03,2.359480673993266317e-03,-5.872510209201601411e-03,-8.824651885622720321e-03,1.480405312976895010e-02,1.780018238449046497e-03,-1.263239335405898570e-02,3.677142850110514533e-03,-1.858339839411426303e-02,-1.022466673940163051e-02,7.718582499639411315e-03,-5.233044955917948160e-03,1.707576902323129642e-02,6.269848454255423816e-03,1.361044245019810139e-02,-3.419540134449776392e-03,-1.334891432671424238e-03,5.178205503682212218e-03,9.514734120792683533e-03,-4.137797499272635737e-03,-2.012883451188872061e-02,-1.196027406888145773e-02,4.871405291040801361e-03,-9.836423297595046217e-03,-2.894410030352460861e-02,4.181652526729025959e-03,-4.871268110470089452e-03,-1.669208341085745562e-02,-8.156722722063108888e-03,3.355318655933532679e-03,-1.064944552786652870e-02,1.370334671660687620e-02,8.290919647095319062e-03,-4.329853114553094268e-03,4.490742422380802901e-03,-1.187381202859802420e-02,3.093590689117897696e-02,-1.537870503744186504e-02,3.345009213511981883e-03,1.781124329401098660e-02,6.847130982737162364e-03,-7.894529422328193657e-01,2.660297285954842106e-03,-6.693921496358881118e-01,
9.458209412124806911e-03,-1.483501707730220352e-02,2.997425334072875008e-02,-3.914912110321770028e-02,-6.307312186126105835e-02,-6.675679108564901154e-02,-5.662073641090305254e-02,5.295041877043691475e-02,6.029238283853819921e-04,-1.358304244682639883e-03,-2.820277057166043974e-02,8.508933845013183506e-02,1.947619562305675373e-02,2.175720143977578555e-02,1.949122965961896936e-02,-6.004640124943512458e-03,2.679262742558382801e-02,-3.040871411935344368e-02,2.168667860060201347e-02,1.099345942487629374e-01,8.449764981727595492e-01,-1.569259063768566054e-01,-2.986532316957099531e-02,1.350019710782280127e-02,1.923013174628903851e-02,2.247774639022053628e-02,-5.499311135523305555e-02,4.943093045949763231e-02,-1.565524366611443022e-02,-7.134213739330791393e-02,4.785975754743408001e-02,4.998916700021949211e-04,4.744709806042832906e-02,-3.399320949057191982e-02,-9.641961910200305685e-02,3.098930438917635484e-03,-4.215730729920066922e-02,1.261765977525053720e-02,-4.071423920490991818e-03,-4.414830828128184920e-02,-3.395571484156242881e-02,1.130438386078233548e-02,1.577428693071560029e-02,-1.294175613309241774e-03,-1.690829993332548797e-02,4.324541013687568936e-02,-4.500210449079763975e-03,2.593357286744828530e-02,5.217782834320099095e-02,1.118480736800336536e-02,-7.314479389070827065e-02,-5.895029875371673306e-02,-1.807791342150638289e-02,-1.526079952366990455e-02,-3.895633185329140658e-02,-5.548219285454140254e-02,1.322236021556837116e-03,1.358505609800052250e-02,-6.911991200387325451e-02,-1.962191608212219412e-02,6.454171619787810177e-02,-2.546747425987570812e-02,2.518071845279520188e-03,-4.447600149986115359e-02,3.873378270097208032e-02,-1.771842944164880168e-02,-1.478464444888351603e-01,7.185176008342747389e-02,-1.229034750140285126e-01,-9.156811949756466992e-03,-3.051117419003174236e-02,8.431195771107061199e-02,-1.355095725787785477e-01,1.931256669659121156e-02,2.038611636896972557e-02,
1.845581909901791717e-03,-7.329292411952769912e-03,2.869057040506321071e-02,-1.561873875441814087e-02,-5.180516417423058906e-02,-4.421379400230227019e-02,-5.652394227983037478e-02,2.960121316142913517e-02,-5.153797299403510557e-03,2.105235807554850513e-01,-2.844522620107919897e-02,6.882850558004942332e-02,3.157145512435602375e-02,1.884722674852012947e-02,2.786223633013368284e-02,-1.770032098873419724e-02,2.526054167415172497e-02,-1.360625850526930858e-02,4.105728312693207593e-03,5.776420362390747437e-02,1.000000000000000000e+00,-7.921361766129948745e-02,-8.290267297761859042e-03,1.160598270770561713e-02,1.214789437883664544e-02,8.259903458421615635e-03,-1.848161201014246469e-02,4.904056354293763464e-02,-1.108126787223293053e-02,-4.402630785643343869e-02,3.308866033475593715e-02,1.017150795248313380e-02,5.922549729570535104e-03,-2.974644778332253409e-02,-5.520768576390941429e-02,-1.668996336778686383e-03,-1.734706912070584084e-02,4.696815293951529802e-03,-7.320564034991983160e-03,-5.424889404826406525e-02,-1.178473792551752719e-02,1.233742893781287348e-02,-1.554377713186019805e-03,-1.035670201903421753e-03,-3.036212384970302103e-03,1.195035323977308717e-02,4.061863839418412142e-04,3.003536639192350491e-02,1.748984659521488072e-02,-5.547540963673953246e-03,-4.986598404443307042e-02,-2.507538018671937041e-02,-1.721619137382377623e-02,8.223447131204271454e-03,-2.783419185802991913e-02,-4.167694447318774642e-02,-3.783462399391611386e-03,1.499773674495344378e-03,-5.508340749049764340e-02,-1.592450556156521299e-02,7.191710276180116046e-02,-4.043887978811860906e-02,-6.670415740860330400e-03,-2.853201723723367694e-02,1.976515254237983660e-02,-2.626310784685730038e-02,-6.816147056765861634e-02,2.183580081541780862e-02,-8.414060274102061476e-02,-1.666108828388438573e-02,-2.169328920911237774e-02,6.513458531705680621e-02,-4.336085523252245499e-01,2.116771663890242711e-02,-1.505073751944473182e-01,
7.112382622744724270e-04,-2.442515369686796070e-03,-4.301696045079031472e-04,3.978697185964309713e-03,-9.787236182442014126e-03,1.882353726568669616e-03,-1.844989616226463958e-03,5.485868421312170562e-03,-2.602554188513480846e-03,9.371962688674624342e-01,6.544694369087999952e-03,-1.046517951541166280e-02,1.141771892392978716e-03,9.938677601734405287e-04,-1.364012321264577998e-02,1.233061306418671565e-02,-1.370711104188311602e-02,4.330968965071325949e-03,8.551914240409852486e-03,8.831743471920644462e-03,9.954924249241829948e-01,-3.228026457865434962e-03,1.153348859330509402e-02,1.018788355894804931e-02,7.084225092675297542e-03,7.362936293990249292e-03,-1.244512552224100177e-02,-3.020162262133129286e-03,5.487622869340680332e-05,-4.577855330103155346e-03,1.808224707613784177e-03,-1.214159967773037474e-02,-7.376795940574276678e-03,4.901833533243020458e-04,-7.825426745386443450e-04,5.564211991105301458e-03,-1.748078716568922386e-03,-9.004006133437429671e-04,9.276101875208622727e-03,5.833515386276928033e-03,-4.140627956860140624e-03,-9.820759579921511939e-03,6.314808844448877023e-03,9.975272734363063720e-03,5.654314003693295364e-03,1.305633316443766123e-02,1.424789589144895285e-03,4.584260589290269083e-03,-1.345660367700673987e-03,3.132049805091196432e-03,5.758958739197891918e-03,1.042025707844363921e-02,7.851944680309117658e-03,2.885394454058920130e-03,-2.179061526807905824e-03,7.575677221509501125e-03,4.365857771624598316e-03,1.005802906614696596e-02,8.584741370009675576e-05,-1.251120948604835664e-02,-5.364744377006576415e-03,-1.744862007011516170e-04,3.159585046612067849e-04,-7.848992551615301128e-03,6.608358528156102127e-03,2.429694859667583251e-03,-1.954818785085507549e-03,2.884212975917288535e-03,-1.145981188331313128e-04,-1.111660101670211565e-02,-9.065076823076731807e-03,2.785509257099992586e-03,-9.251709476437537383e-01,2.265141346075014162e-03,-9.365890784659361978e-01,
1.564559714495611202e-03,-1.660577693504376208e-03,-3.262990985554792625e-03,-2.249888799828510602e-02,-2.011133027457025560e-02,-2.461614177492351535e-02,-2.445785147663433884e-02,1.339322604585183735e-02,5.055230507481328628e-03,6.911583181668106146e-01,-1.491148254121064422e-02,4.208725370622163134e-03,1.517794930684290486e-02,1.017387648674257863e-02,-9.996269911240134032e-03,-1.705500577624400466e-02,9.142350688645071999e-03,-5.548147294839704535e-06,9.652968993153371660e-03,4.837688919586741187e-02,1.000000000000000000e+00,-4.372218753003856428e-02,-1.540532926057109481e-02,1.621990143356473110e-02,2.825199059333266885e-02,5.310915427089918373e-03,-1.222994212084477575e-02,1.866368260330804330e-02,-2.666595731468459490e-03,-1.476429799999353157e-02,1.558638190655323337e-02,7.887383519345146296e-03,1.580824780698643524e-03,-4.148026217297845486e-03,-2.336930733204019284e-02,-7.801449021013954696e-03,-1.267067472077804992e-02,1.782385871337549034e-03,1.101583009631313878e-02,-2.481914136704905377e-02,1.215425060184496982e-02,5.889802788087376145e-05,1.281588467683397910e-02,1.392469491556932656e-02,1.079379314968658549e-02,5.400641659448170845e-03,1.683781451788528109e-03,8.328366477142653754e-03,-5.317173307132333146e-03,3.367036564821917002e-03,3.076144522346764641e-03,3.914389429460464476e-03,-1.671370582997919532e-02,7.374095981663000669e-03,-1.824365835498401381e-02,-3.355632485009896193e-02,5.054131019457468524e-03,-6.783442932318164281e-03,-1.672229516770378302e-02,-2.258287721696239370e-02,1.867786559973997132e-02,-1.903353679637914686e-02,-8.668315270692741881e-03,-1.034722475696275151e-02,1.002975515738676812e-02,-1.159748288504168848e-02,-1.846694268565479691e-02,3.264917997822132378e-02,-2.290682989315262832e-02,8.800127720267253428e-03,-1.261847738649445050e-02,5.852762603494338772e-04,-7.841690645662671155e-01,9.854324092976428015e-03,-6.743137299787633765e-01,
-2.537502864572739950e-02,-2.489453903047669087e-02,2.350358810695622747e-02,-2.273044069125939801e-02,-3.200133199479247847e-02,-2.046029082891974438e-02,-3.827289094781503864e-02,5.980569116700786003e-03,2.983773399382431968e-03,2.420877601178625937e-01,-2.591538342530636668e-02,5.955105682856383364e-02,1.447184681626864644e-02,3.270616201931363098e-02,3.084840868658689228e-02,1.343070062675503357e-02,2.146968365054752886e-02,-2.011837364489423019e-02,3.163991632461356229e-02,4.720264393032178041e-02,1.000000000000000000e+00,-6.022974888917791148e-02,-3.053247158945751258e-02,1.793446342984299224e-02,-7.795694307583839189e-03,-1.746928440950772793e-02,-3.618937358093990420e-02,3.398082559954156145e-02,-2.350161518322653464e-02,-3.040385244804053727e-02,1.298497819099002359e-02,1.471673130485130071e-02,1.727884063954548965e-02,-1.611253679243938863e-02,-4.031279134104878703e-02,8.706258136352139354e-03,-1.495772622131981422e-02,1.372914011965773590e-02,-9.329480623454899924e-03,-3.327193690067335730e-02,-4.552057772218476295e-02,2.745637320662097647e-02,-1.652566829639215418e-02,-5.847236619552232013e-03,-6.761370899435181769e-03,2.998386819445179874e-02,8.170641147791229963e-03,1.752283950989616187e-02,3.448533638997225426e-02,8.916531379908415600e-03,-3.861278465482936850e-02,-4.702810000108803568e-02,-6.334017544137926503e-03,-1.307631704207401144e-02,-8.926676979768261022e-03,-1.412229893847688489e-02,-6.067292599407921039e-03,1.267240695777351912e-02,-4.969699436863968256e-02,-2.175324352850969978e-03,5.306365577192444227e-02,-2.473904713330365990e-02,1.534760907754480208e-02,-2.611905112042300220e-02,7.989514919301988344e-03,-9.108701981684600227e-03,-8.224745754246784446e-02,-1.644996283431510165e-03,-7.522796908726250842e-02,-3.523182173305167142e-02,1.145573878072192464e-02,6.644456314164079447e-02,-4.703082092772349343e-01,6.199101315983121009e-03,-2.007170991346244882e-01,
1.316015221787397715e-03,-1.391837728970441064e-02,1.484910562243719340e-02,-3.228274270521382106e-02,-1.062385050155888189e-02,-3.048003751100557962e-02,-3.841464793992428739e-02,2.419445417028549816e-03,1.978808054092387123e-03,3.902103939636346608e-01,-3.957531017833566811e-02,2.663221581840811286e-02,1.890607772068634423e-02,4.430227787028670657e-03,4.576233330413294678e-03,-2.289204532092434241e-02,1.830853947815576366e-02,-2.700791363352330818e-02,2.479860030899925724e-02,5.943875903274487005e-02,9.764185137324021824e-01,-7.411825400991203794e-02,-1.430747433445618555e-02,2.654781490347981471e-02,2.397885390642966880e-02,-2.303410192811629689e-02,-5.686192557652578264e-03,2.763487408242262106e-02,-1.501779109632795858e-02,-2.823138482107892533e-02,2.195515188072915541e-02,1.215324605146661172e-02,-6.595133172028304347e-05,-1.058940044515829793e-03,-4.975308385868654204e-02,1.987279748778851551e-02,-7.294117880167230034e-03,-1.399910296299083576e-02,-9.727726048446948617e-03,-3.046374846878359063e-02,-1.363153905793961472e-02,2.776292783842664903e-03,8.337747006291126861e-04,1.973856592347585440e-02,1.282397756137524866e-02,3.064472731467897393e-02,5.632893072355166925e-03,-1.063579935055378872e-05,2.052614080318731701e-02,8.933622993607413798e-03,-2.636759536293844380e-02,-4.314046786937593414e-02,-2.628691602797683310e-02,8.548643689605660637e-03,-1.815168492165896916e-02,-6.567329399447247029e-02,-3.555288141635203764e-03,-1.118927899948093940e-02,-4.245955757592981272e-02,-4.619747363514958155e-03,2.349644731301424663e-02,-1.852416271754127666e-02,1.484115202131697751e-02,-5.071961965046542509e-03,7.099659292891795370e-03,8.330318282631157811e-04,-5.385315132830602314e-02,5.007589510839233204e-02,-5.125398822477836291e-02,4.446088400168843324e-03,1.314993226944110791e-02,3.046740175020432040e-02,-5.209644608322473447e-01,1.098109856641479865e-02,-3.363821968101561311e-01,
-3.187709237431994580e-04,-9.590242190771723629e-03,7.119267015048866970e-03,-2.930053890798788030e-02,-8.162276169493059771e-03,-1.474962860602955367e-02,-2.578308201149780501e-02,-2.920023209770245334e-03,1.600376340386676670e-03,7.014499927440348559e-01,-2.447683575359804395e-02,1.273207950229549371e-02,1.647203010807285484e-02,2.890239849997679189e-03,-7.154795645299185866e-03,-1.921645235352244241e-02,9.382548638163505650e-03,-8.834931048704912360e-03,1.668166727111400349e-02,4.499187375760960172e-02,9.970412643694400723e-01,-3.455740088304828928e-02,-1.705056541507859666e-02,2.256386996491435010e-02,1.116328642687941866e-02,-1.602919123750656816e-02,3.565481574740502271e-03,1.510421321743212787e-02,-5.638628861101968939e-03,-2.012896036235497718e-02,6.963270626927949361e-03,8.399701443816111335e-03,4.160128008777977393e-03,-9.146278385292980553e-03,-2.313512646690490254e-02,1.401922970311679506e-02,-9.844086523128244004e-05,-1.438821662513636336e-02,3.508506393361294703e-03,-2.159116162729636870e-02,-5.610773868464005026e-03,7.720368125926945109e-03,2.157537343777304537e-04,1.764550349336199350e-02,9.799301917137499471e-03,1.874105306759823661e-02,-3.611219133735261294e-03,3.923254131298051142e-04,1.542982423087292665e-03,7.954318487411488059e-03,-9.394758604054996576e-03,-2.047401654128523651e-02,-1.684855513976878452e-02,8.178168987451280014e-03,-1.764724593545874062e-02,-3.960296772108169716e-02,4.562003476272561558e-03,-6.700244074749565210e-03,-2.145825650647136679e-02,-1.260871821043459649e-02,1.131224999214811358e-02,-1.784035346675933581e-02,9.477551383704247920e-03,5.708343122104646977e-03,5.566833207494828809e-03,-2.421847235255273137e-03,-2.329469978483249271e-02,4.533780886051573533e-02,-2.722594867208841946e-02,8.463518330533839618e-03,7.503828402775317152e-03,6.756305788459756462e-03,-7.455329452958462388e-01,1.001190175235675862e-02,-6.712843612756127509e-01,
-3.086809621017282124e-03,-6.524111765957756395e-03,-1.058401092413270572e-02,-4.298924764039614710e-02,-1.647620578218757237e-02,-1.713915410292041891e-02,-2.124923178332928375e-02,1.828493310197472702e-02,7.649307556803311078e-03,4.915511524321785619e-01,-2.233382714580078171e-02,-9.492335913286763238e-03,6.556255814759829464e-03,7.674738609472179715e-03,-1.203016310082001550e-02,-5.793987116807074872e-03,-3.241079525767964764e-03,-1.359702925281624369e-02,2.276799913664790537e-02,5.471291308041000695e-02,9.971530470186930062e-01,-6.027644942203734535e-02,-2.653805696561717734e-02,2.627409262405147258e-02,3.340833930981458588e-02,-3.927316735734865533e-03,-1.954445843328794091e-02,1.092513619164989236e-02,-2.409148236314781130e-02,-1.192261849138182006e-02,2.354875123333605505e-02,5.367357966330843153e-03,1.326017896201732785e-02,-5.019992054481619767e-03,-2.290427101081582775e-02,6.965730773973978496e-03,-2.256783421828982561e-02,3.138783645767270411e-03,1.315441093797322172e-02,-2.037559688423362658e-02,-7.517811279300130878e-03,4.152350750353708819e-03,1.848615674153399058e-02,2.836706612598860824e-02,9.563399952103002546e-03,3.359776937768955973e-02,-7.781744232446579507e-03,1.537053227480274578e-02,1.199439872025913692e-02,1.389472343492202666e-02,-1.653018935643256471e-02,-1.482612732695360452e-02,-1.127932435853712043e-02,4.847838781410792122e-03,-2.816081435840899969e-02,-3.414100155803650311e-02,1.324217383357254724e-02,-7.258632039722438117e-03,-2.080913490480507458e-02,-2.155506533535588715e-02,8.573584882875337487e-03,-1.378776695814596526e-02,6.843760706205737065e-03,-1.233503470255184649e-02,1.296508132559026465e-02,9.982539581478964280e-03,-3.890154971199169531e-02,4.931373818346060367e-02,-3.821436132398456365e-02,-7.386781219235640308e-03,-9.264701902422234348e-04,1.065275405289295083e-02,-5.750334215902234147e-01,1.079944364624697839e-03,-4.774314585428389712e-01,
-3.733875566947188196e-03,-1.430592681978082314e-02,9.110271084424768906e-03,-3.304865842820954580e-02,-1.302516292446212129e-02,-2.134633379944005094e-02,-3.082852059450413157e-02,-4.432630718718534243e-03,1.903160831171943866e-03,4.969371146225909919e-01,-3.792695523910251387e-02,2.813061123612979261e-02,2.685339724940322831e-02,4.814445417593253042e-03,1.168017149629595533e-02,-2.035298615334086084e-02,2.566617989239812492e-02,-1.631420472271967459e-02,2.016157087205936935e-02,4.167363806540157201e-02,9.924486705962406452e-01,-4.899689530756785177e-02,-2.919907917546959997e-02,2.612791433434946686e-02,6.044238727054711963e-03,-2.364218365971066174e-02,6.065959895667804788e-03,2.485500362270131117e-02,-7.018613135714891517e-03,-1.479060379014888321e-02,1.003729188836360864e-02,1.612559329081434192e-02,6.578000574563451150e-03,-9.110154346686823862e-03,-3.184258049571801058e-02,8.219588741797536485e-03,-2.129465703493695246e-03,-1.314687698955271065e-02,-8.856508156642232954e-03,-3.488261268072412324e-02,-1.098747380333747747e-02,1.070543558503765473e-02,-1.263257247037710349e-02,1.192669039474693847e-02,-1.521959471091435638e-03,6.623842257536947074e-03,-1.174480971703698723e-04,3.408906741214520926e-03,1.155343546836319450e-02,9.541946485898119418e-03,-1.734468298554463564e-02,-3.748801762857932596e-02,-2.284629860240111263e-02,4.089872507024718690e-03,-1.788436920242415448e-02,-5.469265921332899794e-02,4.081263654473452768e-03,-1.716869980480221700e-02,-3.043932771780689245e-02,-4.443639167814832439e-03,2.976440563629222102e-02,-1.784283301264498120e-02,1.698402731980502497e-02,6.052601494072405719e-03,4.368262836949021732e-03,-1.119773812330484078e-02,-4.446767446088112363e-02,4.387030985538148409e-02,-4.250661408372248540e-02,5.405689036397098857e-03,1.545778071709340631e-02,2.256502391532027099e-02,-5.947906093844685937e-01,6.866070733481688740e-03,-4.563667661701874545e-01,
-8.425173204282446204e-03,-8.394479277999244982e-03,-5.714064592326650871e-03,-3.853666433318631013e-02,-1.365983245853912877e-02,-2.025659377136730319e-02,-3.343813999448685959e-02,1.003403148251626187e-02,2.727366568138628322e-04,4.431265177413649115e-01,-2.064455343716379426e-02,-6.939088578509111649e-03,1.800007355897509029e-02,1.781331457741434920e-02,-2.692057892128388308e-02,-2.496408824779383639e-02,8.164599499911731617e-03,1.182628090031287785e-03,1.862377342564477956e-02,7.661772423868171145e-02,1.000000000000000000e+00,-4.462281709113394279e-02,-3.513811261371427297e-02,3.238317320606225874e-02,2.927816815595292096e-02,-1.179470892407983446e-02,-8.541435826798837727e-03,1.385124726654066163e-02,-3.957422109258871418e-03,-2.724822420113302140e-02,2.510251829326473616e-03,1.802001456206206442e-02,-9.956740672208211351e-04,-1.268592136784297240e-02,-1.542313953570068064e-02,-4.869989728498501889e-03,-1.023309040801514275e-02,-7.433232194741248855e-03,2.314507614013860959e-02,-3.395309894990557187e-02,1.177818050037560994e-02,5.485931003409695583e-03,1.185574755788915136e-02,1.343770423653149748e-02,1.871137545765580099e-02,8.826013012796268126e-03,-1.009231666343619535e-02,1.181339335084230155e-02,-1.496065380600336212e-02,1.048319372141111316e-02,1.789558130088272220e-02,2.348760389926346503e-03,-1.729117764827397402e-02,1.052088408783841134e-02,-1.642774927909331634e-02,-4.748553239699260242e-02,8.222485100575927658e-03,-7.458220991905697983e-03,-1.369764429850781756e-02,-3.037655691997848129e-02,1.837402883732618120e-02,-3.445346902976879205e-02,-2.860331515324165963e-04,-2.933406962430083059e-03,8.939637876713906611e-03,-1.377497148115876258e-02,-1.634787761484054330e-02,5.989229931530827555e-02,-1.895671774990542041e-02,1.887643820296523722e-02,-8.326030939667793118e-03,-1.920416137922202615e-02,-6.088342869129805823e-01,1.350113855809427499e-02,-4.020125672997861832e-01,
2.869463217990937676e-02,-3.306485225661148775e-02,2.231959546388124008e-02,-1.823597055837541253e-02,-5.559429977409197088e-02,-5.493102019396354369e-02,-6.485710311585049515e-02,4.453131223509781272e-02,1.371288378965987084e-02,8.563838985945373961e-02,-7.446288562403748090e-02,6.338047327573356760e-02,2.628118757465902622e-02,1.502459329464558152e-02,2.644355308945730079e-02,-4.273464336950971720e-03,2.041102592845998182e-02,-4.290903338479225976e-02,2.359591641705707094e-02,6.130596129266603633e-02,7.822756103295918084e-01,-1.504334909605466453e-01,-4.866536314576473676e-03,3.299965827002165897e-02,3.976368285285729332e-02,7.176662002391242037e-03,-3.375187242699025092e-02,4.872568870569379124e-02,-2.262120136573624388e-02,-1.109396054468393678e-02,6.488535845681443759e-02,1.261133147573933327e-02,-9.192720472219426467e-03,3.350891904339405383e-03,-1.130977907539091609e-01,1.158709342663854537e-02,-4.456752448620420609e-02,5.363500770944260032e-03,-2.137046160700359354e-03,-3.898838706676154009e-02,-3.088529103462237663e-02,-7.062867169705996234e-03,3.992328860438489752e-02,2.475338550107099300e-02,1.680796547543066816e-02,4.815209983098022589e-02,3.818664067006652396e-02,1.808047635053311453e-02,6.298819347121970003e-02,-1.461316326991566467e-02,-5.883725689089622929e-02,-6.632603883129631339e-02,-3.977166204948184847e-02,2.960934885721626386e-03,-4.187366769386628718e-02,-7.313880819392334920e-02,-2.069099452392007127e-03,-9.144609149170257467e-03,-4.278333348044839612e-02,3.556967814987186532e-03,8.319567249567966227e-02,-1.233377481680254795e-02,2.278305651980304727e-02,-2.558098174833387667e-02,4.052481268335185460e-02,-1.229419553236046209e-02,-1.249827352563211980e-01,5.193339111015118914e-02,-1.075071549772494139e-01,-2.496724341632243493e-02,-1.442091936587600737e-02,7.317047754501110335e-02,-1.350459510295082821e-01,2.041900115932803600e-02,-5.672684820410747730e-02,
-5.047991294307983823e-02,-1.657897953645914510e-02,3.109108994603715023e-02,-8.282784044027616255e-03,-4.163182387962788278e-02,-4.111716228790077837e-02,-3.031741769554726837e-02,2.811139422171125835e-02,-1.753668711848422809e-02,1.804669592114809262e-01,-2.106821058333076258e-04,8.374186206207635819e-02,4.100878683476579661e-03,3.694775420626138923e-02,2.189895941779612665e-02,3.662775032163310962e-02,1.287615964201360014e-02,-6.078489060975308944e-02,3.397076349585035487e-02,5.688423114400353170e-02,9.449192083288185939e-01,-8.553144408530280463e-02,-7.642610262178051173e-03,9.146618491994308597e-03,-9.598209640239000612e-03,-7.953756219653714682e-03,-5.600271667624846100e-02,2.245090244509949373e-02,-4.075689230028931020e-02,-5.083445757432917406e-02,1.433333906180473891e-02,-5.304199500130439560e-03,1.698576773528262529e-02,-4.394367303237339123e-02,-4.680329567689749115e-02,1.982471153836929539e-02,-3.683321279717648167e-02,2.762914096204078379e-02,-1.610928927436861002e-04,-2.703147504698245190e-02,-7.815011025058871130e-02,-1.129278614945140868e-03,-2.234173795627993064e-03,-7.373545845021964307e-03,-8.302146623004880155e-03,4.576720714943514207e-02,-8.254571703126767357e-03,2.461653122607831284e-02,5.692937830885200223e-02,-1.834890145841927556e-03,-6.701894946612679815e-02,-4.888946510011869806e-02,1.225963857213016941e-02,-2.681547886446886131e-02,4.314126666080013786e-03,1.334908345124045936e-02,-2.130084594115240482e-02,4.354264916742205810e-02,-8.092627435762801691e-02,-2.441628151708598315e-03,4.893823954453864478e-02,-1.379723103495006202e-02,6.685066508982063886e-03,-6.221210528777259413e-02,3.437464327839007128e-03,1.805137580693525964e-02,-1.073292166965695987e-01,-1.349809006815527734e-03,-9.470074906779188395e-02,-5.749214676572124427e-02,7.630229282402960916e-03,1.081639880843200119e-01,-2.276909386391238188e-01,1.100884834021824583e-03,-1.363878107997836264e-01,
8.855275300863781063e-03,-1.455503626670267246e-02,2.996541750433633078e-02,-3.914448940832965934e-02,-6.300740171291874248e-02,-6.660015856392224876e-02,-5.620113222629019578e-02,5.286649368661230919e-02,5.589059266806264969e-04,-1.100465918644558116e-03,-2.766519623248420157e-02,8.516121537719718859e-02,1.930540171524883913e-02,2.199556480392777721e-02,1.918861501615548582e-02,-5.547924542716922702e-03,2.648178675044609784e-02,-3.026404111061153923e-02,2.185231292459752922e-02,1.098493774733719786e-01,8.462501598037430739e-01,-1.561651529967792440e-01,-2.999010357362479756e-02,1.327288037302715162e-02,1.902096088844219476e-02,2.251521287654541931e-02,-5.518177288634125432e-02,4.918433256525783781e-02,-1.587206706777604184e-02,-7.143974321477077061e-02,4.747768306951317963e-02,2.215772804808916960e-04,4.751788142628815548e-02,-3.429371473191047265e-02,-9.579797667882329615e-02,3.050467019083474667e-03,-4.210136727724887035e-02,1.285072954042888564e-02,-3.636654527394537345e-03,-4.394232824463688986e-02,-3.417650970126879378e-02,1.153439136754236261e-02,1.565777844198305846e-02,-1.392584901965525368e-03,-1.687869147464008540e-02,4.329641759696430808e-02,-4.690153788402810775e-03,2.595802741287223103e-02,5.193117559957540624e-02,1.100255235465671250e-02,-7.283628337767594774e-02,-5.845019329081038717e-02,-1.777191345888948226e-02,-1.552850951521256469e-02,-3.871787361175867831e-02,-5.457209012849526669e-02,1.244251914099486246e-03,1.398234118860158109e-02,-6.911573004050358682e-02,-1.989945663052453823e-02,6.420174556236209196e-02,-2.568248788518053688e-02,2.279763227398101839e-03,-4.461704369734531550e-02,3.848999471724651872e-02,-1.751933733732836257e-02,-1.472671433551105968e-01,7.133698985100349332e-02,-1.225759760539535115e-01,-9.432079983126250011e-03,-3.053395212257777741e-02,8.421505324082854960e-02,-1.366624529222631168e-01,1.927675509427814737e-02,2.040379730745682518e-02,
-3.349223470950329212e-02,-1.996282319717172099e-02,3.915067237452008603e-02,-3.922317376062059782e-03,-4.888900009599489449e-02,-5.619403556115212112e-02,-4.604227209344050054e-02,3.796556216426644204e-02,-1.346535577063443130e-02,1.058915778651400852e-01,-1.595038030427326989e-02,1.015943194956180906e-01,1.370490912905950043e-02,3.549864551944444113e-02,4.275012850057589853e-02,2.348240485712314168e-02,2.362899044182919378e-02,-6.653711956802593752e-02,2.895125325391825072e-02,5.635683776213831764e-02,9.394166385628665017e-01,-1.221722889766799491e-01,2.000472467910834745e-03,1.021054524020431273e-02,-2.377603149823305506e-03,-5.793241555335952760e-03,-5.591980903560407612e-02,3.669008547330986081e-02,-4.116826474446957235e-02,-5.027327838719380937e-02,3.290278797008849149e-02,7.073108609980550271e-04,1.082013553031171971e-02,-3.422168450835323672e-02,-7.295351010427375549e-02,2.188420194370854430e-02,-4.161960351751907428e-02,2.339651388277066632e-02,-1.373512202116476157e-02,-3.873442983235073078e-02,-7.874616033688595917e-02,-6.601295283519612381e-03,-1.472435762579674557e-03,-8.346912710309664987e-03,-1.213324379919503496e-02,4.462319825458643113e-02,4.088959948568870670e-03,2.860891286118908208e-02,7.127944831752262844e-02,-6.257806285928133039e-03,-8.487471902966906012e-02,-6.481012147155430780e-02,-1.531939826376043506e-03,-2.157006631866698054e-02,-4.626553782988413951e-03,-1.491889634362519867e-02,-2.466652760227303759e-02,3.144157591023027942e-02,-9.267113497977939873e-02,7.415481473752465885e-03,7.758654503573343686e-02,-1.873376454390449086e-02,9.606578385723709215e-03,-6.612835307997025347e-02,1.146684316291315085e-02,5.948336564993269432e-03,-1.325571946970525716e-01,-3.030888843176496317e-03,-1.160281645388502314e-01,-5.979796264403904049e-02,2.527069700342671719e-03,1.283698986701518052e-01,-1.857598366124092737e-01,6.800506378633189471e-03,-6.073233660252966293e-02,
-1.821089144991085151e-02,-2.139370704991167688e-02,-5.391427652429827896e-03,-2.798276259685317235e-02,-3.136061301174834781e-02,-5.644792487633200351e-02,-3.005504502120795451e-02,2.962263495512595934e-02,-1.452210090492451339e-02,-2.111663925216195059e-02,-3.396499354508800139e-02,7.396055548021805703e-02,3.177899424856459043e-02,-1.465655694377027141e-02,-1.427918696751574734e-02,2.292472335224850916e-02,-2.431836792126523816e-04,-4.451229951933743240e-02,7.891717476294186906e-02,-4.528809339003112000e-03,9.246560414425950558e-01,-1.482383445667285748e-01,2.257148169134401763e-02,8.355432506193768982e-02,-5.643184406438760126e-03,-4.227186655910934732e-02,-6.699788047610852684e-03,6.859583697348684700e-03,-3.682975151449507933e-02,-1.770971872684376502e-02,3.146464475418035700e-02,-6.676484651665778522e-03,-2.523113113385535428e-02,-4.279460173341599616e-02,-6.758151896108630097e-02,3.171126808126936703e-02,-1.770447719240605261e-02,-3.662174313690236271e-02,2.830542714870798102e-04,-1.793110532801187040e-02,-6.020009619820207436e-02,-3.616116883634571538e-02,1.948468770097569469e-02,3.428345572055197660e-02,4.860033355054638182e-03,3.263535035713280991e-02,2.856055943394562174e-02,1.174932848058816807e-02,6.692571988394002691e-02,2.284960821018655955e-02,-4.138506084809730201e-02,-4.649638274853283570e-02,-7.548509280986548386e-03,-5.430934692586797050e-03,-8.229349943372720605e-03,-3.536630091571721368e-02,-4.884764147396435975e-03,3.491587639970628737e-02,-2.810567433311395719e-02,6.156260515644790959e-03,3.313910215803771153e-02,-5.162641424827286996e-03,4.019407220417715865e-02,-2.744344130669099449e-02,-1.841578156522682280e-02,3.042875992564365861e-02,-1.507851457988096389e-01,5.240876037369139595e-02,-9.434361923790149373e-02,-5.713449055642198099e-02,1.760770685192314736e-02,5.298488770194236669e-02,-3.943559172396137735e-01,-2.637951923765383816e-02,1.303524579260916250e-01,
8.635257527771221173e-03,-1.357197920582118959e-02,8.964298858845748744e-03,-3.807662148451618211e-02,-2.468322145286599642e-02,-4.561190907688759955e-02,-5.318995996256527248e-02,2.506994131641774454e-02,3.137657961305038469e-03,1.958445631555151834e-01,-5.061538756316912024e-02,2.484965711427102042e-02,3.329624304024667719e-02,6.544253277235434138e-03,5.073470341164336428e-03,-3.845963698825659999e-02,2.495162230138204054e-02,-1.783939662671106194e-02,1.612723004234230609e-02,7.748766243653236463e-02,9.606218518647718918e-01,-1.049590134207201708e-01,-2.146686797673017649e-02,3.608879604305188088e-02,3.944789502480267052e-02,-1.334918024168143050e-02,-1.086133582126146135e-02,3.387972010811192425e-02,-9.753280137879507747e-03,-3.067910909143303777e-02,3.696287291255559437e-02,1.818557026867768239e-02,-8.915879732235409277e-04,-6.520109136176410053e-03,-5.758687268518757757e-02,7.744572914837197231e-03,-1.668931979192053716e-02,-1.729041102821043951e-02,-8.677977297587194383e-03,-5.282383733997592784e-02,-2.124987402453362359e-03,-4.751949430147454217e-03,6.554381429402081349e-03,1.723113919454135473e-02,2.045004188940160832e-03,1.326221784652530100e-02,1.024913884015762194e-03,1.447287820471591935e-02,9.203918236124077057e-03,1.717445058523911494e-02,-2.282757964442364007e-02,-3.508208195668695989e-02,-3.246437465991743043e-02,1.165350017687087278e-02,-2.771871046238784386e-02,-9.411998969515147917e-02,2.828209892388695856e-03,-3.036691486054409611e-02,-4.281767730048218618e-02,-1.822028675498850100e-02,4.767897724210313354e-02,-3.689165875252360671e-02,6.157955853174395394e-03,-9.247027099451439816e-03,1.675250471719270592e-02,-2.038316681126776247e-02,-6.667520414643141047e-02,7.304327322450113180e-02,-6.090733897845018036e-02,1.183511651007792403e-02,-9.281883252152683603e-03,1.893862707759475655e-02,-3.656469388256421427e-01,1.684662471683124468e-02,-1.564149455195914529e-01,
-4.900819564463601563e-03,-6.600617316132678658e-03,2.597710192924179706e-03,1.235594957768343383e-03,-7.949792630040170005e-03,-9.013555278072661603e-04,-7.079125923009840782e-03,-1.352846820535387462e-03,-4.290139018035071805e-03,9.169211080847274875e-01,1.941236172718499922e-03,5.047873803106377871e-03,-8.453912205769076316e-03,4.631178239061346791e-03,-3.778371437795399745e-03,1.100178358378109400e-02,-6.165145996909059123e-03,-1.174232151943791630e-03,7.456347599159434195e-03,7.529306313298400610e-03,1.000000000000000000e+00,6.646119965666649754e-03,-7.331127365295786458e-03,-2.790133565632693657e-03,-3.788476382106439194e-04,-3.698968477802183630e-03,-5.538098530434748187e-03,5.029168326781893739e-03,-3.226191043853837009e-03,-5.952874649393480322e-03,-1.966097861675487890e-03,1.025656749193932377e-03,2.454261545988919990e-03,-3.274415912327605094e-03,-4.253932887541014840e-03,-3.325824062328994191e-04,-5.161773243553297030e-03,5.266014720678693223e-03,1.273180154853914559e-02,-2.601156781212463076e-03,-1.007725410697153656e-02,1.243996369578996564e-03,7.019379552144280887e-03,5.465011715831094574e-03,1.738257597336770321e-03,8.722509182679111239e-03,6.591753675320402572e-03,2.516618614632922662e-03,7.101172307466749323e-04,7.317402715202731001e-04,-4.432483389734866858e-03,5.636964243787220830e-03,-5.532720715679811940e-04,-8.677981603200032867e-03,-7.562203085405235815e-04,1.156460329163052671e-02,-1.283916681163121964e-03,1.246631742951392202e-02,-7.590095911940777929e-04,-2.876026422165636059e-03,7.551409251074406304e-03,-2.867967671970020338e-03,5.863136059654217025e-03,-7.062261913118657308e-03,-2.876086339436603745e-03,2.727495431491394617e-03,-7.776841558411781707e-03,-8.072248057120923193e-03,-4.953765699184336894e-03,-8.695792585837153893e-03,2.516582665335539981e-03,1.052356973272296908e-02,-9.350840607689068218e-01,3.627378896373385786e-03,-9.086547761931402789e-01,
-3.354082559541116983e-03,-4.960909450603053460e-03,6.621766404043806509e-03,-5.202686894695805059e-03,-7.578543406744750813e-03,-7.050184874024065934e-03,-1.419333099085078737e-02,-3.460461264254994368e-03,-2.308422456915454769e-03,8.857224759145768678e-01,-1.546291522675125214e-02,1.969343811777081454e-02,5.569566190033219816e-03,1.500831637199070631e-03,8.857723979191767238e-03,1.776433681681885424e-03,1.636477178311837935e-02,-1.564975538324655732e-02,9.495870641183087532e-03,9.516633554269016138e-03,1.000000000000000000e+00,-1.888920555213644686e-02,-9.428091924821680597e-03,4.629460561864914142e-03,-8.826191253730110431e-03,-7.333073771106483202e-03,-5.039450023936577552e-03,7.899201801570984727e-03,-7.525092386221791246e-03,-7.495341650767743495e-03,7.187071119446643286e-03,1.053419749094635163e-03,8.367779357794851575e-03,-6.623365733310789677e-03,-1.853326433304684773e-02,2.933910182058843338e-03,-5.845315273804744735e-03,-1.193108297010539443e-03,-8.244544977606842184e-03,-1.023472694507986287e-02,-1.173450819774873172e-02,1.016193252387232078e-02,-9.000028047306766338e-04,2.230264509400350925e-03,1.724280121231245438e-03,5.909609219040471220e-03,1.198245807791238106e-03,-3.502119104783396521e-03,1.048371801000302743e-02,-3.998812077940602783e-03,-1.195939025812985806e-02,-2.128596866985944455e-02,-3.197211870383430410e-03,-1.383992363235632232e-03,-1.021675096202611516e-03,-9.146203637206560405e-03,-2.444020921002521517e-03,-1.067441268406686336e-03,-1.168375097415740027e-02,5.083954308360278188e-03,1.565622376386632819e-02,-1.704847022208667850e-04,6.090216340073933282e-03,-5.099166398034442674e-03,2.834334988139026318e-03,-4.610579984705306944e-03,-2.534204814522906385e-02,8.527357275471995207e-03,-2.216150443758781868e-02,-1.214047501134399819e-03,4.474358861318313008e-03,2.013634094391042809e-02,-8.877434827866170775e-01,-1.353605649704911311e-03,-8.786127033379238505e-01,
-7.300363152358005370e-03,-1.477747438612476982e-02,2.457837531184930285e-02,-4.676525653731087445e-02,-5.381570020190169779e-02,-5.308249151933565335e-02,-5.114182008044085986e-02,4.595784322708196257e-02,1.052454780645722954e-03,-2.782710256465855542e-02,-4.480446072748905939e-02,4.972825639786095098e-02,2.410888055676201069e-02,1.592674271359937471e-02,1.407599621382833385e-02,-1.971160531083249179e-02,2.318598391010701434e-02,-4.170453027303722138e-02,1.155315311137873613e-02,1.143060514051417037e-01,9.898444517084351579e-01,-1.356352873611867127e-01,-2.286613263415392810e-02,3.044601698693193137e-02,5.088961809445626677e-02,-1.929034635458045169e-03,-3.918970719025020172e-02,4.769076342623478071e-02,-1.668955483573132323e-02,-5.349229606817528654e-02,4.757383691261017622e-02,6.479469536514792516e-03,8.662383122926171908e-04,-2.021986045762366041e-02,-6.846085991687585193e-02,1.104001108059154274e-02,-2.603517647900759763e-02,-7.386053452881902182e-04,1.257713181309298364e-02,-5.791166712624731122e-02,-1.367500131056474666e-02,-7.202543488960754317e-03,1.063087403186312348e-02,5.941976749957576798e-03,-3.141280976221991900e-03,1.640036048659392065e-02,-7.331004606215929358e-03,3.541544658246151278e-02,9.707701069304651431e-03,2.487173344879953230e-03,-3.963153972844665057e-02,-3.100502689495275924e-02,-2.330951578773928640e-02,8.004607560702863001e-03,-3.842229461823126746e-02,-6.381607072116179946e-02,4.162957601769944754e-03,-2.028217816278701791e-02,-6.810699544592720145e-02,-2.606425507541003037e-02,6.833444282902055578e-02,-3.960528682958837365e-02,-7.918461529811890443e-03,-4.726975887989113417e-02,3.967123619831275372e-02,-2.337831612370855350e-02,-9.196832544151384292e-02,7.467992790017637106e-02,-9.300059980231256684e-02,-1.576806639842452673e-02,-2.125930298015202974e-02,4.077643067308095448e-02,-3.087720254589532143e-01,1.963745847673461706e-02,6.579052118780710778e-02,
-3.087002252736680691e-04,-2.747499496875396331e-03,6.183599658275319363e-03,-2.406114845693071991e-03,-7.706785924834461468e-03,-1.391398529284223855e-02,-1.122165528746395941e-02,7.584352677486912858e-03,-2.727482749082549285e-03,7.072375582404162309e-01,-3.854237303198317088e-03,3.789896436357936826e-03,-5.053805347048564989e-03,1.215446712654949177e-03,-1.377424861279761756e-02,2.258208523054867911e-03,2.244802990395512732e-03,-1.496483390678485237e-02,1.168265245759326360e-02,2.071531825306563435e-02,9.749930805519644306e-01,-2.865223269828409691e-02,7.502988905662061336e-03,6.200672091015983767e-03,9.495591489034175409e-03,4.802994627585613352e-03,-1.722930155618043052e-02,5.371068967615313938e-03,-1.502178631119924050e-02,-1.714200081992384508e-02,1.675758704867972779e-02,-8.590701063806563398e-03,7.832894721347663861e-03,-8.363114687467733399e-04,-2.636595079458622579e-02,7.886773705548324326e-03,-1.189100535280160006e-02,-2.432564919908757582e-03,-1.630809207900693614e-03,-4.658611147501046101e-03,-1.660477892556384491e-02,-3.332241590822035068e-03,7.321847847876698789e-03,9.190415497808031298e-03,6.161278486234503249e-04,1.888830119780886274e-02,1.186111260872159054e-03,3.893276660676205786e-03,2.031823286591748906e-02,1.081580040576616240e-03,-1.543156073337396787e-02,-1.183772507578108313e-02,1.160445203860511337e-04,4.701887697510544195e-04,-7.538560481831655877e-03,-1.022649967043615798e-02,1.134342646998402109e-03,2.708163260161149111e-03,-8.881965775272403912e-03,-7.412724791095094690e-03,4.157894728329933343e-03,-6.118389853384042371e-03,6.128310044411430790e-04,-1.298251153993596613e-02,1.266154643488975884e-03,4.225742109107134131e-03,-1.879831869927863289e-02,1.113063915905916981e-02,-2.286656264692988813e-02,-7.818264707533618615e-03,6.875463713046864669e-04,1.191059506707262128e-02,-8.420157707387929680e-01,5.063986529229425396e-03,-6.906482223016697786e-01,
1.319340494701808433e-02,-6.851554207353745436e-03,6.594562510396857584e-03,-2.662195943948644783e-02,-1.133706506399152769e-02,-2.691809780911457936e-02,-1.912113235889277910e-02,7.269232049729561218e-03,9.137090143907226902e-03,3.361595466315877578e-01,-2.991091988008487412e-02,1.427478831581741742e-02,1.450839605082248837e-02,-1.802811676360763921e-03,1.570983485266118709e-03,-1.962136516422156515e-02,1.301921628209856685e-03,-2.109053168770774755e-02,1.359228873013997377e-02,3.466412999728148842e-02,1.000000000000000000e+00,-6.135344916641247148e-02,-4.678484598425415239e-03,2.415682438546118416e-02,2.182623553846286743e-02,-9.523306058924416173e-04,-9.787947969917373814e-03,9.717649273427813456e-03,-1.526162546971813333e-02,-1.283427868684101220e-02,2.856829677328698303e-02,5.427270652146018275e-03,5.357722706304225702e-03,9.649006945329809126e-04,-3.852621703778984219e-02,1.724144000130218082e-02,-1.297304637581966902e-02,-8.526559605998573765e-04,9.067763579391104056e-04,-1.626039165317096774e-02,-1.080718654031941409e-02,6.403815701970230409e-03,1.366917361118681618e-02,2.248494899770114852e-02,1.521792435086496281e-02,3.667477113837128155e-02,9.673375128704433945e-03,-2.686765876091991301e-03,1.306752879512795840e-02,-2.939001067720321651e-03,-2.850379522662757625e-02,-2.468862440242592174e-02,-3.106664344551244780e-02,1.026933773390131290e-02,-3.066418846899123321e-02,-4.120799954534530574e-02,7.008486963154113292e-03,-1.023392238041067118e-02,-2.497912534327226264e-02,-9.893723715608031266e-03,2.116445383471077940e-02,-8.419307147788809842e-03,1.023917782896197406e-02,-1.077460695818592551e-02,1.311862774450864300e-02,4.968529565527624037e-03,-4.182450416031587015e-02,3.744144334857314738e-02,-4.760216684449439239e-02,-3.657873955644479202e-03,1.377150842550967091e-03,3.054218855968133389e-02,-4.717707084482670732e-01,8.920054383696284991e-03,-2.834684876849877244e-01,
-5.623506447452858957e-03,-4.129700294313567305e-02,3.718891694595659325e-02,-3.155580000499761534e-04,-4.719916046746257254e-02,-3.169378530481394007e-02,-5.604630205517934644e-02,1.511713132088666339e-02,-1.346395333906624109e-02,7.568151568189537826e-02,-5.246846301015613279e-02,1.103580484262582789e-01,2.870708971007961824e-02,3.081740605554260728e-02,7.950751344339897220e-02,-2.042761404980157625e-02,6.420244288359515383e-02,-3.130333937675405104e-02,3.525061909775868696e-04,4.025529950219024905e-03,1.000000000000000000e+00,-9.551876569726353017e-02,-2.313034276523778451e-02,1.470498603885475250e-02,-3.988832205338282283e-02,-3.192907849789779207e-02,-2.711948299955104294e-02,3.817786422265541380e-02,1.376139451701420199e-02,-3.132113374657589239e-02,2.230689914504243448e-02,1.543350495712090230e-02,-7.558777998914419094e-03,-7.186819505896283738e-03,-5.344832422597075444e-02,1.361026860646587754e-02,-2.311291295074644966e-02,3.212866206431805663e-03,-2.315275329817690839e-02,-2.542266994344591011e-02,-3.147609309059941352e-02,3.567670454992268730e-02,-2.386608310289957749e-02,-4.249825920089270875e-02,-2.834171577458228022e-02,3.698002063277795043e-03,2.467179439236452665e-02,2.304121192808838287e-02,4.117446711548133909e-02,9.684966007342020802e-03,-6.547739877895288052e-02,-6.285853466803524581e-02,-2.439642162867907554e-02,-1.073235546329417457e-02,5.759149857953241303e-03,-4.762057974977251806e-02,-1.479773661880994309e-02,1.024348056127567103e-02,-3.423791761207461981e-02,2.171478429146786113e-02,8.479163934302770544e-02,-4.199268832101374707e-03,2.482858803105653794e-02,-2.784152558214996603e-02,1.082165585464182206e-02,-4.215680561278258714e-02,-1.079345223115438790e-01,-1.894070324982883441e-02,-8.390831902039665169e-02,-2.151463340501497817e-02,2.108581626373315365e-02,1.047801057098056293e-01,-2.032573251306885598e-01,-9.485428549547387372e-03,-2.355235727359225595e-02,
6.348735125868568488e-03,-2.079004935191363507e-02,1.513561793924773392e-02,-4.997345124184418563e-02,-2.037457478854366039e-02,-5.019268223808276419e-02,-4.950702269147641554e-02,2.019243199446209974e-02,5.437610939589542294e-03,2.288014372322840495e-01,-5.907497651450052678e-02,4.428732787827190792e-02,3.211429989566152998e-02,2.086977417859625650e-03,3.889607184749732660e-03,-2.602179500870770851e-02,1.724836202007918803e-02,-4.147219145428003573e-02,3.292622659234376720e-02,7.079653119711562526e-02,9.665890074153113964e-01,-1.166264202179445458e-01,-7.284805149080073039e-03,3.789591138277674520e-02,4.475302058296011520e-02,-1.431982933861371959e-03,-2.021855964848245230e-02,4.120498413772332585e-02,-2.755764471773925611e-02,-3.238910890856940700e-02,3.361285224193127918e-02,1.163335266059073968e-02,-8.533327470818034138e-03,6.444623985901048319e-03,-7.610714575336940957e-02,2.072339561627083507e-02,-2.448097622138802748e-02,-1.518436316708571959e-02,-8.122328743311816451e-03,-2.539421198445190875e-02,-2.050457977046980101e-02,5.873501031432753691e-03,1.603053270352506471e-02,3.051543383377514074e-02,4.620595384618019635e-03,3.932080197453152737e-02,1.480076433058214597e-02,2.303603518811149845e-03,4.077289398666458786e-02,9.678315581022691599e-03,-3.576560755214604204e-02,-5.291026577451144602e-02,-3.400820240052052773e-02,4.705817818378510084e-03,-2.674337838278858376e-02,-9.246982040343634801e-02,-2.285882042983338608e-03,-1.429331260638921387e-02,-4.866966051201375432e-02,-6.368398755749390960e-03,3.835775651476816539e-02,-2.269617723018731159e-02,1.707677816064472401e-02,-9.352952009333077160e-03,1.485089163529769787e-02,6.164158391443741447e-04,-7.523806770469315675e-02,6.278135461463243971e-02,-7.020419055771345840e-02,1.654245597827116304e-03,7.117154991394321339e-03,4.328195633195289910e-02,-3.938763833985983176e-01,-1.515354889969820762e-02,-1.729332976188789339e-01,
2.445065585368541572e-03,-3.185548376873586010e-03,1.104449607344433829e-02,-9.774214549462241150e-03,-2.221679078528273135e-02,-1.573520687789448785e-02,-1.760807561620412326e-02,1.846700048918141490e-02,5.831359119363242749e-03,5.521783735764937973e-01,-5.465282142003299870e-03,1.457666877143116296e-02,-9.459655203153165270e-03,9.932661444637405634e-03,-3.872994073597445241e-03,2.640172769698974353e-02,-2.161995391357938306e-02,-9.307705250640681272e-03,2.085712911834607802e-02,3.308344896298551219e-02,9.185202199199578033e-01,-4.689648267623171274e-02,1.029902849382829770e-02,1.730029527081579366e-02,2.466622876297821024e-02,1.826522821595271892e-02,-3.745939131189629512e-02,1.084600658484671624e-02,-2.723460754886639079e-02,-1.812247208217025035e-02,3.043695890654741359e-02,-9.016315301553091965e-03,5.415439401132312452e-03,2.158001182993446454e-04,-4.093662977499824829e-02,6.656925608749985485e-03,-2.749408080885383521e-02,2.433917279926565980e-02,9.165147840306323013e-03,2.376043287876347318e-03,-2.559119655904808222e-02,-8.676491379312409324e-05,2.451306149645281207e-02,1.729849507565348055e-02,1.889381532613738229e-02,4.752032334969463756e-02,1.652269860789198908e-02,1.335358258633045220e-02,2.499479066140877434e-02,-6.405148715822150951e-03,-2.987964309464723883e-02,-1.191004326667629426e-02,-9.434322596225092061e-03,-2.926147529442785222e-03,-2.098557635860460666e-02,5.373830067607915373e-03,-9.415429117168631901e-03,1.816091622830651175e-02,-1.994410413307245999e-02,-4.811511392821973820e-03,1.533257978088963837e-02,-8.825726473702461689e-03,-5.598636558006263308e-04,-2.752607185035237103e-02,2.025734585964738405e-02,1.320249467407994676e-02,-4.294836294853699582e-02,-3.297908200019281089e-03,-5.020579579751670296e-02,-3.917734038802048180e-02,-1.115357890426697113e-02,3.176073300795646814e-02,-5.692301332779856615e-01,6.232036513500608180e-03,-5.593380233405728097e-01,
1.492245696367823157e-02,-1.877641339380724250e-04,9.613074217749800060e-03,-3.165302893113172700e-02,-8.637230367995095415e-03,-2.463112496172618904e-02,-2.233139913061681081e-02,7.619531032695053829e-03,7.493890109230877394e-03,3.261818739312510873e-01,-1.748687891550598783e-02,-6.896722386836556806e-03,1.280785321484285283e-02,5.926725273582029302e-04,-7.181051700331841925e-03,-3.064808311863972262e-02,-4.828916548801007180e-03,-6.559143864326266726e-03,9.842731943156464783e-03,5.008829985627638592e-02,9.970605358114189576e-01,-6.841119854799532218e-02,-6.005892023618321363e-03,2.821133222434557553e-02,2.330025097781512899e-02,7.084675757732181167e-04,-1.979564890120252829e-02,1.482182485456925297e-02,-1.209146257817849020e-02,-1.803349619473928328e-02,3.209570120402627236e-02,5.083535444851414523e-03,1.347252801169689443e-02,1.135696796015270987e-02,-3.055408561130640299e-02,1.635491205641854268e-02,-5.144630898343362235e-03,-8.154959417955739281e-03,-4.544249104663413558e-03,-2.829379077444747065e-02,-2.371833933387038074e-04,1.197470112905299477e-02,9.621102463467411450e-03,2.901867685752956144e-02,1.081152908286776848e-02,3.249776229988054349e-02,-5.726803024587595776e-03,3.495249165310909678e-04,8.053058205814627585e-03,1.124459255428358201e-02,-2.018856696737145878e-02,-1.427304189217135089e-02,-2.613107109036454101e-02,2.053694883295494045e-02,-3.202166656054015320e-02,-5.354122135968097246e-02,1.375862131734576811e-02,-1.432624161209173144e-02,-2.378209980490281206e-02,-1.939364449749389138e-02,1.687930129531230172e-02,-1.490866181029467177e-02,3.093873456539657230e-03,-1.385181539806837336e-02,8.242859956625221696e-03,1.049560362604888553e-03,-4.220131233735213705e-02,4.140941425682035520e-02,-4.175333142131865127e-02,3.436995292984870720e-03,-1.147363397455726519e-04,1.777288681439589957e-02,-4.719478709274062544e-01,1.593212108285857465e-02,-3.009720412709542270e-01,
-1.438926741552986471e-02,-2.810509576204668625e-02,3.266646438850433209e-03,-4.270526188929067712e-02,-4.055415711772682175e-02,-4.637425269525947968e-02,-4.887975354210988277e-02,4.033320726342941309e-02,-2.743003884493706076e-03,2.047846890468826286e-01,-5.288242134896026236e-02,4.036377718315803387e-02,9.845758753901923122e-03,2.745584461085671946e-02,8.700444905476098501e-04,1.174323208537096999e-02,1.522125933116088778e-02,-5.178209533815080617e-02,3.700896250408847177e-02,8.134603007699539590e-02,9.381244788479262686e-01,-1.234548255674069728e-01,-3.682743797370799377e-02,3.132499481128298013e-02,3.613043472510436838e-02,-1.620684590281173260e-02,-4.177250610497613287e-02,2.785892343611001340e-02,-4.122747481707157019e-02,-2.596365377969403476e-02,3.094367282036425762e-02,1.802213980147153796e-02,5.686941797633784587e-04,-1.559191819675644422e-02,-6.866948792615509833e-02,1.268783141064227724e-02,-4.684048741094270357e-02,1.074271074975233220e-02,1.339379620832541196e-02,-3.214541262131491439e-02,-4.143789365149778969e-02,-3.683330996848687214e-03,3.247647638204361614e-02,1.078196175150184129e-02,1.384672852942299762e-02,4.475418642371256639e-02,6.588365983285762381e-04,2.263884794420860727e-02,5.057668551471488499e-02,1.550212062601707269e-03,-4.225832746775527726e-02,-5.813229114604947373e-02,-1.647030519259306408e-02,-1.542602167569618460e-02,-1.722688953674852627e-02,-4.827656327188582785e-02,-5.995527210784076452e-03,7.645290509510099808e-03,-4.339907269004116691e-02,-4.500514968406994082e-03,4.750518405364609342e-02,-2.215242162694293360e-02,2.068429344063563158e-02,-4.158944040126465180e-02,1.640013090539347088e-02,7.615504980442354410e-03,-1.077530240814247725e-01,6.921160515224275556e-02,-8.576668858736125056e-02,-1.379707199826070430e-02,1.629135940752440663e-03,4.365044112979726137e-02,-2.591399091113149034e-01,3.789544059690673136e-03,-1.805370685073893755e-01,
-1.418913881973517255e-02,-4.722253764234860332e-03,3.404753341474907324e-04,-2.601575258483018738e-02,-7.346097413873730643e-03,-1.835762043879605662e-02,-2.062048129512534383e-02,5.240896767907711473e-03,3.254132203529701631e-03,4.651040766403481963e-01,-1.416421780449961948e-02,2.698540423023733370e-04,1.478592344969170691e-02,1.284950682760435670e-02,-1.714428563160503433e-02,-1.687278364016449317e-02,2.637723238740906622e-03,-8.125905534221308790e-03,1.895250373129624133e-02,5.564306332894137369e-02,1.000000000000000000e+00,-3.322509799708789346e-02,-1.865648088756282491e-02,2.435110487331613310e-02,2.295250559078476638e-02,-1.005954628314630755e-02,-6.461652461179511267e-03,8.222692909048624765e-03,-1.280820898623596590e-02,-2.265150687127695864e-02,7.076657002608090913e-03,8.753358675853320503e-03,2.401781569072196885e-03,-8.279498561365675885e-03,-1.654221752599226158e-02,1.370457559981583945e-03,-1.099295657575912605e-02,2.261644868219170323e-03,1.970226041537731509e-02,-2.642842437936730043e-02,2.884182143108197927e-03,3.017527442620293598e-04,8.666238510636390530e-03,1.772140320686797083e-02,1.542611311321736428e-02,1.011584401669290458e-02,-3.076336523975665635e-03,9.819351954301502872e-03,-7.000906646629737051e-03,1.990341334960657373e-03,1.029747984076582155e-02,-1.158159190905658220e-05,-1.864662197347140629e-02,6.566893228341801059e-03,-1.465222263333300357e-02,-3.332628177640747230e-02,5.219149863104332980e-03,-6.186364377440793649e-03,-1.791456602694774941e-02,-2.490161862314346081e-02,1.658554996524069791e-02,-2.921727369615119418e-02,-2.341810333413010467e-03,-4.942597625507827788e-03,7.026610984677585971e-03,-6.502168240068352360e-03,-9.070750740112508426e-03,3.410290209062399186e-02,-1.713968234804445420e-02,4.053713879418434363e-04,2.998100498736496616e-04,-2.804891557240611621e-03,-6.103101885896538015e-01,9.396896868315365858e-03,-4.257944082620209736e-01,
2.175682413971831170e-03,-2.472207489474382777e-02,2.293145474224308686e-02,-5.355079991824471253e-02,-2.597915178579541379e-02,-5.583306304535579079e-02,-5.259451564128003048e-02,2.853556900243753303e-02,5.871442368960446551e-03,9.626429475815798109e-02,-6.476479495900275274e-02,5.775760382941032972e-02,3.763264905818432260e-02,4.306529421688864295e-03,1.546241957115321308e-02,-2.316480578098793069e-02,1.941739815003571887e-02,-5.220517357050723228e-02,3.212520736849754782e-02,7.696223249629355334e-02,9.677999996014176043e-01,-1.392674698550664947e-01,-7.751696616286638669e-03,3.991076984338937789e-02,4.606482250448080268e-02,7.820230632785304253e-03,-3.262668046864795196e-02,4.911186214906235192e-02,-3.277314031199128619e-02,-3.908127866411371942e-02,3.938205121600538150e-02,9.916711225437342334e-03,-6.191534572705885593e-03,1.150543176906633491e-02,-8.828176997194167597e-02,2.017684297902999635e-02,-2.977707535058170610e-02,-1.105468820293915508e-02,-1.496108482837210825e-02,-2.538499445046493025e-02,-2.794044931732933557e-02,1.119861163187726646e-02,1.323896123222458900e-02,2.922328401005631557e-02,-4.803391235435793861e-03,4.143842138234692107e-02,1.586786256513414775e-02,5.563008937389871816e-03,5.291755184744482976e-02,1.029676464261295582e-02,-4.424097212231400955e-02,-6.068279751560560348e-02,-3.376899263187813777e-02,-3.227773719724534932e-06,-2.677187203339581253e-02,-9.866615003991878463e-02,-4.805064455325622208e-03,-1.369172751783437683e-02,-6.047189120479903174e-02,-4.605548503435433205e-03,5.079818100628389482e-02,-2.352909636562685106e-02,1.852622886772485600e-02,-1.667057693304660124e-02,1.986504701227947284e-02,-2.177523758155386346e-03,-9.550758969819791999e-02,5.675978923503502416e-02,-8.617001072529754946e-02,-5.099535014398064864e-03,7.447476734223308636e-03,6.302342926129222977e-02,-2.646779175594661804e-01,-2.575575044939880029e-02,-3.784122797456709875e-02,
-8.546181030367264175e-04,-1.090083478162414549e-02,4.176643087562909029e-03,-2.731047417196484639e-02,-1.109905667223525655e-02,-1.665811845375532452e-02,-2.830439557276093546e-02,3.954295539929063297e-03,2.707375353300400800e-03,6.136521948729246612e-01,-2.564648687645804115e-02,8.828884796599609577e-03,1.545811569677454748e-02,8.808510387476476550e-03,-7.097091289927623392e-03,-1.993853108068963098e-02,8.507064221139643001e-03,-7.684977074006713038e-03,1.405109092264886228e-02,5.070182824194385818e-02,9.981428195234486322e-01,-3.832944904073492393e-02,-1.421754092957720067e-02,2.182983866097149006e-02,2.203410074080007497e-02,-1.153772848521872341e-02,1.062406894242778899e-03,1.760285837724280544e-02,-7.370169810251014919e-03,-2.065085266356892629e-02,1.065538727897754820e-02,1.273270291034054452e-02,-1.653341656554750683e-03,-6.364457558737726672e-03,-2.499299855794995945e-02,1.042086843664931373e-02,-2.850078373005320020e-03,-8.183563304373968339e-03,5.522213938733470939e-03,-2.408509821711540821e-02,-6.485246491105406408e-04,2.910876310804298706e-03,3.831514858812862245e-03,1.453112330823633919e-02,1.228404724410096793e-02,1.681628700197106835e-02,-1.263164891982346272e-03,4.830983867789757219e-03,1.105365264998634661e-03,7.310233726854929726e-03,-5.521188036725126203e-03,-1.552039459364237470e-02,-1.754357647308070478e-02,8.834626218650533380e-03,-1.733710622696361600e-02,-4.179797552523301096e-02,1.128264135079454049e-03,-1.022983622872413595e-02,-2.303461918821135546e-02,-1.701322428757793409e-02,1.605276537984556828e-02,-2.472619897662115679e-02,5.099931669121406108e-03,-6.134869516009680557e-04,7.864269524833862254e-03,-4.258386153717841696e-03,-1.923347303243051754e-02,4.426777618590183733e-02,-2.664071408936970198e-02,6.947526291132891108e-03,2.378275467776601315e-03,1.709737106877915608e-03,-6.832093910508599821e-01,1.275434405335514748e-02,-5.795457826319871941e-01,
-3.274231011485692622e-02,1.505065090162684832e-02,-1.514568994950220987e-02,-2.539672333907476084e-02,-8.989588853967386134e-03,-3.211049012461749536e-02,3.235578416285845443e-03,2.028835609217165270e-02,3.686405892942099839e-03,4.617820321408508399e-01,2.879881069249870529e-03,-9.961960729430618944e-03,2.803596717304844207e-03,8.023706946712828164e-03,-4.295378524237350448e-02,2.630182380397527719e-02,-3.092952808766827669e-02,-2.018196518367449538e-02,4.519761322295093309e-02,7.811720330556432002e-02,9.896046659847402838e-01,-1.423947746076836482e-02,-2.298489991584103803e-02,1.681828569900600812e-02,3.532045379549690611e-02,4.071466743231362886e-03,-1.322734682459932032e-02,2.104137767348288057e-04,-3.598588788659339704e-02,-3.381062722667864395e-02,-7.168619799273710791e-03,1.975764424391408813e-03,8.151031888075428722e-03,-4.081606098529254323e-02,-6.535210159196516710e-03,-1.196721184596712464e-03,-1.530505634672330029e-02,1.247807246758295997e-02,3.189186274848809788e-02,-1.435333586624958427e-02,-3.458471794186575993e-02,-1.891908485888564584e-02,1.955318065494035529e-02,2.040876417279911120e-02,3.000331091033153763e-02,2.574505573860641147e-02,-6.408982545217373553e-03,-1.584927579174999490e-03,-2.128757739143727346e-03,-2.225072797644847555e-02,1.839950762127638695e-02,-4.172050941779581468e-03,5.248338168357610800e-04,-1.412029393316355241e-02,-1.517119937498733785e-02,2.129959550069398383e-02,1.827674890522909372e-03,1.358956364792367894e-02,-3.023840238100222039e-02,-2.169378113591328636e-02,8.190322046435216663e-04,-2.909461749951968573e-02,-7.367829104446301652e-03,-1.821803732905388079e-02,-2.167507891010989757e-03,1.598908025927708171e-02,-1.541442810093426874e-02,4.797049608554244948e-02,-2.442401660558515819e-02,-1.560888170440274947e-02,-1.686373922025097630e-02,-1.236048933411793005e-02,-5.735600179055775349e-01,1.215475848689379426e-02,-4.442130970393877165e-01,
-2.099479918157646022e-02,-1.143040759861550792e-02,4.465856996814799793e-03,-6.028435317785881334e-03,1.615972076564033133e-03,-4.455786220714311185e-03,1.120755718947009753e-02,2.660644972048818724e-02,2.789206902484121092e-03,5.661607680762690276e-01,-1.669347441002360169e-02,-1.994583381207510470e-02,2.820613190222079347e-02,-1.846244215847673772e-02,-4.311075231357888726e-03,-3.956061885570391623e-03,-2.301324599299155171e-02,-2.793098750298210128e-02,-1.741230201916504211e-02,2.214132580900027986e-02,1.000000000000000000e+00,-2.478530613019962886e-02,-3.208594284717489198e-02,1.596372413834246107e-02,3.357939152103845901e-03,3.104647679953541697e-03,-2.671026516279049329e-02,1.869762819271066426e-03,2.890300309721410138e-03,7.527059811737124573e-03,2.279467296677466226e-02,-1.978706242217515357e-02,2.449841628450056749e-02,1.791190682771031523e-02,-2.377950496211801712e-02,-1.194402565153532270e-03,-1.281118022037409411e-02,1.719730234736090504e-02,-4.120877135858624918e-04,1.793293944914933769e-02,-1.860428915655930326e-03,-1.697079573284820952e-02,1.136566551605363087e-02,3.170252200044455099e-03,-1.060817339108823670e-02,1.317246243679410719e-02,-5.063041507524804197e-03,1.040689425446336865e-02,-7.627149602970687003e-04,9.170802376111182597e-03,1.743388161071073028e-02,8.889974889921532439e-03,-4.544341869391025669e-03,-1.986919313540009419e-02,-1.168352100445471317e-02,-6.280390411474663648e-03,-8.521161356236853571e-03,1.043446944846323360e-02,-2.184811560865021485e-02,-1.006253598472733185e-02,-8.351705336307690963e-03,1.526260865714421794e-04,4.086680541328928384e-03,-6.210554923436809732e-03,-1.781272426597582670e-02,2.676576516364545522e-02,2.564962055762606830e-03,-1.057048155273186511e-02,-3.860731366519281477e-02,-1.608719125922279669e-02,7.232880466444801745e-03,1.062524917576513142e-02,-6.699478806848543355e-01,3.559847731301442678e-04,-5.223938138988312163e-01,
-1.228429966410736485e-02,-2.114671063207096066e-02,2.285070580397299822e-03,-2.997563374765885302e-02,-2.960575882729302166e-02,-3.259829187407808915e-02,-3.133664993896652373e-02,2.198754433140438780e-02,-1.002240636798782183e-02,4.813070418318997623e-01,-3.092508562543725206e-02,2.471535130441983308e-02,-4.017193662457729797e-03,2.622243396038964613e-02,-1.398229703624243037e-03,4.819892211768456169e-03,1.352605558051581808e-02,-4.152103431854389792e-02,2.663654943476080270e-02,5.282545459758786199e-02,9.870645719574844268e-01,-8.117058772386112619e-02,-2.482933720925896426e-02,2.525981959521979292e-02,1.782981281280084096e-02,-1.575641915418114769e-02,-2.440015269491971339e-02,1.806903893188825660e-02,-3.357133290931859498e-02,-1.222060424452986460e-02,1.259276737366777504e-02,1.287298835510105807e-02,-3.486414269039186942e-03,-7.500574023203809837e-03,-4.666282349151010256e-02,7.488767053385310402e-03,-3.095239109916785025e-02,2.103244971125989293e-03,6.331685157782152018e-03,-1.798151889015603064e-02,-2.392741903946170312e-02,-5.944102131206698003e-03,2.190614560271578540e-02,-1.799718428272742421e-04,7.859493387514834839e-03,3.341279999320382471e-02,-8.926928366310933616e-03,1.223115754653124332e-02,4.019142704411961253e-02,1.661818099656126035e-03,-2.660652951806450825e-02,-4.569965261283379099e-02,-1.346083973718478766e-02,-6.989462597186121839e-03,-4.436357995481754468e-04,-3.866431639455072955e-02,-9.452247853677056555e-03,5.934784856705364875e-03,-2.494817782884305862e-02,-3.370013655479658227e-03,2.447654292312424762e-02,-9.655488817482862918e-03,1.995735742130227974e-02,-2.742184400247864307e-02,3.045075608160380948e-03,1.364433592353351751e-02,-7.155934430074928354e-02,5.537263383396213989e-02,-5.444505729549396073e-02,-7.227507197528848515e-03,1.157976057191469066e-02,2.285724139040720346e-02,-5.108871751891750357e-01,-6.218097056277942930e-03,-4.641246805238706807e-01,
9.004599652612044550e-04,-1.557312313291700674e-03,5.308629389634127518e-03,-6.439705391802254264e-04,-6.691126590292987598e-03,-8.637996668121825442e-03,-1.042203495048553072e-02,3.065289827956415789e-03,-2.507307506525967289e-03,7.164114530104335188e-01,-2.302700274120172280e-03,4.697641688731026000e-03,-2.299627844308442873e-03,-4.743635752613665352e-04,-1.070586297478031204e-02,2.903580723686309886e-03,2.835212863900320786e-03,-8.038784324206803972e-03,1.022071385728263310e-02,1.065737827049231465e-02,9.845371516146815827e-01,-1.740995322146634411e-02,5.982440063095199531e-03,4.546609301900756198e-03,4.324692629056969481e-03,4.956667508698681807e-03,-1.418330139277873027e-02,6.697377344577655489e-03,-1.121483717279546204e-02,-1.172251272727073759e-02,1.063644817066119672e-02,-7.796598145289272003e-03,5.946097000601126802e-03,-1.255395844224970608e-03,-2.090654882836627190e-02,5.198930359270547384e-03,-7.238873694117756163e-03,-5.420646071075455735e-03,-2.924546613055188302e-03,-3.970290403610019989e-03,-1.249894089558491184e-02,-1.771240885577753804e-03,2.814408112435861850e-03,9.013983807878264004e-03,6.488557877567732723e-04,1.561895930570263331e-02,3.880467054100461815e-03,2.526529379651715028e-03,1.680111509359163979e-02,2.507790689898028655e-03,-1.354760064264644293e-02,-8.709366548423351506e-03,3.249539992210455364e-05,-4.957914750996720438e-04,-5.098053167672174715e-03,-6.976027084012391757e-03,8.256280652141345655e-05,2.075585578085723838e-03,-4.762947354199865632e-03,-5.872660607869617522e-03,2.451130985682593855e-03,-5.810050832026165610e-03,1.960354151901088152e-03,-7.062205946618257957e-03,-1.261192137201698309e-03,2.056401271741170600e-03,-1.209984558266315527e-02,2.974390701480558036e-03,-1.885530368357126590e-02,-5.639113348881694653e-03,2.277067279213681929e-04,1.017530966226337652e-02,-8.041536577043619793e-01,5.940490819648304617e-03,-7.027787967310803463e-01,
-2.335363051811932891e-02,-2.182360092114996736e-02,5.452008745060040948e-04,-2.874894024091098274e-02,-2.388912238726147164e-02,-3.147434833040657631e-02,-2.531582812140776087e-02,2.964213189801105652e-02,4.055199239089671455e-04,3.692835957483957543e-01,-2.456347400856220484e-02,1.985853087013227103e-02,6.175122976056586152e-04,2.843736732342139542e-02,-1.224186722730500471e-02,1.453515185197036888e-02,1.064190985484045684e-02,-4.887306844705384401e-02,3.306908920865423568e-02,7.814616601485745162e-02,9.352676950594711602e-01,-8.182319195413720325e-02,-2.696354201177679141e-02,2.130560841257787136e-02,3.050697282442369623e-02,-1.118143342549449137e-02,-4.081633085419767576e-02,1.380731439813054177e-02,-3.064592577554262065e-02,-3.027309103796131662e-02,1.333043916654682443e-02,8.790880064195007865e-03,1.282022800948056859e-02,-1.074679306364833414e-02,-4.669595915641318318e-02,1.071653216650864972e-02,-4.103702421814811596e-02,1.369472655094091176e-02,1.834108424364968085e-02,-1.482491616762590496e-02,-3.735177355882559475e-02,-3.005048056819741367e-03,2.487416406817342593e-02,1.072946241269867422e-02,1.637913185378654657e-02,3.816775306917807864e-02,-8.202459641290127390e-03,1.304337892961365646e-02,3.573160079231651326e-02,5.320387080170225699e-03,-2.408011218613895915e-02,-3.672260344983946379e-02,-8.286979742542924590e-03,-1.351501641552185079e-02,-7.897109726563010523e-03,-2.513437095148302047e-02,-1.180829563406636182e-02,1.017070818994459183e-02,-3.691451354017684372e-02,-1.141791453748335246e-02,2.427022641603519479e-02,-1.415680143084278517e-02,1.054167652096557779e-02,-3.526347237705136639e-02,1.537908213664508639e-02,1.371125299678257678e-02,-7.554007100584070322e-02,4.773456748676116468e-02,-5.873023895244629256e-02,-1.194184551474066283e-02,6.433702172928349068e-03,2.934690842137225100e-02,-3.934360819152233901e-01,2.111710093000380983e-03,-3.519905347369247650e-01,
-2.252278796394431570e-02,-2.420996869821760711e-02,4.094805736724396739e-04,-3.006686667656178341e-02,-1.944661834358118319e-02,-3.014654522402671760e-02,-2.996671622921122380e-02,2.679280846436693508e-02,-1.757333024806448374e-03,3.484164614239304569e-01,-3.201332513431453392e-02,1.896342749945892359e-02,7.436008194988241726e-03,2.818256591539661932e-02,-1.239886408333298884e-02,-2.641129629247103081e-03,2.039301319689702449e-02,-4.658100071573670342e-02,2.776277000372401674e-02,8.514758005042416533e-02,9.577131241146589202e-01,-7.790704196412874538e-02,-3.056189627161884359e-02,2.172087030645980521e-02,3.098701286434771274e-02,-1.839167198407881676e-02,-2.639154467546728222e-02,1.579338861984115605e-02,-2.372924054451177461e-02,-3.533979708595640090e-02,1.130429502955932787e-02,1.613821311023363667e-02,9.326454213520671277e-03,-1.242009947877608170e-02,-4.510996459968271560e-02,1.361636354577644425e-02,-3.126700192779661991e-02,6.197599426411404266e-03,1.785832539007517483e-02,-2.501523740348381153e-02,-2.773270260001120188e-02,-4.437888548915136511e-03,1.860713792470630734e-02,9.339288164940791337e-03,1.923670246768738323e-02,2.795670051581070764e-02,-9.618510269735331547e-03,1.268525363831349284e-02,2.570992556103473409e-02,9.831749970808659894e-03,-1.842058044087743762e-02,-3.386253060935044690e-02,-1.060233960038215600e-02,-7.520398772185965530e-03,-7.809749569229119948e-03,-3.800709198798649102e-02,-1.131843390663158505e-02,-1.112193666110858348e-03,-3.726079788313023389e-02,-1.858851355180837772e-02,2.478090695368460883e-02,-2.179665422908870498e-02,8.877831757883210273e-03,-2.788069357936121678e-02,1.314157867886716832e-02,8.686865414620201231e-03,-6.485567554528781642e-02,5.935211114530056231e-02,-5.261453772775455395e-02,1.909591254225719015e-03,5.749846503017047583e-03,2.246133777165027628e-02,-3.815119859196791641e-01,3.628726065939891221e-03,-3.638271327746231409e-01,
-5.424370197092469315e-02,-2.617163792808563372e-02,5.310648917403222244e-03,-5.732857748077287152e-02,-2.535148095105087956e-02,-1.976777051127668466e-02,-4.394015127667807996e-02,1.221712904966800253e-02,-6.386851667816946521e-03,1.892245711628381399e-01,-2.676744298635500008e-02,2.306456964581121358e-02,2.430276414626900833e-03,4.188842514950327084e-02,-9.258217793059925899e-03,1.622360079302624658e-02,1.048870154999217291e-02,-3.728703220806130203e-02,5.057911332366949592e-02,1.005760431677077682e-01,9.848340159684209061e-01,-7.236565523277935230e-02,-6.437981124759033225e-02,3.686373494775981030e-02,1.306486327754029578e-02,-4.272777043168402705e-02,-4.111423098566070744e-02,1.773061075985860741e-02,-5.368324974931844296e-02,-4.611394551683423160e-02,6.814077936160052237e-03,2.231801368070708558e-02,2.540055736489875426e-02,-3.372672228407393863e-02,-3.060299451104440266e-02,1.747996280007303033e-02,-3.115482763709588598e-02,1.416299625754769212e-02,2.153701888324492664e-02,-4.339470812387091758e-02,-5.470238066163369411e-02,1.808101423539202821e-02,9.686423176029895082e-04,7.166696905815252844e-03,1.191499997599287236e-02,4.651575863027820890e-02,-1.970694958609421274e-02,2.967322469682802388e-02,2.899105481894077505e-02,1.755728429096892196e-02,-2.855147341155767418e-02,-5.005922954941716130e-02,1.394101926763632870e-03,-1.228106582562288676e-02,-1.258030984981529932e-02,-1.982067530991210580e-02,-1.026503955409619265e-03,1.567416659169365914e-02,-5.004617042903490576e-02,-2.254449568485696054e-02,3.095363612705396769e-02,-4.423027920189219475e-02,2.158039648676208372e-02,-3.649242587762692447e-02,2.106857129169525077e-03,1.139687667495107790e-02,-8.025440253510614241e-02,5.672878431060852933e-02,-6.982580017630932168e-02,-2.356760675365137184e-02,1.905457042366425283e-02,3.477504378932000373e-02,-2.872444994821383712e-01,1.180651153785518270e-02,-1.379064035221833906e-01,
-3.053397868691951761e-02,-2.949257674099297136e-02,-5.073862921903260492e-04,-2.523006540012079357e-02,-3.305451550802768895e-02,-3.682009436892463849e-02,-3.252248759269584649e-02,4.051253139357037514e-02,-2.553391225203275640e-03,2.340942345234998667e-01,-2.940539647526426228e-02,2.387849397991032019e-02,-1.678970280749111574e-03,3.837490038845468854e-02,-1.045624105766716772e-02,2.566620905051659324e-02,-5.813091469082688756e-04,-5.356860635726459763e-02,3.798133457922754641e-02,7.338592549034569923e-02,9.399470736354976763e-01,-1.148989070639612198e-01,-2.108225815494428612e-02,3.706782881517791534e-02,3.005438257567510929e-02,-1.760567985985077299e-02,-5.671491425126738706e-02,1.306948538344461508e-03,-4.264844577944505721e-02,-2.237089403322246872e-02,1.722771802851200129e-02,8.544255731835282211e-03,-1.600307707792786999e-02,-6.513639232582951050e-03,-5.288608967758198776e-02,1.718458182407744950e-02,-5.820427860325433356e-02,2.299053342234550629e-02,3.171781597696179955e-02,-8.929045351026711208e-03,-3.970297599976584102e-02,-1.977895241071786839e-02,4.133439637063204136e-02,8.979361146740914498e-03,2.126976006945061060e-02,4.643164324045339492e-02,-4.115501360016537991e-03,1.984808245885279904e-02,4.836049372483218478e-02,2.538949169681709202e-03,-1.396205371255677952e-02,-3.384553736183167072e-02,-7.554283230642003938e-03,-2.368705031967156815e-02,-5.793970383596996898e-04,-2.683137896824128871e-02,-9.829200720951555142e-03,2.585118030758384172e-02,-3.751296025134111950e-02,-7.764614432986281789e-03,3.479608888166769998e-02,-1.499917811161089430e-02,1.640621129860446101e-02,-5.126603042050115883e-02,1.409664762803964147e-02,1.799540252431429746e-02,-9.506641812237295874e-02,4.801255049285557686e-02,-5.792929482397735086e-02,-2.793199585726366396e-02,2.811453227762690868e-03,4.113249553472844711e-02,-2.318867098479514555e-01,3.130106453717918825e-04,-2.332667411487259757e-01,
-2.913766779896733924e-02,-4.338334266800525363e-02,2.245644569364425164e-02,-1.591232985525359647e-02,-4.396173709389276929e-02,-4.492045635834160389e-02,-4.017469880041282110e-02,3.690476614795542426e-02,5.159618397509372137e-03,5.418998187939871869e-02,-4.654361299679123098e-02,5.493235034389777616e-02,4.469490379574074870e-03,4.643318085378956434e-02,2.842565603398406610e-02,2.877100368496345040e-02,2.297698804008623322e-02,-6.060876642144444326e-02,4.237195944779821588e-02,7.607940685941153025e-02,9.198708537379580719e-01,-1.296219751958434530e-01,-2.826879721390188061e-02,3.307261747827060228e-02,1.660681992547633629e-02,-1.444938167906508614e-02,-6.632550256181762860e-02,2.329348941921890320e-02,-4.207424665914660800e-02,-4.138123909981180487e-02,2.566422626603605955e-02,7.494253424060490047e-03,8.908904410203881755e-03,7.916844400038990787e-03,-8.539337172973304746e-02,1.586876216814425006e-02,-5.415348648235197848e-02,3.039467166729389880e-02,2.197160882849352737e-03,-1.110313292715770991e-02,-6.149904761376882856e-02,-1.434852214689422839e-03,1.701674267795136308e-02,-8.613708141650550099e-04,9.709939185135418177e-03,4.981582426480141157e-02,1.207683255085576640e-02,1.282015651536252618e-02,6.987008567996749742e-02,8.853877210810139758e-05,-4.737531949020318267e-02,-7.231789664206778045e-02,-1.590226033720591042e-02,-2.771985001874101989e-02,-8.565767241592544753e-03,-3.001577946563685137e-02,-2.154426142292761165e-02,2.280309889645534133e-02,-6.055683437184364998e-02,8.594786801688798772e-03,6.109120574518133590e-02,-1.340668856524696376e-02,2.420624108227883986e-02,-5.273211197148376661e-02,2.325410949825647625e-02,3.662432179550673128e-03,-1.406567975602264764e-01,2.059200866341433234e-02,-1.049568913223799943e-01,-4.752160982536553224e-02,1.472388010002818534e-02,7.957373717767977916e-02,-1.724275800976108886e-01,5.627937828306527057e-03,-4.268757345104640111e-02,
1.341104315486028875e-03,-1.371109865577573722e-02,1.376814927371648040e-02,-8.649073313552260722e-04,-3.775637102167234832e-03,-2.157914591460100007e-03,-2.012987331192285165e-02,-1.306824030524335670e-02,-1.551280554814342405e-03,6.145826354623605825e-01,-2.721439890871334491e-02,1.222488400418231258e-02,1.430717478685439997e-02,4.881397331614939494e-03,1.217289368857166210e-02,-4.707240746492716288e-03,1.936063115540692658e-02,8.708434511082389223e-05,8.575534849258719880e-03,-6.002808016584648687e-03,1.000000000000000000e+00,-4.173382063034169684e-03,-9.956739270832763375e-03,5.540187400651028216e-03,-1.578231988990736606e-02,-1.991904906632478364e-02,-4.160004895174290729e-03,1.403895657353197712e-02,-5.563120430283324119e-03,-1.360845040224172887e-02,-6.987013197117333124e-03,8.073095391501680651e-04,8.992253713980626756e-03,1.569734936577498255e-02,-2.289250781236715215e-02,1.853384499110807826e-03,4.006172677504325763e-03,-9.788940346643714915e-03,-2.402910813382751892e-02,-4.335034240057121513e-03,-7.066247098003006211e-03,1.914816817334970278e-02,-2.120468331698123488e-02,-1.659262271235431574e-03,-5.284530630017644781e-03,4.806339958035671378e-03,7.964385019133768395e-03,-5.809314115266920459e-03,2.059195183355678332e-02,1.385194600651007413e-02,-1.679648809998108422e-02,-2.700066184229549249e-02,-8.353532982440948357e-03,8.800348857200485445e-04,-4.424502642673709685e-03,-2.059868349139286348e-02,-4.529484051503350484e-03,6.443893335254888971e-04,-2.682545707360175363e-03,9.159307930997707958e-03,1.232198670900504223e-02,-1.093434860929086427e-02,1.268164189988504713e-02,1.394699336500884734e-02,-9.001315526974584813e-03,-1.322283277054777523e-02,-2.363045888558512608e-02,-1.226621082684489772e-02,-2.687418868668571548e-02,1.113134252418657155e-03,1.694701072856564378e-02,1.342229685625302704e-02,-7.894390551929023614e-01,9.987388685258280496e-03,-6.012604969986898862e-01,
-3.452066255838104336e-03,8.234847099085982475e-04,5.227778797715170594e-03,-5.654966798508728049e-02,-4.109291754980328587e-02,-4.647065694169909972e-02,-4.709248745921025547e-02,3.271817141550429492e-02,7.563370469997141715e-05,2.024880085093391791e-01,-3.706916970579735016e-02,2.454038324750019984e-02,4.014054245562283241e-02,1.431182508135532203e-02,-2.131842643034487922e-02,-3.566037814092895553e-02,1.102999013797734129e-02,1.370465046854965618e-03,1.746735083508910852e-02,9.557455999559578974e-02,1.000000000000000000e+00,-8.848005251668483728e-02,-3.713159802152951605e-02,2.872673910648184531e-02,5.213409264929013548e-02,2.815985782759072853e-03,-1.004595775489989379e-02,3.628705139176317124e-02,-1.531755003715299518e-02,-4.229827137068149778e-02,2.613713394453476627e-02,1.402524442492390430e-02,5.100028700735813499e-03,-2.996325436359010413e-02,-3.696317401597640218e-02,-9.979194104908710450e-03,-1.506249677443956790e-02,-7.089730826062200231e-03,2.471103115673738485e-02,-6.105205888787602897e-02,9.155533549617837774e-03,7.475306776977533665e-03,1.394738236407463278e-02,2.476879341493562481e-02,7.944471306260639937e-03,9.401001179634410621e-03,-1.801869227606858054e-02,3.068241479682096151e-02,-1.075096303463159153e-02,7.731740409618138854e-03,-2.194373242901948018e-04,-1.051772337881951014e-04,-2.481827925301299462e-02,7.938896669282378624e-03,-3.975615022899396667e-02,-7.109085913825953507e-02,2.404005390967240904e-02,-1.836112534540648711e-02,-4.008500516044335299e-02,-5.122461853619983407e-02,4.806584929425193609e-02,-5.955795884052082767e-02,-7.822321357786944304e-03,-1.481529517353424863e-02,2.145846226525667533e-02,-3.025486737473561880e-02,-3.870351053658421803e-02,8.904172992737438563e-02,-5.245323433194477858e-02,1.399221128868198202e-02,-3.203799384290265811e-02,4.964023351341939616e-03,-4.582740197527877535e-01,2.214570597081469763e-02,-1.421732528743531732e-01,
-1.215637070448423798e-02,1.758477839826265432e-02,-1.423384670987703354e-02,-4.712955840187989243e-02,-1.554529007994927270e-02,-4.432435426984263765e-02,-1.938453854619331257e-02,6.639264598150194929e-03,2.179432759929967288e-03,2.598432899353420034e-01,-6.741229373042952056e-03,2.999932194914610786e-03,6.460312489032088434e-04,1.747898359701942003e-02,-4.031212818684251631e-02,3.007980692024934261e-03,-1.213753467639157313e-02,-4.486054955619312584e-03,5.512400992094034641e-02,8.743530591238571348e-02,1.000000000000000000e+00,-2.989883035713852139e-02,-1.171328465706916733e-02,2.583624597095688855e-02,4.778046438777300980e-02,-3.648276575507932752e-03,-2.290046614125199054e-04,7.101392851390934957e-03,-3.823562391980467323e-02,-5.009869533082388082e-02,-4.972555817635379571e-03,1.689427089836906953e-02,-1.733616194233615626e-03,-4.576421286860744841e-02,-5.150572816337031790e-03,9.837741468354267368e-03,-1.039700574509837272e-02,-2.983696696967260906e-03,2.794842687762647687e-02,-3.692141298746286215e-02,-2.487638716968887040e-02,2.291957746193356085e-03,1.321172561811327292e-02,2.991205648232794778e-02,3.599353722273818112e-02,3.057410223490815773e-02,-8.058641447882142900e-03,-8.051618561928222673e-03,-7.119376043952470312e-03,-1.867806178075307605e-02,-3.480192748139803588e-03,-1.644662015307553002e-02,-1.758466728551548106e-02,6.061859619752949652e-03,-2.399393587201509651e-02,-1.525839268066774695e-02,1.233975258930916988e-02,-5.554637975493773722e-03,-3.085851406434985886e-02,-2.701464025931990531e-02,1.196712615860679937e-02,-3.818300510436296868e-02,-8.446450612136651576e-03,-1.527015960353930550e-02,1.395313220219624936e-02,-7.332906744719243017e-03,-2.971178953961764457e-02,8.198605062454662751e-02,-2.369380677452124356e-02,1.538120478068772529e-03,-1.412009734592164362e-02,-1.500079685352590994e-02,-4.038267252534285534e-01,1.942417401402444718e-02,-2.647394030212088523e-01,
-5.694181491866002337e-02,-2.223031518908038379e-02,4.369243232221625695e-04,-1.384827697238007367e-02,1.625498019293576679e-03,-1.049196854506922669e-02,2.199296766168581593e-03,4.008478329334645707e-03,-1.140215040358317795e-03,6.236668301970732520e-01,-5.360738262395936884e-03,9.394668801431270422e-03,-1.038059784948157024e-02,3.930680790448265460e-02,5.723766741290958228e-03,2.237891093748493182e-02,1.504213687627946848e-02,-3.312098827633183640e-02,2.559744659239014675e-02,3.399136431200662500e-02,1.000000000000000000e+00,-3.280843656335490077e-02,-2.557542797076510152e-02,1.854286492957599869e-02,-5.950407279799277069e-03,-2.950026896421472505e-02,-3.843840893656522151e-02,-7.917147801936544327e-03,-2.823090650349915215e-02,-3.068613816151898738e-02,-1.933037150215776226e-02,4.630445509959155159e-03,9.053119184994783478e-03,1.644279015810986427e-03,-5.346145234501976928e-03,1.238104110046909720e-02,-2.806123511689439382e-02,1.502337832034478932e-02,8.950759729834976280e-03,8.202565317637917908e-03,-4.012464758145459914e-02,6.540247260231627814e-03,-1.288363963426297429e-02,-1.112378794381696859e-02,-6.010478544207052340e-03,1.503717302105588016e-02,-1.682323619417911559e-02,-5.463035595014827106e-03,3.155849951432719569e-02,2.040650426391669991e-02,-3.584469293889759600e-03,-3.039515610758669606e-02,6.749424844579850896e-03,-2.613220529160351080e-02,1.687525087772705212e-02,1.879349517402414960e-03,-1.813371953034926404e-02,1.904479405155265134e-02,-3.014342594243810125e-02,2.210289743029668005e-03,8.573083825280436443e-03,-6.835023270804619147e-03,1.554553837146408282e-02,-2.359625317828414892e-02,-5.239897932482171644e-03,1.094505843537311139e-02,-4.555743388558658458e-02,-4.202508909436929174e-03,-2.489337365853873385e-02,-1.650085543382937003e-02,2.801834482527250716e-02,2.714108781093032413e-02,-6.895451694490870587e-01,-3.407121475003042042e-03,-6.178138928939554964e-01,
-2.155929889839394015e-02,-2.325536460298621710e-03,1.221327210763704929e-02,-3.529131146557137194e-02,-1.119724158734813554e-02,-1.777741775126346485e-02,-2.857474373071183793e-02,-2.512602335619181643e-02,1.033123607448631354e-02,3.365448595715047064e-01,-9.736726845439535455e-03,2.737240023166601088e-02,-1.239224287235446073e-03,3.155218317770196179e-02,9.984961468485556435e-04,5.150946794508600400e-03,7.671150421604239211e-03,9.506045335609739078e-03,5.257084980074633318e-02,5.954572248457834505e-02,1.000000000000000000e+00,-1.135317441629676849e-02,-2.102878207454886988e-02,2.716514529306902903e-02,1.226987894242798083e-02,-1.711628369536314503e-02,-2.765046112534110673e-03,1.734590316441485952e-02,-3.780821936736849381e-02,-5.308122248925166431e-02,-1.163101396667708294e-02,2.457777083160830184e-02,1.498571726095440475e-03,-1.919709047176657371e-02,-1.166491027611052042e-02,4.436315503917417913e-03,4.220712709389916328e-03,4.280806623780952953e-03,2.251565252750505064e-03,-3.370474800114095504e-02,-2.892142978460507385e-02,2.938839325743104053e-02,-2.723917120058578015e-02,1.276859354775427419e-02,2.775133823146784529e-02,2.338496795100834802e-02,5.235830205498490098e-03,-3.884182290812538893e-03,8.809345096592556545e-03,-2.921077334048872987e-03,-1.228837345113016621e-02,-3.212792515426957790e-02,-1.721907700285391779e-02,6.804004734405041770e-03,-2.350143725994614102e-02,-9.701621973423616568e-03,2.398949404029741134e-03,3.388033000016405644e-03,-3.876765966368438815e-02,-5.732889984843142495e-03,3.237735397550654814e-02,-4.462145654560525226e-02,3.915027991299974142e-03,-4.202626036131116433e-03,6.973955379032505467e-03,-1.756360059515411909e-02,-4.431679321740937538e-02,2.236604644077801185e-02,-3.686433941574097234e-02,-2.367655221594625742e-02,8.228360594097441916e-03,1.977329492656182061e-02,-5.447489702796209921e-01,1.739750220275867662e-02,-3.542220587311987190e-01,
-1.256632454383857511e-03,-2.883049641434965588e-02,1.669632881179474934e-02,-6.062831841505758601e-02,-2.834050417080923034e-03,-3.453752139385660169e-02,-5.710647499319503745e-02,-4.729764771974218768e-02,2.180832723464016295e-03,1.291397815076148481e-01,-7.360645875965390883e-02,4.307119840080086276e-02,3.554128218690692786e-02,1.491397499625134876e-03,1.293669046509917514e-02,-3.616627230605419518e-02,2.614907183336724616e-02,-2.399924245299734152e-02,4.796020826276913007e-02,9.011433311621616649e-02,9.941201369731083037e-01,-8.230459467899019221e-02,-3.232244421843125798e-02,4.632904599356824776e-02,1.487083307157780268e-02,-4.745867408606342652e-02,-3.575645167271643277e-03,4.250320572941241465e-02,-2.058852710460629815e-02,-2.929867544404044996e-02,2.012525596252467269e-02,1.009764951694253066e-02,-6.605688780724015136e-03,-1.770309255239516447e-03,-5.711132759219497007e-02,3.254463599039832816e-02,-5.209617055412968205e-03,-2.923446154157031615e-02,-4.901129137058555379e-02,-4.225935584387351812e-02,-2.557084260133404147e-02,1.965269193194993380e-02,-1.327285239774471035e-02,3.126258860088571012e-02,6.333726703650829903e-03,4.988328885161782966e-02,1.027402902659496110e-02,2.711665979884717720e-02,9.133072394043725492e-04,2.736167035947563789e-02,-4.321845018855462878e-02,-6.477661259331622678e-02,-3.768586665040807304e-02,9.567545052935963121e-03,-2.723403415453416743e-02,-9.459188032670562407e-02,4.973321286095740146e-03,-1.076459464382228254e-02,-1.987431088629692394e-02,-2.970914435914889101e-03,4.560209374481750261e-02,-2.185979436219810668e-02,5.143946222135924173e-02,6.804234589598348593e-03,-6.505169933864372867e-03,1.191194232237831330e-02,-6.566488565523101451e-02,5.654719709895913993e-02,-6.725435242131076741e-02,2.608141363463892248e-03,1.043350902479320969e-02,7.448682424272261293e-02,-3.757572399671020857e-01,8.080527620761726419e-04,-2.801486994222407434e-02,
-3.853989930370185779e-03,-1.329618000132493268e-02,1.096249440190759546e-02,-3.872235623536687127e-02,-2.895701807817536999e-02,-2.987624015282055318e-02,-3.845711088762113899e-02,2.201308026268061768e-02,2.747161039154663587e-03,2.079028756859221394e-01,-3.579975244686976060e-02,2.104790199214678220e-02,2.467727025808419405e-02,9.985020625411695355e-03,-7.829581627660172488e-03,-2.022742805718083861e-02,1.305225297093069552e-02,-1.176511608582675103e-02,1.546681369141185772e-02,7.261101602734595928e-02,1.000000000000000000e+00,-7.738044687355596918e-02,-2.486675912089445634e-02,2.505293631769639831e-02,3.193696594052199134e-02,-5.416966111461925248e-03,-1.415511433281316928e-02,3.102800487516542416e-02,-1.784669779904665687e-02,-3.272637018970978029e-02,2.103964327044505200e-02,8.850330725535026158e-03,8.027616008439660000e-03,-1.192955019848015649e-02,-4.073868200205504886e-02,4.978108811217765023e-03,-1.207110493857195543e-02,-8.418690016117336139e-03,9.127986190022254268e-03,-4.264184892664397986e-02,-9.468009176833233273e-03,4.088424990230363210e-03,7.963218000600864183e-03,1.757485392806113628e-02,5.795915949972350358e-03,1.644923383766200287e-02,-1.184123733588634403e-02,1.646697577869885368e-02,6.837207843421101716e-03,8.302447522790136827e-03,-1.342831793452230194e-02,-2.088203599325605744e-02,-1.751726484307832010e-02,6.259123637561340645e-03,-2.693717815930974854e-02,-5.563058429050804499e-02,1.044201938250589358e-02,-1.048496278505642483e-02,-3.606836115182400493e-02,-2.942473741429354681e-02,3.467978511522987195e-02,-4.084752316071140615e-02,5.387425654049083495e-03,-1.457544284747380736e-02,1.789773674473415371e-02,-1.402231611411224044e-02,-5.056861121896180644e-02,6.631779927610848002e-02,-5.385108979962741765e-02,8.859597736612798841e-03,-1.330416322823867181e-02,1.294950711025760759e-02,-4.050616701075107073e-01,1.767757236548500588e-02,-1.544313034885208546e-01,
3.245907552222529469e-02,-3.884339955281449625e-02,2.679416632879035032e-02,-1.477929275943709424e-02,-5.965674278476475190e-02,-5.534142926540192020e-02,-6.971363145853580467e-02,4.766656826728052354e-02,1.475462733826608114e-02,6.839648592508534852e-02,-8.141517626988739598e-02,7.070766080244442198e-02,2.898630109795300586e-02,1.518180294232274880e-02,3.479693189498164752e-02,-5.193050444788843180e-03,2.468466200458120999e-02,-4.660184944575734844e-02,1.995446809289466022e-02,5.742051304688640390e-02,7.659732478662906185e-01,-1.622277706959446841e-01,-4.508735094591254794e-03,3.356810777781241151e-02,3.740736615208796467e-02,7.645607564170223136e-03,-3.725987471959912867e-02,5.317319459844145613e-02,-2.054540334882592328e-02,-7.255290388029460302e-03,7.166538016623201501e-02,1.231943934216704270e-02,-9.908133985333208646e-03,8.862254504641800534e-03,-1.239255560328125921e-01,1.170512838168262688e-02,-4.768720636761755033e-02,6.341883756818541412e-03,-5.986449165773007758e-03,-3.899696402823962793e-02,-3.159446794975156320e-02,-7.107329467152784241e-03,4.133660559583212518e-02,2.314816887443908733e-02,1.412356233051365040e-02,4.937773826731255145e-02,4.313911971517889882e-02,2.096823448521612995e-02,7.041038330204436868e-02,-1.364112208601269034e-02,-6.490968879775484146e-02,-7.181798020681262607e-02,-4.193286078203452977e-02,2.488030242819650253e-03,-4.320629164790306537e-02,-7.886805043438910312e-02,-3.903891676518422491e-03,-9.151572453518165909e-03,-4.407686669273717939e-02,7.351193794282615164e-03,9.108054778962602904e-02,-9.616270291022382954e-03,2.631252528089523049e-02,-2.651185274728814698e-02,4.287404587560192382e-02,-1.333283182129274894e-02,-1.350964897689775346e-01,4.694766020164595210e-02,-1.163109530333679281e-01,-2.814864727597272270e-02,-1.367118707825139279e-02,8.343063662470484587e-02,-1.097821313043156888e-01,2.014605168514922023e-02,-3.585870215288777807e-02,
3.720930949809923317e-03,-8.667721613878250075e-04,9.522053205834449252e-03,3.686103957465422520e-03,-9.312481176727234466e-03,-7.752834952427452991e-03,-1.203505074411851186e-02,7.063234516758581938e-04,-8.169073763463218646e-03,7.595545065345359337e-01,-2.225639779856448258e-03,1.032493685123421490e-02,-2.012052237619484270e-03,-4.257893472526171411e-03,-4.445813900296476107e-03,7.259791404565882345e-03,9.064637988020209439e-03,-4.059527707345939040e-03,7.881535343291716511e-03,-3.659160032533512186e-03,9.993789721730829134e-01,-1.649819528878763986e-02,5.089915900535313398e-03,-1.487369611623552690e-03,-9.660380550238229641e-03,5.170209904717715937e-03,-1.659227810831699623e-02,9.852026818771777647e-03,-8.937039888830052015e-03,-8.973701028386484729e-03,1.070209330765691663e-02,-1.273313167316621891e-02,1.287501632327797164e-02,-2.256459001335448834e-03,-2.021531950214627144e-02,4.470483494940125264e-03,-2.098977987052788596e-03,-1.251853953121439753e-02,-1.563813454293496580e-02,-6.099950858125723074e-03,-1.743910338669612051e-02,4.375315537649925729e-03,-6.834737795257065512e-03,2.807858756892949240e-03,-1.424866819454546560e-02,9.948466558980989527e-03,-4.268543509537703340e-06,2.730109123278749717e-03,2.407892842770097130e-02,4.664671812731207429e-03,-2.258362188486564073e-02,-1.752041897932653247e-02,7.356818442747003334e-03,-3.559092962402815238e-03,-6.364026511422073151e-04,-4.018177836181414421e-03,5.948969845277279726e-03,3.549418364570861711e-03,-9.905850502118299095e-04,-1.376220519729819125e-04,4.698492045104231407e-03,-2.128636909299532094e-03,6.043522589186318614e-03,-7.255045343025397914e-03,-8.317718978308852318e-03,-5.612128301682763054e-03,-2.043958924956430778e-02,-3.171391119195100805e-03,-2.398412052699155222e-02,-2.797403146216820291e-03,2.911972716766544206e-03,1.444104500375069378e-02,-9.229639250052020794e-01,6.634593654536794817e-03,-7.219428152834054258e-01,
1.912264729671441302e-02,-1.847535247001009032e-02,2.000685642343970339e-02,1.697353141771162227e-02,-3.813040531730061550e-02,-2.799801209244277486e-02,-2.942571873563191304e-02,3.670232021720455651e-02,4.574063458695556972e-03,4.601520143491653059e-01,-2.609491514797212044e-02,3.403585133178239908e-02,8.231148588327439480e-03,1.247212909796953026e-02,2.081103441156401807e-02,5.196329925851153955e-03,1.072323134707467429e-02,-1.966183942744071517e-02,-1.880635506161833226e-03,3.105117742205255882e-02,9.332832991494174335e-01,-7.718045225075359506e-02,2.020783772405938122e-02,3.490903028152008130e-03,2.124685689491422058e-02,2.354153860313955293e-02,-2.631752799580475144e-02,2.455714021042088024e-02,-1.290340980465956317e-03,-2.036228396676549590e-02,3.313713570055236424e-02,-6.185238493422874628e-03,-3.816617050059039826e-03,1.353838731647412363e-02,-6.450942036186628481e-02,3.137314958998493018e-03,-2.266323012039067608e-02,1.066588606263439536e-02,-2.390981542792571662e-03,-5.718106611330842429e-03,-6.380806540122505033e-03,-1.589440355748536482e-02,1.992107044271919022e-02,-2.552873557928506988e-03,5.414361544288568556e-03,1.431480107959028428e-02,2.088747280512441573e-02,8.120709819799599993e-03,2.632188355396161694e-02,-1.190971383073190053e-02,-2.529095560572731591e-02,-2.118496587155619532e-02,-1.427597510538631403e-02,7.605709562774273791e-04,-1.254588345834243418e-02,-2.425474053656668724e-02,-9.708444312764014442e-03,-2.534988481807352867e-03,-2.162439440730709247e-02,-6.164390056100432086e-03,4.200489622186667077e-02,-1.009260422187308916e-02,-1.224113184031081804e-03,-2.613673673442559395e-02,3.552781660745204378e-02,-1.722764137946846200e-02,-6.112105895524560378e-02,9.267587097684448078e-03,-4.841687661799046194e-02,-1.402596171640847073e-02,-2.552290018460324356e-02,3.548515148143435716e-02,-5.011606054314086123e-01,1.754201046728277869e-02,-4.475659929964996842e-01,
-5.613876129218653421e-03,-2.747131084288870900e-02,1.981771162833347033e-02,-2.038446351253814931e-02,-3.680313709577445053e-02,-5.066824355607135733e-02,-3.337563347435765215e-02,3.659290214376539896e-02,-8.111896090858099206e-04,-1.255093700698484138e-02,-3.062169291471908417e-02,3.363019317289257792e-02,6.809324227055876116e-03,2.083196516606106163e-02,7.340902786696289553e-03,6.182540373990616511e-03,6.061067463900235777e-03,-5.092415465965051402e-02,3.655782648728042594e-02,7.156103376093150836e-02,9.255235111651924917e-01,-1.313140579848328104e-01,-8.736514778159209724e-03,3.182516890966399781e-02,1.851753411371840144e-02,-3.605708055001728002e-03,-6.548162228943193996e-02,1.902288023326010463e-02,-3.415814090715049017e-02,-3.996973706737910054e-02,3.573069705310957800e-02,-5.247734559102275320e-03,2.430623971727409113e-02,8.455114095437265415e-03,-7.328112080539798956e-02,2.361708908836216911e-02,-4.301187407609187441e-02,1.079516280359685276e-02,-3.302141346976693662e-04,-1.744695415555215512e-02,-5.882126294710775249e-02,-1.481206010221142177e-03,2.454623110217700119e-02,1.298581691597511245e-02,-2.432779133048689618e-03,5.484009037079871918e-02,4.183507001456518437e-03,1.164867815020541666e-02,6.345705432258494372e-02,7.132162277764968195e-03,-5.401157329331578855e-02,-6.350752263434170097e-02,-1.842565853546896879e-02,-1.224764291208339101e-02,-1.986002599268422986e-02,-4.268221783422807003e-02,2.875985071808860247e-03,9.624740092955837156e-03,-4.731939633284498736e-02,-1.050000683896767928e-02,4.419877878752475481e-02,-1.252206492105564437e-02,2.078593146147358905e-02,-4.831311516817103635e-02,1.527816955982597147e-02,1.026717239000860329e-02,-1.224234570658181837e-01,3.427290686029215577e-02,-9.602044360217491803e-02,-3.160136826521418169e-02,3.737306589870474649e-03,7.018712937189486301e-02,-6.717881878733411583e-02,3.044124555253345521e-03,2.802519775428180537e-02,
6.651246619132872989e-03,-5.378589156969207190e-03,1.057457931583307038e-02,-2.787311816036384601e-02,-1.157353424050855915e-02,-2.509884949357812228e-02,-3.835946497807900951e-02,6.138907088311216717e-03,2.723799914601475990e-03,4.019150147090458369e-01,-3.174222410913291048e-02,6.551862005282790141e-03,2.787869520402127752e-02,8.141794638603268297e-04,-1.028246700545114127e-03,-3.687141359073262126e-02,1.411975324181103351e-02,-2.333468111449826798e-03,8.982644068342997626e-03,5.928069050600071749e-02,9.791551043353982076e-01,-5.843085658526114834e-02,-1.442726511680301679e-02,2.811455473759300050e-02,2.552024843561477227e-02,-1.333604065378119277e-02,-3.544368007840801790e-04,2.486719151552106710e-02,-3.559600575509611758e-04,-2.440599667180382709e-02,2.300359168081760400e-02,1.277075140676102970e-02,1.451123754644420213e-03,-2.460178343159465618e-03,-2.953908116231317160e-02,9.409088627408187383e-03,1.074353274177292095e-03,-1.781694679266147135e-02,-8.262764508514545275e-03,-4.362727610252515459e-02,7.079680469557876610e-03,3.639599670030551314e-03,-4.028105839767206001e-03,1.661920878163711257e-02,4.323073483695857866e-03,5.800866406991788370e-03,-2.871692319952576927e-03,8.358843904842436570e-03,-9.657846158372753592e-03,1.578794072408194749e-02,-9.029169649503468453e-03,-1.518082037816576718e-02,-2.195089933010086619e-02,1.821678995226723441e-02,-2.189498130588044336e-02,-6.737929868004552403e-02,4.590571133031112055e-03,-2.415584491852192789e-02,-3.162522594406814991e-02,-1.685282360860715367e-02,2.794628636569666119e-02,-3.010753552736660302e-02,9.695651439809603242e-04,1.793720436559526760e-04,1.034766491549815592e-02,-1.927226707428226701e-02,-3.151304283523759936e-02,4.776800679682408068e-02,-3.246422029744040066e-02,1.309874374898793714e-02,-4.570300982881447469e-03,5.217855578107293349e-03,-5.679342009923997736e-01,1.631479925568723316e-02,-3.651593445690979323e-01,
-1.076611273340271312e-02,-1.006817713486367338e-02,3.153589782568418098e-04,-5.587832736430538988e-03,-7.564523753387685651e-03,-3.471488450291059652e-03,-5.508096316125888139e-03,4.150823381430596842e-03,-6.448029400093155958e-04,7.989059182231907918e-01,-4.666455472440943913e-03,-1.664294738646859145e-03,1.621900314325476142e-03,8.157695048925963505e-03,-5.370642780096418142e-03,9.756539956679209646e-03,-3.868057095435318768e-03,-9.719529546324125885e-03,1.472523062003397790e-02,1.812069240886210464e-02,1.000000000000000000e+00,-1.795347587415602880e-02,-3.506208501806888818e-03,1.357650307590824862e-02,6.324596655402024356e-03,-5.078441100259780891e-03,-1.461262435666893511e-02,2.975958827689580838e-04,-8.904900613752675734e-03,-6.017372772180236418e-03,1.765529682631878164e-03,-2.341429596897018697e-03,-1.702001538023788143e-03,2.493336641017553784e-03,-7.770386137315690722e-03,6.528977910867521210e-03,-8.296896024953759330e-03,4.309978647944117530e-03,8.475237164090739309e-03,-1.873395153954214749e-03,-1.297187103892390170e-02,-4.373407494752493477e-03,5.089972330758019831e-03,8.284963852875203844e-03,5.842119371843305931e-03,1.398792372370287994e-02,1.974837627967251236e-04,4.606644518515607203e-03,9.782652247491618533e-03,3.844820871899569999e-03,-8.664391249195053935e-04,-7.025154593696384189e-03,6.337550756470942265e-04,-1.934390973265849665e-03,-1.650196186081438060e-03,-2.917736042808394617e-03,-5.025375177492199651e-04,7.515699580287832360e-03,-9.915636382874667976e-03,-5.935123701008214632e-03,3.962130517057741580e-03,-1.605717019834510331e-03,6.824094634903363557e-03,-9.620090969111972506e-03,4.607543798684601496e-03,4.828693022089358342e-03,-1.951298639826439321e-02,7.471277152675905031e-03,-1.277853873795332246e-02,-1.265676287044466969e-02,3.356950457624959666e-03,1.165647522972396329e-02,-7.956488885072835471e-01,4.676864625966555694e-05,-7.924242698283485709e-01,
-1.189879611720846310e-02,-3.719662300616153466e-02,3.516352987211945891e-02,8.874498173877323759e-04,-4.014013220450265984e-02,-3.298951789373388149e-02,-2.956689508416202039e-02,3.018375234333812995e-02,1.114138127649706017e-02,1.275382789242409665e-01,-4.084462315705163216e-02,4.132074403053252742e-02,1.197151248435937218e-03,4.205316412714757424e-02,4.060046255191597508e-02,1.751260420847568222e-02,2.970194076505780489e-02,-4.796313383697264898e-02,1.500418105954921810e-02,8.192247709980432502e-02,9.641260213163903581e-01,-9.349371418862745253e-02,-3.211430382041366438e-02,2.459548877712616766e-03,1.511299904994586885e-02,4.657165785500037847e-03,-5.830330652461991614e-02,2.551230818104903084e-02,-2.261709302744566163e-02,-4.615628214720901568e-02,2.150871890527787655e-02,3.574845813397967142e-03,2.861978342329193903e-02,2.423156789360606661e-02,-7.783143079868831693e-02,4.373925247224200175e-03,-4.044820061947174139e-02,4.017678931512694285e-02,-5.812258400433048533e-03,-7.082440515186968349e-03,-4.302539185997333177e-02,9.199784134800170071e-03,7.893509639532572675e-03,-1.634953425711383607e-02,2.744567979423958443e-03,3.219550284881084190e-02,4.775998170369534501e-03,8.907095868562062840e-03,4.864418579048632174e-02,-1.077284903743978492e-02,-4.122148745305571904e-02,-6.568345886058798988e-02,-1.546719134966232506e-02,-2.499633266926520880e-02,-1.082150739785401120e-02,-2.226021049568301796e-02,-1.779078562189801566e-02,7.843193267660017454e-03,-5.552744349319689576e-02,6.759122872930183219e-03,5.615184725629364382e-02,-1.382803561171349824e-02,1.310498220460658281e-02,-4.668890983355895863e-02,3.816914434828649894e-02,-1.767959048370108116e-02,-1.078308757676781360e-01,6.689451267183899041e-03,-8.817662012493307500e-02,-2.812988342111235468e-02,4.067651839884247472e-03,6.692357927389777639e-02,-3.519095254069837253e-01,1.501532084062213307e-02,-1.237708232329050584e-01,
-1.247721763857813013e-02,-9.618452512789980174e-03,-9.436401941931536752e-04,-1.890093984526629300e-02,1.782724645501707061e-03,2.634818707388089654e-03,-4.681052280953432176e-03,-2.374636317366611571e-03,6.898736667974940916e-03,8.358158040803455036e-01,-3.821990753098303273e-03,-5.758687632312575744e-03,6.004786215451404785e-03,5.100909369585744646e-04,-1.280353770346463749e-02,-9.992586471063312839e-03,-1.799578751793859399e-03,1.524417840467653940e-03,5.781162529822553298e-03,2.240985759890395945e-02,9.920959469554713195e-01,-6.104087964999976856e-03,-1.078315686583595015e-02,1.030788913582671509e-02,8.375624423445783562e-03,-1.080785524674046476e-02,-3.479383113097691757e-03,5.815192258452880902e-03,-1.079587406023221549e-02,-1.038330268339312731e-02,-4.293550997539585801e-03,3.829063108495202143e-03,6.558709250612006214e-04,-3.092713861870843330e-03,5.717938476653472435e-03,8.814277373720612319e-03,5.957648182420317851e-04,-4.770960487530163247e-03,1.105644587870041780e-02,-1.095745588331266396e-02,-7.532439759481714646e-03,3.851065774380014409e-03,3.571215220647752653e-04,1.084182603651855786e-02,6.844341067154412553e-03,6.023522631025101867e-03,-1.315996309932673629e-02,9.698264227452416120e-04,-8.008251999695473189e-03,9.117816316857147232e-03,8.768205981936422200e-03,2.399864729299289179e-03,-1.778778781461019115e-03,1.893410018834556195e-03,-6.924137751584315964e-03,-1.245811918058200435e-02,6.848672616248295542e-03,1.790902622472320682e-03,-1.156043698569065442e-02,-1.408633452475706316e-02,-5.808875751970157819e-03,-1.870281432666057869e-02,1.915207640619687873e-04,1.201806031588915219e-03,-3.198662511649678908e-03,5.516099920306348492e-03,3.380726362745667402e-03,2.566088198109409829e-02,-9.218129448958431449e-03,5.810517838053182692e-03,2.919068570080816244e-03,-1.157074473382006113e-02,-8.816749380998382124e-01,1.512553471716842485e-03,-8.052591734671850077e-01,
-5.636760916158145562e-02,-8.773998226983936721e-03,2.456582124208649826e-02,-8.262980101045220432e-03,-2.272952441979573385e-02,-1.885682219955826106e-02,-1.219026827804464003e-02,7.344894358883190776e-03,-1.121254001139111028e-02,3.267117582004837595e-01,1.736517269198623037e-02,6.548440690118728646e-02,2.095083042908138168e-03,2.963801108707424595e-02,1.594548791630932827e-02,3.658960469202556659e-02,4.517243752292295945e-03,-3.996775782569823160e-02,3.241828419375886172e-02,3.715852918305867503e-02,9.698738092736091110e-01,-3.496863566538408602e-02,-1.431065966761580280e-02,1.883720928017104754e-03,-2.408138490824253755e-02,-1.224191803005557008e-02,-4.338458753226267367e-02,1.103794447333903382e-02,-3.049715663732292109e-02,-4.403715928546318714e-02,-3.784163782880132360e-03,-7.496260942690263773e-03,2.749968610614683673e-02,-4.435345518978187768e-02,-1.246819661085035030e-02,1.680931581173825129e-02,-1.989097668280866496e-02,2.692831550019313125e-02,3.114785175714056709e-03,-1.871402322686335079e-02,-7.203138539195014234e-02,1.376677502937762379e-02,-1.790403791619385801e-02,-4.444928835371312822e-03,-1.022346857209897354e-02,3.491203014958603468e-02,-1.017385533890031481e-02,1.761130806402526339e-02,3.772553176243485101e-02,1.836706207030242358e-03,-4.904558007181571144e-02,-3.106561525665227017e-02,1.902543381318717863e-02,-2.508922876803064764e-02,4.301822550582453178e-03,3.713075775495907477e-02,-1.200602296246687517e-02,4.569377262988225108e-02,-6.809612176471541156e-02,-3.679723746777138586e-03,2.909220495325698375e-02,-9.469914731743890590e-03,3.627198810372011129e-03,-4.159221808479224985e-02,-4.843065748037340967e-03,1.802885273357708595e-02,-6.978866287587726092e-02,-2.178007139086168137e-02,-6.594564655847032220e-02,-5.369036500214231789e-02,1.414297220039450326e-02,9.257209124411536205e-02,-4.007030861803348443e-01,1.192673129172408877e-03,-2.870373280715984299e-01,
-1.324602890431923231e-02,2.894972091792060581e-02,-2.169290320438147998e-02,-5.793221839313843213e-02,-1.295962523331033178e-02,-5.296474787066637879e-02,-1.853897541621325057e-02,3.793900714241217175e-03,3.193139687568855370e-03,1.380345502845322647e-01,-2.997957065965031493e-03,-1.047214404744901094e-03,-1.901715596438880492e-03,1.831230068653253232e-02,-5.232016139533154919e-02,4.130140463141449697e-04,-1.625568351266452188e-02,9.423691699521168086e-05,6.546821292003028592e-02,1.019045706996884043e-01,1.000000000000000000e+00,-2.229287110601969879e-02,-1.256775140543048980e-02,2.629008234319037129e-02,5.919704769093144664e-02,-3.052891585458664472e-03,8.770121429581091699e-03,5.973840753403909798e-03,-4.604699643688466709e-02,-5.940241014457788710e-02,-9.997581556056997498e-03,2.353598089274838712e-02,-1.856783852179183861e-03,-5.961852282357936461e-02,3.875897274157852346e-03,8.817348595650496834e-03,-7.848157208598922560e-03,-4.784492727355673519e-03,3.432411312239708884e-02,-4.686640685989606342e-02,-2.501892774901148114e-02,4.045232856891067819e-03,1.427050560848689038e-02,3.610808103074111819e-02,4.513281504126586546e-02,3.049495867447222369e-02,-1.125234629808213004e-02,-1.363915505954352465e-02,-1.618577778378271076e-02,-2.666292446441911532e-02,6.427658370144304252e-04,-1.481978100970663195e-02,-2.206728515326322054e-02,9.693653598990648740e-03,-2.867795774861644706e-02,-1.492817721151385758e-02,1.701180746780289743e-02,-1.195192466881416296e-02,-3.282090322688277495e-02,-3.286188412357701760e-02,9.956520357241843699e-03,-4.739277875127847345e-02,-1.495076085401680389e-02,-1.368464465411355253e-02,1.482615185818711261e-02,-1.085444124501693842e-02,-2.239020227063743401e-02,1.010192433440273985e-01,-1.740921625483800306e-02,7.917263211799141498e-03,-1.913531570113344293e-02,-2.931767071167965671e-02,-3.064867606368295583e-01,2.443002561940057007e-02,-1.492609934212995038e-01,
-2.943574226612912480e-03,8.627709328503617436e-04,6.376320805459018484e-03,-4.831898319773916123e-02,-3.056178811884829871e-02,-3.495678169799258661e-02,-3.875584082674017605e-02,1.921194136380462852e-02,4.317804972400119583e-04,3.197106611487816608e-01,-3.050634991427991771e-02,2.033585515685890771e-02,3.562006243465018485e-02,8.930484468698547673e-03,-1.617337168557420740e-02,-3.353964504293758392e-02,9.971426119337134172e-03,4.588697791198463605e-03,1.537913623514104711e-02,7.729664465410523988e-02,9.949551388147418463e-01,-6.369664768532601329e-02,-3.158906111181399023e-02,2.423155868814571776e-02,3.822880334374875733e-02,-1.019580787078095373e-03,-2.492373752750312588e-03,3.106382987853360811e-02,-1.056609696232518186e-02,-3.591042833568594550e-02,1.898159827233084074e-02,1.233003525167419671e-02,7.253920181121822040e-03,-2.507529589403018760e-02,-2.709356313367441982e-02,-5.836900680246639300e-03,-5.393019796172501823e-03,-1.021141826977593932e-02,1.697949847244500238e-02,-5.279892640892745737e-02,6.841668455570534824e-03,1.019775877820496522e-02,4.508577654259420670e-03,2.188782277517103275e-02,4.919546076445683452e-03,6.987490087096800973e-03,-1.519738926871981241e-02,2.329206075797972050e-02,-1.254396607544215614e-02,8.457257141790103239e-03,-3.069522845704220071e-05,-1.616700231739258420e-03,-2.169588855122047549e-02,9.249475872886743846e-03,-3.423672827660345641e-02,-5.957453229313370430e-02,2.160538389666745440e-02,-1.772882469298670688e-02,-3.354996644858113802e-02,-4.289193973608715349e-02,3.768677678824569610e-02,-5.074880177576583662e-02,-4.835680695535037737e-03,-5.843871396238299598e-03,1.543343070416939092e-02,-2.624341534537256646e-02,-2.618420491352149715e-02,7.175751391748710906e-02,-4.111685462596956220e-02,1.289443352297209126e-02,-2.269713638171339670e-02,3.259044551464272688e-03,-4.876499144667191254e-01,1.903855085188819671e-02,-2.719042140882688696e-01,
-5.252453422440876013e-02,-1.484051496336290243e-02,6.555968239289807278e-03,-1.515941370759982948e-02,-1.919117797014694912e-02,-1.397232645590870855e-02,-1.248910515794978554e-03,1.712253520483783889e-02,-3.137672560644913511e-03,2.372058662742067559e-01,1.513961027501463386e-02,2.125353031281260532e-02,-3.001528833819895603e-02,4.720514994271559489e-02,-1.654778677860382907e-02,5.575331478444043448e-02,-2.346703995257889280e-02,-4.232323570742217711e-02,4.627465399097486359e-02,6.140638852200798470e-02,9.870636513828140446e-01,-4.249761908964869161e-02,-1.709177027479183777e-02,1.616656252455711540e-02,6.699491206886243506e-03,-2.423141196942707118e-03,-5.935152714866816226e-02,-2.613422741098739462e-03,-4.883441382950856613e-02,-2.281139400515968738e-02,4.166823410825110097e-03,-3.363664637232279681e-03,8.871469920445976265e-03,-2.321283239877372159e-02,-2.528113949138604058e-02,9.753659229482658163e-03,-4.146599408596228498e-02,4.313638110273348286e-02,3.993075079567279251e-02,7.476311702223424616e-03,-6.069603263002437687e-02,-2.428538506707711431e-04,3.339616128109119952e-02,-3.458461414180061255e-03,1.372507587005369380e-02,5.874472069424794413e-02,2.652319922661267038e-03,1.378793228316810976e-02,4.572671487157489723e-02,-9.711184526739299189e-03,-1.076188458854922606e-02,-2.505461451655828509e-02,1.189454869237081477e-02,-2.673573133956392428e-02,1.287255635247004648e-02,4.135587855144418068e-02,-2.501642900676394332e-02,4.253480229008408525e-02,-3.041542996159488435e-02,-1.348251831656462410e-02,5.048999012284537298e-03,-7.306615792791556173e-03,7.972611935444143261e-03,-5.161442761175336069e-02,3.654559260475941096e-03,3.479406008627075608e-02,-4.956269965032253366e-02,4.868786425506199667e-03,-4.339400844172264238e-02,-5.418991328260658324e-02,6.487945401007562396e-03,3.486871616374445648e-02,-3.771116788327921188e-01,-2.166005298730101780e-03,-2.070078917094605453e-01,
6.521161273363641764e-03,-1.240525673648592946e-02,3.922444544224601773e-03,-3.464641603444946344e-02,-3.020184492987613295e-02,-3.663004543198583596e-02,-4.263411584765312845e-02,1.321441792085909644e-02,-1.699819041596480583e-03,2.976101960372452560e-01,-4.255142552896983010e-02,2.853165165444292509e-02,2.067355568674481003e-02,9.715087396160431571e-03,7.337096907212210285e-03,-1.062042335232049259e-02,6.616987729497345733e-03,-9.515914301620744323e-03,2.281735414714578664e-02,3.041127861238917401e-02,9.440993092519459617e-01,-6.471714430046109079e-02,-9.787349141908799691e-03,3.491426883276994853e-02,2.138578419639758713e-02,-8.290830515974560722e-03,-2.169392326229785742e-02,2.501634449748357802e-02,-1.569261322403322709e-02,-6.796072668636444179e-03,3.471613516310979863e-02,5.686347269766179463e-03,-6.456660678792495653e-03,-3.760926457415793983e-03,-4.761584323126486634e-02,1.383820433910847719e-02,-2.591835631871767728e-02,-5.788348578564181390e-04,-5.603839867934909055e-03,-2.151098358684516709e-02,-1.145688644487832288e-02,8.333779400771106766e-03,1.579017217491164929e-02,2.582223275059725501e-02,7.768762857523260970e-03,3.415047965690765075e-02,1.162832733970407817e-02,1.845669746037834243e-03,2.012554621683882924e-02,-2.722590846528802344e-03,-3.163336982052673135e-02,-2.939682325808248148e-02,-3.168622287075695265e-02,1.557410447919457992e-03,-2.216936152584141409e-02,-5.006743001100170953e-02,5.611177931111495376e-04,-2.815163873887666091e-03,-2.975056147316489791e-02,-1.923194894023660256e-03,3.723109623687285291e-02,-8.200800347743153126e-03,7.033853489264253744e-03,-6.947046542417681207e-03,4.826227762799756293e-03,-6.179200599881442407e-03,-5.564946591900604006e-02,3.172301081516779031e-02,-5.941604218747371935e-02,-1.587634035594824924e-02,8.489734596752165560e-03,3.253976302310658542e-02,-3.971012345580177128e-01,1.264538797144708696e-02,-2.701577731606152888e-01,
1.138444118420657569e-03,-1.520885537573480120e-02,6.787376662791024667e-03,-1.904025723444482260e-02,-2.680339922995404744e-02,-2.324047759565121135e-02,-3.362749172300488754e-02,1.790195241893770273e-02,-1.731442234935280568e-03,3.872419461417705433e-01,-2.639823048658191881e-02,2.990957556516587318e-02,3.081171980876137861e-03,1.703588185136134178e-02,5.126774037616778357e-03,3.962027639271632068e-03,1.089239704424660449e-02,-2.225439532046464761e-02,1.536064028118493477e-02,3.942159828261604382e-02,9.754133215085216690e-01,-7.003457227648121619e-02,-1.491454319146274035e-02,1.426263145496571225e-02,1.759274019267462880e-02,-6.337094525942173207e-03,-2.721906390614021315e-02,2.022975988594492847e-02,-1.792124543566059325e-02,-1.096234106929463645e-02,2.678328492068095604e-02,1.052615384544113367e-02,1.521462923417380222e-03,-1.038782677664451363e-02,-4.070634572590800526e-02,6.523332032929448840e-03,-2.639884841763621287e-02,1.077644804389958286e-02,5.702130232281492350e-03,-2.234050211046557585e-02,-2.107017450780703308e-02,6.015934790320403430e-03,1.868772870178342024e-02,2.434266336377305384e-03,6.403316569737643391e-03,2.885889083539892180e-02,9.059225446080079736e-03,1.695810747828010306e-02,2.662871175319040523e-02,6.674304473912387459e-04,-3.146710722533421767e-02,-3.019861523316630381e-02,-1.167036814797757852e-02,-3.873310668098837138e-03,-1.081470874977457156e-02,-2.825606937627510307e-02,-3.799710638816321435e-03,3.205435615865452788e-03,-2.196492795696013828e-02,-3.220477313002106240e-03,3.368531726703245105e-02,-1.090656628085020369e-02,1.035533987771794853e-02,-2.492049757755170764e-02,1.318603818238352036e-02,6.398793709032350903e-04,-5.800449211518723946e-02,2.746022259016989153e-02,-5.083293025157394063e-02,-1.362414215745854501e-02,-7.395896861396005634e-04,3.368357718683223645e-02,-4.723166000882061955e-01,3.824378512933249066e-03,-3.632515396216159376e-01,
-2.382527793321150000e-02,3.467257491353564671e-03,-2.015774443439459496e-02,-1.768464267445948521e-02,-4.340663124060978317e-04,-3.044041807282787254e-02,-2.264028529955643566e-03,1.676410996369770076e-02,-6.594959549594365741e-03,5.640489911389869881e-01,-2.207200814016515959e-03,-2.134942337507033175e-02,9.333493948502536060e-04,8.462997239683292192e-03,-1.830906188575501595e-02,2.957209418197226554e-04,-8.813366195357219546e-03,-2.853373943784972763e-02,3.502615087523379411e-02,2.857269397082037171e-02,1.000000000000000000e+00,-4.651571823110717957e-02,-1.236446212570438581e-02,3.114552113746172843e-02,2.481859799731456553e-02,-1.995737333612560699e-02,-2.549876500175484589e-02,-1.013109441738744860e-03,-9.410709939325427945e-03,-2.906888075425972159e-03,1.371479893940782263e-02,4.028998065417898093e-04,-1.055833590252220483e-02,-8.998070300177849964e-03,2.030264831499517617e-03,-3.333906722498095835e-03,-1.587723250594436433e-02,-5.672035204845768570e-03,5.460272836050441940e-03,-2.669724596052418589e-02,-1.666662476009489469e-02,-2.478678466397490204e-02,1.195803343094451257e-02,1.071637461423259607e-02,-2.923631460134191052e-03,7.332461773592031766e-03,6.379005698990971099e-03,1.316085734936921214e-02,2.272130144844822230e-02,5.405871308782428906e-03,6.922894488448219988e-04,-9.383178697360097451e-03,-2.493507981367188309e-03,-1.382863436464778616e-04,9.402410272769116256e-03,-3.233593814359893986e-02,2.376351467207158015e-04,-5.254047260551137494e-03,-1.615838373192908517e-02,-4.827313225409519785e-03,1.092380311970221618e-02,4.322525695509967132e-03,4.735498897940381216e-03,-1.663208444168117611e-02,-1.305119924019108694e-02,1.939676653294083661e-02,-3.075248999260090516e-02,2.785212298572814493e-02,-6.749822402605922499e-03,-1.423342218240353403e-02,1.370886244570966757e-02,1.050405215094852782e-02,-6.400657418050930136e-01,-1.621082127254468500e-02,-5.543898930709003636e-01,
-1.414232866106647848e-04,-1.236725581498052551e-03,-2.478696370818666422e-03,-7.432382462120273002e-03,-9.690187045817550554e-03,-2.814000409411321346e-03,-1.988877097247709891e-03,1.220421597144596970e-02,1.772546405242436673e-03,8.413825339315174068e-01,1.270205458321771452e-02,-2.911860157362852405e-03,-4.626137972740994309e-03,-1.006161636893419808e-02,-2.977772979979734225e-02,2.295058615008404440e-03,-7.756262867024020166e-04,7.672653513153260968e-03,7.984204445926434621e-03,2.168696986486438189e-02,1.000000000000000000e+00,3.858046407391364091e-03,6.821859124041741718e-03,4.422888116776295050e-03,1.547335777995682143e-02,4.910134384203072161e-03,-1.150819137066697376e-02,-1.202173336465233583e-02,-3.562712403106072824e-03,-1.842725287182030158e-02,1.933764702373685831e-03,-3.970375761704968143e-03,-3.265526666823449255e-03,-4.181522821655774685e-03,6.947413927870967065e-03,-1.044506054200973998e-02,-4.863819731861206315e-03,9.033641786333583557e-03,1.870107312143133582e-02,1.370935549911911355e-02,-1.070231643880238670e-03,-6.554754573439992849e-03,1.230037173139118117e-02,1.048652529025339598e-02,6.311226335405225804e-04,6.549787107701616345e-03,-2.572672174319964519e-03,-5.339313596738837209e-03,-1.309137479859700214e-02,4.184025232137856751e-03,1.090599417070804364e-02,8.592612398560446271e-03,1.192050070261119893e-03,1.112635168694486821e-03,6.067726051695015305e-03,1.301778747719888746e-02,7.141446431835262940e-03,1.473781112109193159e-02,1.056524603027319403e-02,-7.314433389694723064e-03,-7.280345314823855052e-03,-9.728496102529710088e-03,-5.139375391096034310e-04,-1.264685939628555633e-02,8.351030651260551736e-04,5.479138867675439895e-03,5.262702366325779785e-03,8.465371367556401294e-03,3.786322482259660976e-03,6.720799188977815558e-04,-1.752214994691674368e-02,-1.997818102367649590e-02,-8.848880357587387557e-01,7.483705468401273715e-03,-8.330962968596935525e-01,
-3.041732814032445040e-02,-1.989019761219915777e-02,-1.093415731697986240e-02,-1.448685736732515175e-02,-8.960454563180699633e-03,-9.029828368094166419e-03,-8.653065633647080701e-03,1.869432982595876230e-02,-6.345451357884996160e-04,5.526210380957070623e-01,-7.558917224525835490e-03,2.666418840549064639e-03,-1.438590046423518162e-02,2.840440647976719829e-02,-1.573801042216180918e-02,2.134234840790556406e-02,9.596921765084226452e-03,-4.552499206160535300e-02,2.388975976268272947e-02,5.611231206818569767e-02,9.627072404819819740e-01,-4.513640393298647829e-02,-2.948776206346358578e-02,5.169531387353308929e-03,1.835461957821915568e-02,-1.359221908201407175e-02,-3.210419137470987144e-02,5.953158944184133716e-04,-2.691701469889944132e-02,-1.662395809934346866e-02,-3.754187205433318769e-03,1.367981412057822643e-02,8.951861166978516277e-03,-1.220555391222105657e-02,-1.697912587213901148e-02,2.600408490574551701e-03,-3.227472467114944621e-02,1.660085778842604332e-02,2.737022236380103057e-02,-6.568838309907881062e-03,-3.200596673641865897e-02,-2.805054913178299872e-03,2.159988689869135614e-02,-2.604806112751611308e-03,1.675553390133238674e-02,2.668988294898618982e-02,-1.028971378826631748e-02,1.334472599474501725e-02,2.732776112109186820e-02,6.406287760915231957e-03,-6.887171898929789204e-03,-2.177183852492362709e-02,7.378675935696188404e-03,-1.418183456348346208e-02,6.451415612169129898e-03,8.375209732972982435e-04,-1.421558843154645431e-02,1.388402419007944526e-02,-1.502671701017342307e-02,-8.858878988644470695e-03,8.591800331524651613e-03,-7.608753719000603197e-03,1.070510843316974971e-02,-3.202433430487530708e-02,8.551600938929041634e-03,2.003817249304143053e-02,-4.195968844861506369e-02,3.009426041807782698e-02,-2.550322474997541305e-02,-5.742919653424911376e-03,5.077299077483185194e-03,1.245470731317083191e-02,-5.212447295147115689e-01,-7.343570090093953784e-03,-5.458659375185176410e-01,
3.290733799569796336e-03,-4.029258004899454945e-03,2.649226454900645983e-03,-3.591999649467911976e-03,-8.784382746841595813e-04,-2.556473540218106438e-03,-8.785009157963687143e-03,-4.909874288483193766e-03,1.484423362752918321e-04,8.821149103774177380e-01,-9.730706469963277053e-03,2.100169453330226477e-03,4.462771603783866579e-03,-1.345548406572753125e-03,-2.091024600375014815e-03,-8.501580049406492687e-03,7.702365285687269927e-03,-5.224825532805302264e-04,4.351109338256727022e-03,2.860095493029526925e-03,9.989206955302133428e-01,-6.248864767793238517e-03,-1.931242850769458036e-03,7.540284500984990868e-03,-9.268919668242768875e-05,-6.159141186943159658e-03,1.701549480827228844e-03,4.687518237949175881e-03,3.532427982366800195e-04,-3.020305975589197139e-03,3.939991118973535343e-03,9.546865401904238053e-04,-1.482505460353602741e-03,2.897967379729995269e-03,-8.862413143029653634e-03,4.131368478553856111e-03,1.024459599131152809e-03,-8.860357927523666507e-03,-3.474645005889952752e-03,-5.359576338797130551e-03,6.782173617386408337e-05,4.210215695487197138e-03,-3.296184366442226569e-03,5.000863852958779485e-03,1.412449067934324048e-03,2.868566562824515637e-03,4.340704244062982414e-03,-1.239462320943669413e-03,2.506283989304672680e-03,3.634900859167174408e-03,-2.424205676271103810e-03,-6.062539216759546545e-03,-6.608407120489188241e-03,3.976170124745383519e-03,-4.524601880550272945e-03,-1.511973977919274341e-02,1.848592070930236107e-03,-5.310077012703487420e-03,1.030203650205840926e-03,-1.132884113696471309e-03,3.736456761883593519e-03,-3.888135314967358647e-03,4.500290112201951395e-03,4.914786951570600942e-03,-1.855890039365726621e-03,-4.802554122868457273e-03,-1.563145693635623057e-03,5.749137428811203210e-03,-6.032222647426067896e-03,3.208512898735818575e-03,6.188613074418428811e-03,-1.756708456389019513e-04,-9.234717310134307189e-01,2.607881932959459949e-03,-8.701523357281047977e-01,
-1.247125862688390832e-02,-4.018194201891664430e-02,3.247749857222535785e-02,-7.666763590545407764e-03,-5.127869995212188531e-02,-4.008606883358724338e-02,-4.651350648111660796e-02,3.185755643301373119e-02,5.816314480434930934e-03,2.469046371817285035e-02,-3.991565089729544324e-02,7.604141278991718989e-02,-2.284502510624656160e-03,4.642278056743812803e-02,3.916874423541166078e-02,3.579784943943686848e-02,2.363887902555488613e-02,-5.440781905770629423e-02,3.600473878775718728e-02,5.800258042339433845e-02,9.347453409485482201e-01,-1.274554884376200847e-01,-1.543356737621344107e-02,2.214374703524966612e-02,9.954271509168171814e-03,-2.340756832356194646e-03,-7.191484927071284394e-02,3.281415080691603425e-02,-3.812666477581429930e-02,-3.257130626215640201e-02,3.945233450399560082e-02,6.607647957139559797e-03,1.539580920266788062e-02,3.261063164858654315e-03,-9.480201836564308704e-02,1.424410901032155050e-02,-5.527873533989068056e-02,3.843351453687109121e-02,-5.941220080589811395e-03,-1.250257893225563063e-02,-6.689723792009562298e-02,1.253916359030047417e-02,1.804756349725411199e-02,-4.784334133225947965e-03,7.222300194911669344e-03,5.917241420902440152e-02,2.659101443024625144e-02,1.871802114028649905e-02,7.908316630645023659e-02,-5.909243498891287444e-03,-6.984572260294151513e-02,-7.597240901545962433e-02,-1.710423857417943394e-02,-2.475033035789650504e-02,-1.116044063130320826e-02,-1.877247117489887901e-02,-2.175430877000467805e-02,2.556077828043025468e-02,-5.685050044233970479e-02,1.464807488671844918e-02,7.108109972117146458e-02,-6.294176143530134243e-03,2.370709342412814780e-02,-5.487395619361321725e-02,2.971541683493679475e-02,4.652647478429408276e-03,-1.422689528040797513e-01,-3.390713808559736586e-03,-1.168747684379111418e-01,-5.870721739430426883e-02,1.183216896601506536e-02,1.019022756689526854e-01,-1.831230103784613483e-01,5.355256258918058079e-03,-8.798853920502391901e-03,
-1.018705745261146156e-03,-2.790388484761883309e-02,-9.039219430443695691e-03,-7.448720376473930149e-02,-1.432792721089207189e-02,-4.601247668794036794e-02,-5.707890581181992995e-02,-2.383085560931834379e-02,1.373444802627373992e-03,9.016637209043221191e-02,-7.364680685429048079e-02,2.651523430289331670e-02,3.286698650915294284e-02,9.961327007898406610e-03,-1.801952347637914009e-03,-3.627132570443632187e-02,2.070459759707490854e-02,-1.739044583520730700e-02,5.123729682638226979e-02,7.879726787302658575e-02,9.782541329603250402e-01,-7.314730640275765583e-02,-3.409238482557302996e-02,5.446101145011876532e-02,3.709740653667994992e-02,-3.624074552258959442e-02,-1.371936843829640179e-02,3.691849302256833276e-02,-1.966053115053514183e-02,-1.580410213796332869e-02,3.184710427125810039e-02,1.356930513518943282e-02,-9.124390466293035018e-03,-4.698914426837935004e-03,-5.813260796443166928e-02,2.672435364961195381e-02,-2.558854855996276975e-02,-1.696780774177110429e-02,-2.806813272101818524e-02,-3.202651245005757458e-02,-1.577202856551391391e-02,1.463753519314272243e-02,7.360983134588230947e-03,4.495282093112504262e-02,9.600897639052599630e-03,5.195923162669707773e-02,1.443667019937611151e-02,8.049461077985820001e-03,5.649556354626647514e-03,1.445083928259043374e-02,-3.520535799670417021e-02,-5.407524878338476615e-02,-4.937083541596856695e-02,4.887289275209712180e-03,-3.120914128708267982e-02,-9.681579046977742409e-02,7.934486471132783103e-03,-1.844055835846514224e-02,-2.608410992587875676e-02,-1.233180855524932187e-02,4.104982718966292132e-02,-1.343313986231228427e-02,2.951647864865222792e-02,1.034522654388095320e-02,-9.101595651894681116e-03,1.041034985542194785e-02,-5.384998151430697710e-02,6.855370342515133830e-02,-6.847114284473108936e-02,-2.838574918298242961e-03,2.336402615086638490e-02,4.567712308442858915e-02,-3.404036879630318868e-01,6.205200261737167655e-03,-6.749380045093289482e-03,
-7.274865530204795150e-04,-4.063565447790387988e-03,2.765702558322633009e-03,-4.018165779498442114e-03,2.239383172770920138e-03,4.686736405544261741e-03,-1.183487749308088921e-02,-1.252806221869282033e-02,-1.546880550687045053e-03,7.260826241391586944e-01,-1.018491209553240573e-02,-1.106213952304951897e-02,2.403302609690911862e-03,2.296998130214521205e-03,-5.721694309044227100e-03,-6.976522493669312652e-03,-1.119931841921328886e-03,-1.746895104609246657e-03,4.737129566595523616e-03,8.878310088589913551e-03,1.000000000000000000e+00,-3.895552484456388243e-04,-2.202323232669292641e-03,1.300870306041594648e-02,5.271829887907614198e-03,-1.610732237970832065e-02,9.673494203326082896e-04,1.419567757104778227e-03,-2.355848003874104826e-03,6.285640849157378418e-03,6.507650767997100183e-03,6.692721436649547682e-03,-8.091397896555066471e-03,3.672314245189121917e-03,3.640026036867388681e-03,9.659787970641039306e-03,4.010728829173585558e-03,-8.137016722136848970e-04,1.347049446714914528e-03,-1.263613647353238612e-02,4.831110035953441184e-04,8.287465790224504691e-03,1.830856502469288896e-03,5.806730560562135303e-03,1.266904629409558100e-02,7.201925264685347938e-03,4.720318308665018281e-03,3.662210534292682090e-03,-9.986782837556808234e-03,-7.706899812402051575e-05,2.355943238931804129e-03,-1.284308704662007194e-03,-4.202421110946027993e-03,1.221879377124605003e-02,3.891926244235302534e-04,-9.896831727402924384e-03,-1.065166422715396743e-03,-4.218845804972079158e-03,-2.476964035230067029e-03,-6.050726846630605227e-05,2.925092713064222916e-03,-4.491281772804465999e-03,4.352618346590932108e-03,-1.966529663463022215e-03,-1.027064970497780505e-03,-6.445384912665205746e-04,7.322784519171759987e-03,1.883818226339206703e-04,2.181697523684966722e-03,-4.006298011565771061e-03,1.076503757598851196e-02,-4.862332985267822649e-03,-8.044712841895569344e-01,8.649718836711076109e-04,-7.019897940568214922e-01,
1.111606836133229026e-02,-2.126225443487879483e-03,-1.899046812345078945e-03,2.283840449994348730e-03,4.265169419602319115e-03,3.485045406232394061e-03,-1.932082315527271742e-03,-5.660604887961621402e-03,2.560634875896798234e-03,9.265043485097568254e-01,-7.587597519100113333e-03,-8.632732002723337780e-03,1.770751112942401187e-03,-6.138103292643904557e-03,-2.450957017510046500e-03,-1.017223618286044150e-02,5.101481585241552860e-03,8.074643601822573818e-03,-3.043976090789777059e-03,-1.128568852632105823e-02,1.000000000000000000e+00,5.365587453211377936e-03,2.898195157948266557e-03,2.382391289056080483e-03,7.083563151782931245e-04,-1.719988188026141084e-03,6.744006282479184447e-03,3.702376592279541662e-04,6.033157407867482773e-03,6.748929843191748060e-03,2.838417833951098479e-03,1.669223448716314817e-04,-4.541888931235142819e-03,1.161460507525792843e-02,-1.826425624535553058e-03,4.920549503166890722e-04,3.969759163848941147e-03,-9.523055295529946757e-03,-3.629405143179729835e-03,1.445273474244270114e-03,9.699130466154533628e-03,2.953658590317092148e-03,-1.249058404577807826e-03,3.983870171953802358e-03,1.424618714686063260e-03,-4.544957942527653658e-03,5.515245053733061620e-03,-4.287665136521087370e-03,-2.472137680212019642e-03,3.395857609096950350e-03,5.399305054297179676e-03,1.653289849504925232e-03,-6.931506598580164207e-03,5.268725196134660740e-03,-3.509589457196154658e-03,-1.249614775995542063e-02,4.047290350755602777e-03,-1.039101121002534989e-02,1.314782789047669320e-02,1.764053316843734644e-03,-2.899832927888233020e-03,1.103681064535148512e-03,2.126156897373832066e-03,1.128914508580674929e-02,-1.679900611167541935e-03,-7.105111356534529223e-03,1.157201150486881526e-02,8.689063632074840726e-04,6.856732343850101968e-03,9.072389890921541550e-03,3.221481019686913555e-03,-1.402993065534642418e-02,-9.687620133724358373e-01,1.996457035989227750e-03,-9.141951080912195104e-01,
-2.644518720256588962e-03,-1.674462961299915884e-02,-2.728967707700313827e-03,-3.728864760582509263e-02,-4.768590152825665646e-02,-8.062776981832792123e-02,-3.222603957570223387e-02,5.031523669629475726e-02,4.003109150750252046e-03,1.214856768740630866e-01,-5.571656263740221160e-02,7.044824991239670431e-02,2.883881206477699466e-02,6.809237674623219369e-03,8.213793422617916840e-03,-1.018876647137593669e-02,2.121959617916699056e-02,-6.175775904573854030e-02,3.642666172126290264e-02,8.433885790781989189e-02,9.056776695424426649e-01,-1.430172778908302966e-01,-3.377027216358704370e-02,3.466165274342739022e-02,2.578262495789947900e-02,-8.285717126283592693e-04,-2.323971447452462258e-02,3.903307248424848358e-03,-1.570640326177908525e-02,-3.493297423539370311e-02,4.122187918907944254e-02,5.318227181179337182e-03,1.637061113985180741e-02,-4.679523017895519110e-02,-7.946174752678469844e-02,8.214027343969899919e-03,-5.214241016363668074e-02,1.357169959672184512e-02,2.509368098622807158e-02,-3.603868821179639220e-02,-4.097789490857466771e-02,-1.736807053321564459e-02,4.487379014411158051e-02,6.936854186725346233e-03,-2.389307850776720143e-03,3.217409463952059651e-02,1.962619373975498654e-02,1.349763786642164953e-02,3.951994738966081722e-02,-2.054871488115414535e-02,-4.204864345254767655e-02,-6.249096138234986469e-02,-3.718312296281678364e-02,-2.418484827279724844e-02,-2.983775831267646006e-02,-5.260376988961103228e-02,1.317401360768312435e-02,4.855380224180048943e-03,-5.622157414388271462e-02,-1.459755005580639503e-02,5.561279590211599955e-02,-9.204117338872601362e-03,2.143711541517689251e-02,-3.807962628929492666e-02,2.245770242677776196e-02,-9.801142029165262753e-04,-1.116741207861722540e-01,9.522663192599145621e-02,-9.269188639900322046e-02,-1.385118375851028712e-02,-1.983356398617216326e-02,6.123665486209493164e-02,-2.092311508774025164e-01,-5.517356428465989421e-03,-7.731757057496024821e-02,
1.225882422472701627e-02,2.527781248057803243e-03,-9.417071683974764795e-03,-3.675415767830259028e-02,1.472245955100804499e-03,-1.813238111130391350e-02,-9.892508824346786522e-03,-3.789948175011159659e-04,1.530888601852300329e-02,5.486021028256817811e-01,-2.027691400088274531e-02,-1.213899558200346937e-02,1.691321046259708361e-02,-6.620848005248675117e-03,-1.675647096207714959e-02,-3.092774807090770128e-02,-6.849598924744709771e-03,-3.774559668087413764e-03,1.490928252448343871e-02,3.804794070333022971e-02,1.000000000000000000e+00,-3.287602667287231606e-02,-1.195932686429558578e-02,2.326948652963372471e-02,2.944754067132180492e-02,-2.366866738509948095e-03,4.828972131848657812e-03,4.802362679380297802e-03,-9.280533957134337597e-03,-6.210751553162655036e-04,1.910623112441603433e-02,1.208913561683418181e-02,2.672482986271811078e-03,-5.224846895002965841e-03,-1.283806199632436806e-02,1.120477319557047075e-02,2.595328754520596221e-04,-9.680669128522563505e-03,1.189523444329360080e-02,-2.284969681516899054e-02,6.427775978089411701e-03,6.025612483991197449e-03,1.414737555106548167e-02,3.099713520371227826e-02,1.787232034682838255e-02,2.780273651901068663e-02,4.813212618674997390e-03,-6.329722568148385409e-03,-1.125077725480938540e-02,-1.369731140331831992e-03,1.087832794979953777e-03,-2.679642885330540958e-03,-2.790833814376730837e-02,1.601304739543921857e-02,-3.086872566960111031e-02,-4.234111276481869468e-02,1.821257623938024595e-02,-2.175140748236847887e-02,-1.364019102841035336e-02,-2.428241640087944186e-02,5.924703926124395836e-03,-1.358936275448041631e-02,3.656349556138416279e-03,4.527174984150561521e-03,7.657987744259339702e-03,6.285099705324481700e-03,-3.069501498989768429e-03,5.156450412013553752e-02,-1.495451083979779106e-02,1.065959177316688766e-02,-3.937594404169728057e-03,-3.058567325434476125e-03,-6.862999663161464259e-01,9.108840697018756941e-03,-5.091712216361043986e-01,
1.102403388862267450e-02,-3.712175162329620949e-03,5.283598490532199522e-03,-1.712946864353653148e-02,-5.107848056885131924e-03,-1.472674502989296946e-02,-2.547979036260125674e-02,2.554563423686927702e-03,3.643264681289391446e-03,5.656749935752021230e-01,-2.416224182325339698e-02,-1.967489260937883497e-03,1.953549911264491901e-02,-3.037808456808966003e-03,-1.568189759462970158e-03,-3.010873552856536961e-02,1.056942579467191889e-02,4.002177927436047222e-03,2.677349505875581751e-03,3.398227499638521720e-02,9.857903503170037940e-01,-3.602918855804237291e-02,-7.673451288701929954e-03,1.888959718454505388e-02,1.893499716565773261e-02,-8.327388655289268832e-03,3.638191367634419256e-03,1.632226372732160924e-02,3.694231051712722723e-03,-1.182723348840351331e-02,1.695057195295324864e-02,8.947736005563510994e-03,-1.440610639166809911e-03,4.943783877280552227e-03,-1.901167343983457131e-02,5.471565900137757149e-03,3.058325228234317489e-03,-1.577366194962118856e-02,-7.035107566663100637e-03,-2.836070119951539931e-02,1.166125178241765765e-02,2.946065052567542648e-03,-2.435788322858329410e-03,1.269573181877672874e-02,3.521240647221948045e-03,-4.400309351896419807e-05,-6.761558875395794893e-05,3.619774322956112456e-03,-9.649281590500659658e-03,1.224334703483972303e-02,-1.884018519231960703e-03,-7.409203140683450622e-03,-1.789992741433951706e-02,1.513009425108021756e-02,-1.637193654555456346e-02,-5.105473097364245094e-02,5.318844019585029434e-03,-2.235717695404405500e-02,-1.431689721073109087e-02,-1.051258169810518305e-02,1.670664915423064611e-02,-1.942407341179691604e-02,3.418155777389440125e-04,5.821197924327867895e-03,7.070537180335383781e-03,-1.680691558621643517e-02,-1.426750654751573165e-02,3.276291201657749452e-02,-1.636479326673833529e-02,1.432367590069676816e-02,-3.588537607851907847e-03,-5.680910564277717387e-03,-6.976090928928173618e-01,1.208513463684883146e-02,-5.355846107588769778e-01,
-1.135441612898480113e-02,-2.753373586957194585e-03,1.967388301965160985e-02,2.395440356210920552e-03,-1.981417793545420941e-02,-1.548109908894487741e-02,-1.647297748825783670e-02,6.143813089909872655e-03,-1.072377258456149590e-02,6.736955711917336220e-01,2.420671855315831289e-03,1.741239390891034799e-02,6.885755425712764000e-04,1.050936382877599995e-02,-6.281062553215588373e-04,4.823526239594255749e-03,-5.004605555584133031e-03,-1.425431340605999575e-02,1.289095032371533923e-02,3.940590633618715199e-02,1.000000000000000000e+00,-3.486811172627576894e-02,-5.610383455732156910e-04,1.637006730231387042e-02,-5.043094460992769701e-03,-9.824664646420962211e-03,-1.384939718859352908e-02,3.313489984889478324e-03,-4.314484648657861306e-03,-2.408347090517622135e-02,1.267649971207195081e-02,-4.992885844025685138e-03,8.453203719050006448e-03,-1.783673594493999370e-02,-1.530933812185754361e-02,1.241174685538716689e-02,-7.229943488774228516e-04,7.170097495502972262e-03,1.421078227114923320e-03,-2.094583298223208490e-02,-1.888311966185053647e-02,-1.213648817374495675e-03,1.242712536030254888e-03,-4.730174785654961411e-03,3.240552868323635927e-03,1.714184735742118812e-02,-4.548713087707066671e-03,1.488274333753973611e-02,7.193510270751997393e-04,-2.002691144952689628e-03,-1.509186099488499244e-02,-9.135070546831536992e-03,3.673550731473046449e-03,2.057340602851578937e-03,4.811029468612948445e-04,5.388206254266281192e-03,-4.950907410372164252e-03,1.791246941818855354e-02,-2.901834402664910106e-02,-5.797424902260643609e-03,1.746545662885883379e-02,-8.966528455054384450e-03,2.948379874476037693e-03,-2.669793767521309868e-02,6.062681090135232931e-03,-3.930514611450927620e-03,-4.033505922479099087e-02,7.070137617540439567e-03,-2.809877284764731101e-02,-2.260510857807143670e-02,2.923768891214401089e-05,2.425181625186317858e-02,-7.463303607109323945e-01,7.891364246889544937e-03,-6.470957758534562343e-01,
-3.608013908454325847e-03,-6.408389166036801074e-03,3.177644186135527432e-03,-2.813359577613212120e-02,-2.886435552489923645e-02,-2.093886890302341583e-02,-1.088819643850227892e-02,2.464171866543542894e-02,-9.625526931368345862e-04,4.770960704570580968e-01,-4.749878354915068601e-03,2.456510997602365243e-02,-2.898688073105861601e-03,9.288104340875324647e-03,-4.835275571961571844e-03,2.228628957471595617e-02,-1.074568422093664932e-02,-1.438717800630529529e-02,2.267497709491991115e-02,2.679617218568559195e-02,9.886648150350345610e-01,-7.042156270721568190e-02,-2.538582139650109176e-02,1.426940842565344383e-02,6.978130005469374009e-03,-1.116106150031454664e-03,-4.219473343969382118e-02,5.033498143368474176e-03,-1.859815928188483444e-02,-6.486799441997936205e-03,1.765744289132178957e-02,-9.256555682055994660e-03,1.841926945135374805e-02,-1.858965243838203024e-02,-2.194243326683892326e-02,6.293888214658652919e-03,-3.377572080640302982e-02,9.361594867627758693e-03,2.102296840576502179e-02,-7.840645405929776157e-03,-3.672291065275011535e-02,8.948871482840974403e-03,2.047354501492713863e-02,1.035997250151853011e-02,-7.090787141600760068e-03,3.924744249207449964e-02,-1.834420323337448630e-03,2.374725897282838755e-02,2.844295490344948135e-02,7.138875284680988240e-03,-3.913275906008983202e-02,-2.163951371971415544e-02,-8.651831095097539670e-04,-1.483191570635934708e-02,-1.760971450445878925e-02,-7.510634089250289858e-04,1.675584753062985682e-02,2.508238789196678975e-02,-1.275817298738553783e-02,-5.582539610733915775e-03,1.511227454829517682e-02,5.779527284352874161e-03,1.560588560269363101e-02,-2.358349418834917305e-02,1.216796962191535325e-02,1.316115007027799863e-02,-7.390156004153719538e-02,2.369662363859722370e-02,-5.164428819019022121e-02,-2.641805224783062150e-02,-3.644218482086650397e-03,4.500344610629966602e-02,-5.779962646568935769e-01,-6.555189692693953740e-03,-4.733161947601379760e-01,
-7.799592473933802950e-05,4.105855646804601466e-04,-6.854099648053613159e-03,-6.489691320018087239e-03,1.005087865374704181e-02,8.570724459388834315e-03,-6.326114662769166674e-03,-1.297438657728533506e-02,-1.580068137777890236e-03,7.150286735126596405e-01,-7.917370320930428029e-03,-2.642494387815341739e-02,3.326312313611023974e-03,-4.584184450271283628e-03,-1.858114823379347635e-02,-1.374960723830988968e-02,-4.221201811979408136e-03,-3.190672103033920237e-04,1.688650904413752096e-03,1.020026475623631024e-02,1.000000000000000000e+00,-9.643456571983433090e-02,-3.583006118998546818e-03,1.090336927595253469e-02,1.101697872844437098e-02,-1.695893380451882204e-02,1.253167480219025132e-02,-3.047196278281427695e-03,9.268466335059510897e-04,1.124572077584150287e-02,6.354766394571775834e-04,8.649201627470494425e-03,-1.224778994995676107e-02,6.740444176612792049e-04,1.565533338938029928e-02,7.211334109800890402e-03,8.888880714203462041e-03,-8.829007971379490560e-03,9.038465300738526831e-03,-1.408507564548600044e-02,1.050323870201891330e-02,2.336458335126051500e-03,5.859799882381999669e-03,1.001483048437940004e-02,1.623939181225800277e-02,1.530678436120203045e-04,-1.144816349418542600e-03,2.493984072204954994e-03,-2.089460345360594218e-02,1.171965353233495044e-03,1.595323028517192654e-02,9.938319064496492494e-03,-2.005402491945029730e-03,1.702052971136416848e-02,3.495873223399170726e-04,-1.231053712244500886e-02,2.256242634072841424e-03,-1.015123853360225779e-02,6.895532772116075293e-03,-6.073833709905329670e-03,-7.877407854759987949e-03,-5.563612293306596790e-03,1.833387247649254093e-03,5.363713751337678692e-03,-3.506759862053343557e-03,1.146276986313990442e-03,-1.020793063973080006e-02,1.365662053014496391e-02,2.059342078605341797e-02,1.018178630447374999e-02,6.339970933847666190e-03,-2.508301992045836037e-02,-7.492953869352418916e-01,-3.405836766824932305e-04,-6.877086177756210894e-01,
-5.568124281959588306e-03,-1.818230626522336982e-02,4.167399846990091361e-04,-9.993463749171453345e-03,-1.942704585383157184e-02,-1.789334079731122115e-02,-1.720826511044087209e-02,2.031219630945985816e-02,-1.249783296601003494e-03,6.480690023483776097e-01,-2.347225709169795810e-02,1.666827830522989162e-02,-2.334110176120320068e-03,1.381089697861957732e-02,-1.192962460247047607e-03,1.589194867088864621e-02,6.591781389116518203e-03,-2.703858034965975904e-02,1.866709341718800555e-02,2.410750317402234749e-02,9.791597964529686893e-01,-5.583798838898872130e-02,-9.873790834933395336e-03,1.329919507323812713e-02,1.234946764816213793e-02,-5.741863284580638396e-03,-2.629091236487034189e-02,6.997300837848521166e-03,-2.047105332705882394e-02,-7.387569826876997509e-03,1.327870084381272200e-02,2.270254513294765394e-03,-3.951997969142078716e-03,1.211714477698723480e-03,-3.505089010928127968e-02,7.548285494839347513e-03,-2.763722172718264397e-02,7.115735481340035307e-03,8.273125633787792310e-03,-1.336148616492841695e-03,-2.447962141251587218e-02,-5.207820212062044297e-03,1.931527364748788361e-02,3.189610763315199951e-03,6.658858247725031598e-03,2.377638307077015178e-02,5.214770118001628083e-03,8.381718872121747890e-03,3.452793082746108411e-02,-2.091427035794714284e-04,-1.819663309672389651e-02,-2.840956425223205989e-02,-4.622041742525194452e-03,-1.316541958139344666e-02,-3.411540856350179368e-03,-1.351909638047534348e-02,-3.505963558043245255e-03,8.495216568885589556e-03,-1.017511418300772602e-02,2.516347084641559761e-03,1.775971796193081750e-02,-1.638210304150341384e-03,1.391416983853405696e-02,-2.149868000117095140e-02,7.571134878813834401e-03,5.994032567088039554e-03,-5.149533648199627195e-02,2.243163588440054076e-02,-3.746440680147595376e-02,-1.153142409066341273e-02,4.086571210900361191e-03,2.104325909010378026e-02,-6.595171923103337885e-01,-9.030267107652638348e-04,-6.379710102792885973e-01,
-1.260433717707625662e-02,-2.580909481595003441e-02,-2.660330631057528095e-03,-4.489459602412634864e-02,-2.773247251631578286e-02,-3.378253936327041407e-02,-3.920526318202846583e-02,4.865114526736991074e-04,1.074300931863486110e-02,1.290972325360109085e-01,-5.007733333940178611e-02,5.336795954168604278e-02,1.005893678152762483e-02,1.429462669280233238e-02,1.784486643155730815e-02,-1.602261177320318808e-04,8.301962336214205318e-03,-3.026202799811404665e-02,4.624311402428321616e-02,3.140070988070825520e-02,9.858516344252526986e-01,-6.727071443744719881e-02,-2.474113107937385425e-02,3.577004050984820827e-02,2.305617772313806046e-02,-1.987455506409324668e-02,-2.698458910808296946e-02,2.719474655191303492e-02,-4.052000394181355658e-02,-6.038955401094964898e-03,3.214476183525616371e-02,1.376054713034692567e-02,-1.156071968124119595e-02,-1.262419725051452599e-02,-5.525257545864317721e-02,1.147556574305975963e-02,-4.031499801164139390e-02,1.304566691889158525e-02,-6.259190665097280615e-03,-1.721390620330616461e-02,-4.020421553705817719e-02,7.772166257858349747e-03,1.315373255281924308e-02,2.279668879594728970e-02,9.935403278808083161e-03,5.518675456306672278e-02,3.587780996369843711e-02,1.601275825473787509e-02,3.688021480143385822e-02,-6.422971748394623592e-05,-3.911057491086797344e-02,-4.675345021044349791e-02,-2.829735005505103207e-02,-1.551574218176063336e-02,-2.362346898288595481e-02,-3.979184249625783432e-02,-7.238703609261619432e-03,5.070297432740705768e-03,-3.643543289205394831e-02,9.239442801425891416e-03,4.785726522584236864e-02,-7.327040771393369262e-03,2.674066003238129152e-02,-7.070157319288797315e-03,7.754842624325089528e-03,2.013215378961413782e-02,-6.706622910406320748e-02,1.646774718459267928e-02,-6.684169701463893443e-02,-4.822094545747714761e-02,1.210534422792358678e-02,7.298051983558544176e-02,-5.032052749873670017e-01,-5.247747964690716721e-03,1.159722325988388203e-02,
7.233052282308971570e-03,1.773424041788457250e-03,1.251808509150020843e-03,-6.907180267280325914e-02,-1.290824820472146611e-02,-6.596407317930118308e-02,-4.243214418037451197e-02,1.144821831109502086e-02,8.068361747912247348e-03,1.097899867495552573e-01,-5.039973433375004308e-02,3.614012338731453777e-02,2.804573985248626222e-02,-1.875784134692470850e-03,-1.987530025469341766e-02,-2.865159431895969444e-02,6.861146111912297396e-03,-3.161748310024636854e-02,5.570042317051529523e-02,9.379026306694679405e-02,1.000000000000000000e+00,-9.508819501282222975e-02,-2.030386551624997760e-03,3.977737834356862012e-02,6.575987664095458984e-02,4.274845249483072429e-03,6.115558789374642146e-04,3.674035700199008991e-02,-4.003190381630722383e-02,-5.347643181024981651e-02,1.857553316605498051e-02,1.912260824010379029e-02,-1.148711734193620312e-02,-1.806156213547496781e-02,-5.756015322619945085e-02,2.212838121149615644e-02,-1.432278773500245128e-02,-2.515409837128447512e-02,7.552049619064388208e-04,-3.530487401792563318e-02,-2.403821150380356861e-02,8.658888605122309901e-03,1.390121987494296468e-02,4.592982028465641964e-02,2.182590833621320114e-02,4.184201521980389360e-02,9.832644231718562500e-03,-1.583563162950557546e-02,2.398680514609421999e-02,-5.361677139672255121e-03,-2.814521193057473863e-02,-5.096207380025505329e-02,-4.079939419866983330e-02,1.226096149447074984e-02,-3.652245443461303587e-02,-8.837294446795099234e-02,6.985401303114174842e-03,-2.381678917117541205e-02,-5.307417749030376997e-02,-1.535445133800468487e-02,2.710178888205821979e-02,-3.550970767578420784e-02,6.583818688577880440e-03,-2.080141751726855004e-03,1.696122872945697579e-02,-4.064972263054675294e-03,-5.989649221748741259e-02,9.918975584613423657e-02,-5.604176378820507898e-02,1.456863564842086900e-02,-2.580609143719464482e-03,1.689075573229664728e-02,-3.148714247347614847e-01,-9.953483940619192727e-03,-7.509548105059385414e-02,
-1.007165490614234972e-02,1.175586923032465730e-03,7.733933313490929523e-03,1.150782825682281768e-02,3.458653743049676377e-03,-5.621933300287734747e-03,2.245569819189739691e-03,-4.131463680398843319e-03,-6.485206684480462197e-03,6.209622989914586055e-01,1.758110358903865883e-02,2.059616437131539524e-03,-1.851651867787526204e-02,-1.002153678610551851e-03,-1.657249526960273556e-02,1.647427445472626067e-02,-4.047546080520113970e-03,-2.178706676802880104e-02,1.474609359568218071e-02,2.660734223577272684e-03,9.729432463607075032e-01,-1.662984553632604334e-03,2.116128582603915564e-02,-1.151827583484987122e-03,-5.978877722975201037e-03,5.712576116826606140e-03,-2.201844963252286291e-02,-4.986230290465732545e-03,-1.916559440501469647e-02,-1.664704077278390815e-02,1.014744890839162049e-02,-1.761819130171886350e-02,1.235783621285576131e-02,-3.332350337591183723e-03,-1.586595853790937621e-02,1.358499622114463755e-02,-9.934292946952881509e-03,2.592967694348547250e-03,-7.128052361707735984e-03,9.414431596644116940e-03,-3.133232207594453117e-02,-5.300106014876388924e-03,5.997948953985997311e-04,7.000915663346967549e-03,-1.791670853723646012e-03,2.536406632062184960e-02,2.417088885056688924e-03,-1.825672341546673294e-03,2.766030876380770609e-02,-5.678553123564210325e-04,-2.168168186820972412e-02,-1.021015721449243518e-02,1.229486050493190999e-02,-3.835343873261099455e-03,3.217954894100084201e-03,1.735222715399420412e-02,-6.330231373866630762e-03,1.635317032580797456e-02,-9.264237316172457826e-03,1.539021142025157052e-03,-1.134197991405227340e-02,7.684940619245225757e-03,-1.731771261029333570e-03,-1.566582293078495197e-02,-1.063211651908502545e-02,1.982967877225986358e-02,-7.203607047542505415e-03,-1.758709522881521381e-02,-1.625615707892179296e-02,-2.277805334879343807e-02,1.270245091082163741e-02,2.280952513920608826e-02,-8.380488195549425923e-01,-1.737524132614701178e-03,-5.827563613616209004e-01,
-1.126080486760772797e-02,-8.937626671040270823e-03,2.573136430174177355e-02,-1.357125497322500540e-02,-4.748367250427050490e-02,-3.915831935850327267e-02,-4.559907473987538651e-02,2.859977398304584459e-02,-4.785784159047776209e-03,1.595295458201364536e-01,-1.857548638408530045e-02,6.283370989395234418e-02,1.671202570925281114e-02,2.721574404461151592e-02,1.942499738853114416e-02,9.346954519765110492e-04,1.352655449969717913e-02,-2.145285395401063527e-02,1.444759069414616390e-02,6.044926995446825435e-02,9.994837237811492470e-01,-7.597621515690905492e-02,-9.167831328216878573e-03,1.235838255228208400e-02,9.543930284855724894e-03,8.091487207326154146e-03,-3.085589145215147933e-02,3.846692157993870237e-02,-2.180946380276243407e-02,-3.964947284073570832e-02,2.817972898243028060e-02,6.838851239227106832e-03,5.865115500405105979e-03,-2.951607020658997271e-02,-5.324466776128014556e-02,1.216020663289615888e-04,-2.483920913236524475e-02,1.595380489062677570e-02,3.002651296328130714e-03,-4.113696134627656265e-02,-2.597898788135288156e-02,1.052566825080371131e-02,7.019514857557043741e-03,-2.151401587472491495e-03,2.975308222195444628e-04,2.442939963484574983e-02,2.514562440453026182e-03,2.748966605495854407e-02,2.857711106542424467e-02,-8.659588623534480306e-03,-4.445201035840307124e-02,-2.810579628207141023e-02,-1.085166065931763776e-02,1.458739515563939326e-04,-1.843752547564014160e-02,-2.019627241629530501e-02,-1.012363267434913949e-02,1.201088449844138484e-02,-5.099284212899615754e-02,-1.503265496743296653e-02,6.050073750614314017e-02,-3.365958029587338973e-02,-3.866346637032616433e-03,-3.646349216980934033e-02,1.655796271976893441e-02,-1.242910238469214709e-02,-6.974675030683283317e-02,1.547678698113050949e-02,-8.119509887047846042e-02,-2.892145548208714667e-02,-1.697124137395180504e-02,6.413714883918131271e-02,-3.728728764426341158e-01,1.702572216503340716e-02,-1.033875229251621786e-01,
8.958469981502154011e-03,-5.251339035173344799e-02,3.757039030964279269e-02,-3.358027154591900626e-02,-2.012010595368468602e-02,-3.850882584075746712e-02,-5.659318690225238063e-02,1.966131281948394621e-03,7.014277030179903900e-03,-3.115730053660409338e-02,-9.111490654147233470e-02,3.574523130558270728e-02,2.727415373702079399e-02,5.344918533069125885e-03,3.333704739494240676e-02,-1.291989615240735252e-02,3.334588350040423171e-02,-7.813957321633454092e-03,3.599885168607906000e-02,6.227102434639116002e-02,9.566160332153013091e-01,-1.082623554855867420e-01,-3.974920923273506101e-02,5.619007002131145428e-02,1.671738040206001419e-02,-3.656030423781152228e-02,-2.493025004911705172e-02,4.555584556442694721e-02,-1.958700298712506491e-02,-4.271087407114235179e-02,2.220704901950917853e-02,1.031499891283962217e-02,2.738777151839637269e-02,4.864024732827269826e-02,-8.398749284650228186e-02,1.560829859140105969e-02,-6.918314508283088422e-03,-1.817875357835942174e-02,-4.058391380963365797e-02,-1.484138256127446459e-02,-3.570213536701848100e-02,3.553775353296748790e-02,-2.630003430135213413e-02,-6.010776999842695409e-04,-5.314944377254246337e-03,2.784872361930785037e-02,2.368355980148511078e-02,-5.294692454946755764e-03,4.156744439953627412e-02,3.256098964649166427e-02,-5.100987550671384069e-02,-9.715724105805086108e-02,-3.839258376951578539e-02,3.854849329153913363e-03,-3.237975634484301091e-02,-8.861233811702401431e-02,-6.270735414698669005e-04,-1.221666083668730696e-02,-3.747253192702678509e-02,2.103241599685715113e-02,4.836332550161387411e-02,-2.989442145900178191e-02,4.828268322363862136e-02,-5.311869893999624642e-04,1.622491099455239394e-02,-4.224830744128822702e-02,-1.277835061404287154e-01,3.338522073109284616e-02,-1.055052905817938508e-01,-7.548958074470160651e-03,3.196400461173790064e-02,2.790546687605917342e-02,-2.493808515549571225e-01,1.050411548948762194e-02,3.840621617347543537e-02,
2.630049410799464166e-02,-3.103185436236993201e-03,5.926699887877561086e-03,-2.953505920162464299e-02,-3.690120578456822925e-02,-4.272164365936556574e-02,-3.777468571044743234e-02,3.145980365532702328e-02,-4.007079290832583039e-03,3.839245761163345705e-01,-4.591794991654973845e-02,2.155941596889798112e-02,2.956892248555654731e-02,-6.152930445851695004e-03,-9.142085739890056656e-03,-2.521257382193280855e-02,5.161823696942815370e-03,-2.206901917978293035e-03,3.630446014495337503e-03,4.139964824765756529e-02,1.000000000000000000e+00,-7.508693501776692036e-02,-1.241897682300061555e-03,2.568669656993079892e-02,4.057485899990711381e-02,7.825621047059501748e-03,4.070422916328444302e-04,2.292026472423901520e-02,1.133589280186593495e-04,-1.348073836867317091e-02,3.738570287106521284e-02,-3.990507234776603449e-03,-1.096507794413008677e-02,-9.602529559478517607e-03,-4.800973839816101429e-02,6.539030668694148160e-03,-1.681578376166263655e-02,-1.278996291412403188e-02,7.192870473162854897e-03,-2.570334642136783490e-02,1.116119949336409276e-02,-8.793331452883008589e-03,2.599798041690711339e-02,2.316164833387725566e-02,7.224258421910064940e-03,1.514326558967643739e-02,4.183598405521271255e-03,9.351051855141616354e-03,-2.161158881317829308e-03,-7.804137307708014504e-03,-1.853326069903820511e-02,-1.060492769170529384e-02,-3.060222501494361952e-02,9.976689988201156711e-03,-3.211089455643625218e-02,-6.025984969928722956e-02,1.298582480689396308e-02,-1.464819146822123053e-02,-2.024415350809013331e-02,-2.039589292262219619e-02,3.033377470038315510e-02,-1.873744652807344743e-02,9.020263918400606151e-04,-4.983955601159360121e-03,2.177705092050066793e-02,-2.063541178708656887e-02,-3.958629911221737963e-02,7.029651906439099429e-02,-4.299583912234746647e-02,1.220784322140142444e-02,-2.102981729602114089e-02,5.177309954014396726e-03,-4.643464109092658720e-01,1.985023515304427943e-02,-3.431165363895959941e-01,
-1.274269698202081663e-02,-1.235076436442408832e-02,9.999642810085997841e-03,8.067971286850224300e-05,-3.154779015998473990e-02,-3.383060457590360975e-02,-2.598129980608540612e-02,2.650874148800400065e-02,-1.304893327767954123e-02,5.259780323939723390e-01,-1.411840428084665255e-02,4.002290686724280927e-02,2.308175640593872202e-03,1.246392363119699920e-02,-4.424933267963712669e-03,1.496833445447703914e-02,1.821710060664440134e-02,-2.903217004252503855e-02,2.033347757399136396e-02,4.000686688479976327e-02,9.366941551578100711e-01,-6.392565921802476070e-02,-9.826743297665578580e-03,1.152620067346285700e-02,-1.003409060938917352e-04,2.604288679683451624e-03,-3.122320537989193082e-02,8.486719538194839596e-03,-1.121845390315647298e-02,-3.454173581189918379e-02,1.230566109296127308e-02,-1.056266184182238360e-02,1.127316811586057815e-02,-2.094662926362013400e-02,-4.699850329587855480e-02,-1.266708698199725536e-03,-3.136701723989615553e-02,3.814122538478265101e-03,6.021375568303195119e-03,-1.183595873193149170e-02,-3.149896195239910990e-02,-1.196788098569230227e-02,1.315974252663115046e-02,-7.273707618256762542e-03,-9.148213038962869914e-03,1.046633336602741798e-02,4.123522380427069312e-03,1.683310557391461126e-02,3.670301722432597197e-02,-4.193240082047897110e-04,-2.603336914492350440e-02,-3.082681419984313609e-02,1.932992256475914855e-03,-1.938885299427577091e-02,1.480844827293499588e-03,-1.006256388879621377e-02,-4.652556365187313081e-03,1.960327681875325898e-02,-2.100755379208877130e-02,-3.114750738644898181e-03,2.523789699209602611e-02,-9.439665962313326109e-03,8.193360704475703693e-03,-2.619485037113988141e-02,3.935796443944561754e-03,-7.328070193497224216e-03,-6.095165610976394088e-02,2.073187113046284086e-02,-5.071608924368660454e-02,-1.374083496899109608e-02,-7.630301106655388438e-03,3.185296118940622589e-02,-6.055011678957349375e-01,7.575749992101317443e-04,-5.110138875500220834e-01




])

a,b = tdcs_function_vol(x)

np.save("checklzm/all_tacs_"+model+label+".npy",b)
np.save("checklzm/target_tacs_"+model+label+".npy",a)
print("done")


NameError: name 'pos' is not defined